In [1]:
import pandas as pd
import psycopg2	
con = psycopg2.connect(dbname='mimic', user='postgres', password='postgres' )
query_schema = 'set search_path to ' + 'mimiciii'+ ';'
cur=con.cursor()

In [2]:
# labodata from labevents table
query_lab =query_schema+"""
drop materialized view if exists labo CASCADE; 
create materialized view labo as
SELECT *, case
            when itemid = 51006 then 'BUN'
            when itemid = 50806 then 'CHLORIDE'
            when itemid = 50902 then 'CHLORIDE'
            when itemid = 50912 then 'CREATININE'
            when itemid = 50811 then 'HEMOGLOBIN'
            when itemid = 51222 then 'HEMOGLOBIN'
            when itemid = 51265 then 'PLATELET'
            when itemid = 50822 then 'POTASSIUM'
            when itemid = 50971 then 'POTASSIUM'
            when itemid = 50824 then 'SODIUM'
            when itemid = 50983 then 'SODIUM'
            when itemid = 50803 then 'BICARBONATE' -- LABEL is 'TOTALCO2'
            when itemid = 50882 then 'BICARBONATE' -- LABEL is 'TOTALCO2'
            when itemid = 50804 then 'TOTALCO2'
            when itemid = 51300 then 'WBC'
            when itemid = 51301 then 'WBC'
            when itemid = 50802 then 'BASEEXCESS'
          else null
        end as label
FROM labevents
where itemid in
  (
     51300,51301 -- wbc
    ,50811,51222 -- hgb
    ,51265 -- platelet
    ,50824, 50983 -- sodium
    ,50822, 50971 -- potassium
    ,50804 -- Total CO2 or ...
    ,50803, 50882  -- bicarbonate
    ,50806,50902 -- chloride
    ,51006 -- bun
    ,50912 -- creatinine
    ,50802 -- base excess
  )
"""
cur.execute(query_lab)

In [3]:
# survival, icu stay etc from admissions, patients, and icustays tables
query_icustay_detail =query_schema+"""
drop materialized view if exists icustay_detail CASCADE; 
create materialized view icustay_detail as
SELECT ie.subject_id, ie.hadm_id, ie.icustay_id

-- patient level factors
, pat.gender, pat.dod, pat.dod_hosp, pat.expire_flag
-- dod_hosp: date of death in hospital
-- dod: date of death in hospital and/or SSN
-- expire_flag: death in hospital and/or SSN

-- hospital level factors
, adm.admittime, adm.dischtime
, ROUND( (CAST(EXTRACT(epoch FROM adm.dischtime - adm.admittime)/(60*60*24) AS numeric)), 4) AS los_hospital
, ROUND( (CAST(EXTRACT(epoch FROM adm.admittime - pat.dob)/(60*60*24*365.242) AS numeric)), 4) AS admission_age
, adm.ethnicity, adm.admission_type
, adm.hospital_expire_flag -- death in hospital?
, DENSE_RANK() OVER (PARTITION BY adm.subject_id ORDER BY adm.admittime) AS hospstay_seq
, CASE
    WHEN DENSE_RANK() OVER (PARTITION BY adm.subject_id ORDER BY adm.admittime) = 1 THEN True
    ELSE False END AS first_hosp_stay

-- icu level factors
, ie.intime, ie.outtime
, ROUND( (CAST(EXTRACT(epoch FROM ie.outtime - ie.intime)/(60*60*24) AS numeric)), 4) AS los_icu
, DENSE_RANK() OVER (PARTITION BY ie.hadm_id ORDER BY ie.intime) AS icustay_seq

-- first ICU stay *for the current hospitalization*
, CASE
    WHEN DENSE_RANK() OVER (PARTITION BY ie.hadm_id ORDER BY ie.intime) = 1 THEN True
    ELSE False END AS first_icu_stay

FROM icustays ie
INNER JOIN admissions adm
    ON ie.hadm_id = adm.hadm_id
INNER JOIN patients pat
    ON ie.subject_id = pat.subject_id
ORDER BY ie.subject_id, adm.admittime, ie.intime
-- admittime: to hospital, intime: to ICU
-- remove WHERE adm.has_chartevents_data = 1
"""
cur.execute(query_icustay_detail)
con.commit()

In [4]:
# urineoutput table
query_urineoutput =query_schema+"""
-- First we drop the table if it exists
DROP MATERIALIZED VIEW IF EXISTS urineoutput CASCADE;
CREATE MATERIALIZED VIEW urineoutput as
select oe.icustay_id, oe.charttime
, SUM(
    -- we consider input of GU irrigant as a negative volume
    case when oe.itemid = 227488 then -1*value
    else value end
  ) as value
from outputevents oe
where oe.itemid in
(
  -- these are the most frequently occurring urine output observations in CareVue
  40055, -- "Urine Out Foley"
  43175, -- "Urine ."
  40069, -- "Urine Out Void"
  40094, -- "Urine Out Condom Cath"
  40715, -- "Urine Out Suprapubic"
  40473, -- "Urine Out IleoConduit"
  40085, -- "Urine Out Incontinent"
  40057, -- "Urine Out Rt Nephrostomy"
  40056, -- "Urine Out Lt Nephrostomy"
  40405, -- "Urine Out Other"
  40428, -- "Urine Out Straight Cath"
  40086,--	Urine Out Incontinent
  40096, -- "Urine Out Ureteral Stent #1"
  40651, -- "Urine Out Ureteral Stent #2"

  -- these are the most frequently occurring urine output observations in MetaVision
  226559, -- "Foley"
  226560, -- "Void"
  226561, -- "Condom Cath"
  226584, -- "Ileoconduit"
  226563, -- "Suprapubic"
  226564, -- "R Nephrostomy"
  226565, -- "L Nephrostomy"
  226567, --	Straight Cath
  226557, -- R Ureteral Stent
  226558, -- L Ureteral Stent
  227488, -- GU Irrigant Volume In
  227489  -- GU Irrigant/Urine Volume Out
)
and oe.value < 5000 -- sanity check on urine value
and oe.icustay_id is not null
group by icustay_id, charttime;
"""
cur.execute(query_urineoutput)

In [5]:
# echodata table
query_echodata =query_schema+"""
-- This code extracts structured data from echocardiographies
-- You can join it to the text notes using ROW_ID
-- Just note that ROW_ID will differ across versions of MIMIC-III.

DROP MATERIALIZED VIEW IF EXISTS ECHODATA CASCADE;
CREATE MATERIALIZED VIEW ECHODATA AS
select ROW_ID
  , subject_id, hadm_id
  , chartdate

  -- charttime is always null for echoes..
  -- however, the time is available in the echo text, e.g.:
  -- , substring(ne.text, 'Date/Time: [\[\]0-9*-]+ at ([0-9:]+)') as TIMESTAMP
  -- we can therefore impute it and re-create charttime
  , cast(to_timestamp( (to_char( chartdate, 'DD-MM-YYYY' ) || substring(ne.text, 'Date/Time: [\[\]0-9*-]+ at ([0-9:]+)')),
            'DD-MM-YYYYHH24:MI') as timestamp without time zone)
    as charttime
    
  , substring(ne.text, 'Indication: (.*?)\n') as Indication

  -- sometimes numeric values contain de-id text, e.g. [** Numeric Identifier **]
  -- this removes that text
  , case
      when substring(ne.text, 'Height: \(in\) (.*?)\n') like '%*%'
        then null
      else cast(substring(ne.text, 'Height: \(in\) (.*?)\n') as numeric)
    end as Height

  , case
      when substring(ne.text, 'Weight \(lb\): (.*?)\n') like '%*%'
        then null
      else cast(substring(ne.text, 'Weight \(lb\): (.*?)\n') as numeric)
    end as Weight

  , case
      when substring(ne.text, 'BSA \(m2\): (.*?) m2\n') like '%*%'
        then null
      else cast(substring(ne.text, 'BSA \(m2\): (.*?) m2\n') as numeric)
    end as BSA -- ends in 'm2'

  , substring(ne.text, 'BP \(mm Hg\): (.*?)\n') as BP -- Sys/Dias

  , case
      when substring(ne.text, 'BP \(mm Hg\): ([0-9]+)/[0-9]+?\n') like '%*%'
        then null
      else cast(substring(ne.text, 'BP \(mm Hg\): ([0-9]+)/[0-9]+?\n') as numeric)
    end as BPSys -- first part of fraction

  , case
      when substring(ne.text, 'BP \(mm Hg\): [0-9]+/([0-9]+?)\n') like '%*%'
        then null
      else cast(substring(ne.text, 'BP \(mm Hg\): [0-9]+/([0-9]+?)\n') as numeric)
    end as BPDias -- second part of fraction

  , case
      when substring(ne.text, 'HR \(bpm\): ([0-9]+?)\n') like '%*%'
        then null
      else cast(substring(ne.text, 'HR \(bpm\): ([0-9]+?)\n') as numeric)
    end as HR

  , substring(ne.text, 'Status: (.*?)\n') as Status
  , substring(ne.text, 'Test: (.*?)\n') as Test
  , substring(ne.text, 'Doppler: (.*?)\n') as Doppler
  , substring(ne.text, 'Contrast: (.*?)\n') as Contrast
  , substring(ne.text, 'Technical Quality: (.*?)\n') as TechnicalQuality
from noteevents ne
where category = 'Echo';
"""
cur.execute(query_echodata)
con.commit()

In [6]:
# weightdurations table using echodata table
query_weightdurations =query_schema+"""
-- This query extracts weights for adult ICU patients with start/stop times
-- if an admission weight is given, then this is assigned from intime to outtime

DROP MATERIALIZED VIEW IF EXISTS weightdurations CASCADE;
CREATE MATERIALIZED VIEW weightdurations as

-- This query extracts weights for adult ICU patients with start/stop times
-- if an admission weight is given, then this is assigned from intime to outtime
with wt_stg as
(
    SELECT
        c.icustay_id
      , c.charttime
      , case when c.itemid in (762,226512) then 'admit'
          else 'daily' end as weight_type
      -- TODO: eliminate obvious outliers if there is a reasonable weight
      , c.valuenum as weight
    FROM chartevents c
    WHERE c.valuenum IS NOT NULL
      AND c.itemid in
      (
         762,226512 -- Admit Wt
        ,763,224639 -- Daily Weight
      )
      AND c.valuenum != 0
      -- exclude rows marked as error
      AND c.error IS DISTINCT FROM 1
)
-- assign ascending row number
, wt_stg1 as
(
  select
      icustay_id
    , charttime
    , weight_type
    , weight
    , ROW_NUMBER() OVER (partition by icustay_id, weight_type order by charttime) as rn
  from wt_stg
)
-- change charttime to starttime - for admit weight, we use ICU admission time
, wt_stg2 as
(
  select
      wt_stg1.icustay_id
    , ie.intime, ie.outtime
    , case when wt_stg1.weight_type = 'admit' and wt_stg1.rn = 1
        then ie.intime - interval '2' hour
      else wt_stg1.charttime end as starttime
    , wt_stg1.weight
  from icustays ie
  inner join wt_stg1
    on ie.icustay_id = wt_stg1.icustay_id
  where not (weight_type = 'admit' and rn = 1)
)
, wt_stg3 as
(
  select
    icustay_id
    , starttime
    , coalesce(
        LEAD(starttime) OVER (PARTITION BY icustay_id ORDER BY starttime),
        outtime + interval '2' hour
      ) as endtime
    , weight
  from wt_stg2
)
-- this table is the start/stop times from admit/daily weight in charted data
, wt1 as
(
  select
      ie.icustay_id
    , wt.starttime
    , case when wt.icustay_id is null then null
      else
        coalesce(wt.endtime,
        LEAD(wt.starttime) OVER (partition by ie.icustay_id order by wt.starttime),
          -- we add a 2 hour "fuzziness" window
        ie.outtime + interval '2' hour)
      end as endtime
    , wt.weight
  from icustays ie
  left join wt_stg3 wt
    on ie.icustay_id = wt.icustay_id
)
-- if the intime for the patient is < the first charted daily weight
-- then we will have a "gap" at the start of their stay
-- to prevent this, we look for these gaps and backfill the first weight
-- this adds (153255-149657)=3598 rows, meaning this fix helps for up to 3598 icustay_id
, wt_fix as
(
  select ie.icustay_id
    -- we add a 2 hour "fuzziness" window
    , ie.intime - interval '2' hour as starttime
    , wt.starttime as endtime
    , wt.weight
  from icustays ie
  inner join
  -- the below subquery returns one row for each unique icustay_id
  -- the row contains: the first starttime and the corresponding weight
  (
    select wt1.icustay_id, wt1.starttime, wt1.weight
    from wt1
    inner join
      (
        select icustay_id, min(Starttime) as starttime
        from wt1
        group by icustay_id
      ) wt2
    on wt1.icustay_id = wt2.icustay_id
    and wt1.starttime = wt2.starttime
  ) wt
    on ie.icustay_id = wt.icustay_id
    and ie.intime < wt.starttime
)
, wt2 as
(
  select
      wt1.icustay_id
    , wt1.starttime
    , wt1.endtime
    , wt1.weight
  from wt1
  UNION
  SELECT
      wt_fix.icustay_id
    , wt_fix.starttime
    , wt_fix.endtime
    , wt_fix.weight
  from wt_fix
)
-- get more weights from echo - completes data for ~2500 patients
-- we only use echo data if there is *no* charted data
-- we impute the median echo weight for their entire ICU stay
-- only ~762 patients remain with no weight data
, echo_lag as
(
  select
    ie.icustay_id
    , ie.intime, ie.outtime
    , 0.453592*ec.weight as weight_echo
    , ROW_NUMBER() OVER (PARTITION BY ie.icustay_id ORDER BY ec.charttime) as rn
    , ec.charttime as starttime
    , LEAD(ec.charttime) OVER (PARTITION BY ie.icustay_id ORDER BY ec.charttime) as endtime
  from icustays ie
  inner join echodata ec
      on ie.hadm_id = ec.hadm_id
  where ec.weight is not null
)
, echo_final as
(
    select
        el.icustay_id
        , el.starttime
          -- we add a 2 hour "fuzziness" window
        , coalesce(el.endtime,el.outtime + interval '2' hour) as endtime
        , weight_echo
    from echo_lag el
    UNION
    -- if the starttime was later than ICU admission, back-propogate the weight
    select
      el.icustay_id
      , el.intime - interval '2' hour as starttime
      , el.starttime as endtime
      , el.weight_echo
    from echo_lag el
    where el.rn = 1
    and el.starttime > el.intime - interval '2' hour
)
select
  wt2.icustay_id, wt2.starttime, wt2.endtime, wt2.weight
from wt2
UNION
-- only add echos if we have no charted weight data
select
  ef.icustay_id, ef.starttime, ef.endtime, ef.weight_echo as weight
from echo_final ef
where ef.icustay_id not in (select distinct icustay_id from wt2)
order by icustay_id, starttime, endtime;
"""
cur.execute(query_weightdurations)

In [7]:
# kdigo_uo table using urineouput and weightdurations table
query_kdigo_uo =query_schema+"""
DROP MATERIALIZED VIEW IF EXISTS kdigo_uo CASCADE;
CREATE MATERIALIZED VIEW kdigo_uo AS
with ur_stg as
(
  select io.icustay_id, io.charttime

  -- three sums:
  -- 1) over a 6 hour period
  -- 2) over a 12 hour period
  -- 3) over a 24 hour period
  , sum(case when iosum.charttime <= io.charttime + interval '5' hour
      then iosum.VALUE
    else null end) as UrineOutput_6hr
  , sum(case when iosum.charttime <= io.charttime + interval '11' hour
      then iosum.VALUE
    else null end) as UrineOutput_12hr
  , sum(iosum.VALUE) as UrineOutput_24hr
  from urineoutput io
  -- this join gives you all UO measurements over a 24 hour period
  left join urineoutput iosum
    on  io.icustay_id = iosum.icustay_id
    and iosum.charttime >=  io.charttime
    and iosum.charttime <= (io.charttime + interval '23' hour)
  group by io.icustay_id, io.charttime
)
select
  ur.icustay_id
, ur.charttime
, wd.weight
, ur.UrineOutput_6hr
, ur.UrineOutput_12hr
, ur.UrineOutput_24hr
from ur_stg ur
left join weightdurations wd
  on  ur.icustay_id = wd.icustay_id
  and ur.charttime >= wd.starttime
  and ur.charttime <  wd.endtime
order by icustay_id, charttime;
"""
cur.execute(query_kdigo_uo)

In [8]:
query_kdigo_creat =query_schema+"""
DROP MATERIALIZED VIEW IF EXISTS kdigo_creat CASCADE;
CREATE MATERIALIZED VIEW kdigo_creat as
-- Extract all creatinine values from labevents around patient's ICU stay
with cr as
(
select
    ie.icustay_id
  , ie.intime, ie.outtime
  , le.valuenum as creat
  , le.charttime
  from icustays ie
  left join labevents le
    on ie.subject_id = le.subject_id
    and le.ITEMID = 50912
    and le.VALUENUM is not null
    and le.CHARTTIME between (ie.intime - interval '6' hour) and (ie.intime + interval '7' day)
)
-- ***** --
-- Get the highest and lowest creatinine for the first 48 hours of ICU admission
-- also get the first creatinine
-- ***** --
, cr_48hr as
(
select
    cr.icustay_id
  , cr.creat
  , cr.charttime
  -- Create an index that goes from 1, 2, ..., N
  -- The index represents how early in the patient's stay a creatinine value was measured
  -- Consequently, when we later select index == 1, we only select the first (admission) creatinine
  -- In addition, we only select the first stay for the given subject_id
  , ROW_NUMBER ()
          OVER (PARTITION BY cr.icustay_id
                ORDER BY cr.charttime
              ) as rn_first

  -- Similarly, we can get the highest and the lowest creatinine by ordering by VALUENUM
  , ROW_NUMBER ()
          OVER (PARTITION BY cr.icustay_id
                ORDER BY cr.creat DESC
              ) as rn_highest
  , ROW_NUMBER ()
          OVER (PARTITION BY cr.icustay_id
                ORDER BY cr.creat
              ) as rn_lowest
  from cr
  -- limit to the first 48 hours (source table has data up to 7 days)
  where cr.charttime <= cr.intime + interval '48' hour
)
-- ***** --
-- Get the highest and lowest creatinine for the first 7 days of ICU admission
-- ***** --
, cr_7day as
(
select
    cr.icustay_id
  , cr.creat
  , cr.charttime
  -- We can get the highest and the lowest creatinine by ordering by VALUENUM
  , ROW_NUMBER ()
          OVER (PARTITION BY cr.icustay_id
                ORDER BY cr.creat DESC
              ) as rn_highest
  , ROW_NUMBER ()
          OVER (PARTITION BY cr.icustay_id
                ORDER BY cr.creat
              ) as rn_lowest
  from cr
)
-- ***** --
-- Final query
-- ***** --
select
    ie.subject_id, ie.hadm_id, ie.icustay_id
  , cr_48hr_admit.creat as AdmCreat
  , cr_48hr_admit.charttime as AdmCreatTime
  , cr_48hr_low.creat as LowCreat48hr
  , cr_48hr_low.charttime as LowCreat48hrTime
  , cr_48hr_high.creat as HighCreat48hr
  , cr_48hr_high.charttime as HighCreat48hrTime

  , cr_7day_low.creat as LowCreat7day
  , cr_7day_low.charttime as LowCreat7dayTime
  , cr_7day_high.creat as HighCreat7day
  , cr_7day_high.charttime as HighCreat7dayTime

from icustays ie
left join cr_48hr cr_48hr_admit
  on ie.icustay_id = cr_48hr_admit.icustay_id
  and cr_48hr_admit.rn_first = 1
left join cr_48hr cr_48hr_high
  on ie.icustay_id = cr_48hr_high.icustay_id
  and cr_48hr_high.rn_highest = 1
left join cr_48hr cr_48hr_low
  on ie.icustay_id = cr_48hr_low.icustay_id
  and cr_48hr_low.rn_lowest = 1
left join cr_7day cr_7day_high
  on ie.icustay_id = cr_7day_high.icustay_id
  and cr_7day_high.rn_highest = 1
left join cr_7day cr_7day_low
  on ie.icustay_id = cr_7day_low.icustay_id
  and cr_7day_low.rn_lowest = 1
order by ie.icustay_id;
"""
cur.execute(query_kdigo_creat)
con.commit()

In [9]:
query_kdigo_stages_7day =query_schema+"""
-- This query checks if the patient had AKI according to KDIGO on admission
-- AKI can be defined either using data from the first 2 days, or first 7 days

-- For urine output: the highest UO in hours 0-48 is used
-- For creatinine: the creatinine value from days 0-2 or 0-7 is used.
-- Baseline creatinine is defined as first measurement in hours [-6, 24] from ICU admit

DROP MATERIALIZED VIEW IF EXISTS kdigo_stages_7day CASCADE;
CREATE MATERIALIZED VIEW kdigo_stages_7day AS
with uo_6hr as
(
  select
        ie.icustay_id
      -- , uo.charttime
      -- , uo.urineoutput_6hr
      , min(uo.urineoutput_6hr / uo.weight / 6.0)::numeric as uo6
  from icustays ie
  inner join kdigo_uo uo
    on ie.icustay_id = uo.icustay_id
    and uo.charttime <= ie.intime + interval '7' day - interval '6' hour
  group by ie.icustay_id
)
, uo_12hr as
(
  select
      ie.icustay_id
      -- , uo.charttime
      -- , uo.weight
      -- , uo.urineoutput_12hr
      , min(uo.urineoutput_12hr / uo.weight / 12.0)::numeric as uo12
  from icustays ie
  inner join kdigo_uo uo
    on ie.icustay_id = uo.icustay_id
    and uo.charttime <= ie.intime + interval '7' day - interval '12' hour
  group by ie.icustay_id
)
, uo_24hr as
(
  select
      ie.icustay_id
      -- , uo.charttime
      -- , uo.weight
      -- , uo.urineoutput_24hr
      , min(uo.urineoutput_24hr / uo.weight / 24.0)::numeric as uo24
  from icustays ie
  inner join kdigo_uo uo
    on ie.icustay_id = uo.icustay_id
    and uo.charttime <= ie.intime + interval '7' day - interval '24' hour
  group by ie.icustay_id
)
-- stages for UO / creat
, kdigo_stg as
(

  select ie.icustay_id
  , ie.intime, ie.outtime
  , case
    when HighCreat7day >= (LowCreat7day*3.0) then 3
    when HighCreat7day >= 4 -- note the criteria specify an INCREASE to >=4
      and LowCreat7day <= (3.7)  then 3 -- therefore we check that adm <= 3.7
    -- TODO: initiation of RRT
    when HighCreat7day >= (LowCreat7day*2.0) then 2
    when HighCreat7day >= (LowCreat7day+0.3) then 1
    when HighCreat7day >= (LowCreat7day*1.5) then 1
    when HighCreat7day is null then null
      when LowCreat7day is null then null
    else 0 end as AKI_stage_7day_creat

  -- AKI stages according to urine output
  , case
      when UO24 < 0.3 then 3
      when UO12 = 0 then 3
      when UO12 < 0.5 then 2
      when UO6  < 0.5 then 1
      when UO6  is null then null
    else 0 end as AKI_stage_7day_uo

  -- Creatinine information
  , LowCreat7dayTime, LowCreat7day
  , HighCreat7dayTime, HighCreat7day

  -- Urine output information: the values and the time of their measurement
  , round(UO6,4) as UO6_48hr
  , round(UO12,4) as UO12_48hr
  , round(UO24,4) as UO24_48hr
  from icustays ie
  left join uo_6hr  on ie.icustay_id = uo_6hr.icustay_id
  left join uo_12hr on ie.icustay_id = uo_12hr.icustay_id
  left join uo_24hr on ie.icustay_id = uo_24hr.icustay_id
  left join KDIGO_CREAT cr on ie.icustay_id = cr.icustay_id
)
select
  kd.icustay_id

  -- Classify AKI using both creatinine/urine output criteria
  , case
      when coalesce(AKI_stage_7day_creat,AKI_stage_7day_uo) > 0 then 1
      else coalesce(AKI_stage_7day_creat,AKI_stage_7day_uo)
    end as AKI_7day

  , case
      when AKI_stage_7day_creat >= AKI_stage_7day_uo then AKI_stage_7day_creat
      when AKI_stage_7day_uo > AKI_stage_7day_creat then AKI_stage_7day_uo
      else coalesce(AKI_stage_7day_creat,AKI_stage_7day_uo)
    end as AKI_stage_7day

  , AKI_stage_7day_creat

  -- Creatinine information - convert absolute times to hours since admission
  , LowCreat7day
  , HighCreat7day
  , ROUND(extract(epoch from (LowCreat7dayTime-intime))::numeric / 60.0 / 60.0 / 24.0, 4) as LowCreat7dayTimeElapsed
  , ROUND(extract(epoch from (HighCreat7dayTime-intime))::numeric / 60.0 / 60.0 / 24.0, 4) as HighCreat7dayTimeElapsed
  , LowCreat7dayTime
  , HighCreat7dayTime

  -- Urine output information: the values and the time of their measurement
  , UO6_48hr
  , UO12_48hr
  , UO24_48hr
from kdigo_stg kd
order by kd.icustay_id;
"""
cur.execute(query_kdigo_stages_7day)
con.commit()

In [10]:
query_rrt2day= query_schema+"""
DROP MATERIALIZED VIEW IF EXISTS rrt2day CASCADE;
CREATE MATERIALIZED VIEW rrt2day as
with cv as
(
  select ie.icustay_id
    , max(
        case
          when ce.itemid in (152,148,149,146,147,151,150) and value is not null then 1
          when ce.itemid in (229,235,241,247,253,259,265,271) and value = 'Dialysis Line' then 1
          when ce.itemid = 582 and value in ('CAVH Start','CAVH D/C','CVVHD Start','CVVHD D/C','Hemodialysis st','Hemodialysis end') then 1
        else 0 end
        ) as RRT
  from icustays ie
  inner join chartevents ce
    on ie.icustay_id = ce.icustay_id
    and ce.itemid in
    (
       152 -- "Dialysis Type";61449
      ,148 -- "Dialysis Access Site";60335
      ,149 -- "Dialysis Access Type";60030
      ,146 -- "Dialysate Flow ml/hr";57445
      ,147 -- "Dialysate Infusing";56605
      ,151 -- "Dialysis Site Appear";37345
      ,150 -- "Dialysis Machine";27472
      ,229 -- INV Line#1 [Type]
      ,235 -- INV Line#2 [Type]
      ,241 -- INV Line#3 [Type]
      ,247 -- INV Line#4 [Type]
      ,253 -- INV Line#5 [Type]
      ,259 -- INV Line#6 [Type]
      ,265 -- INV Line#7 [Type]
      ,271 -- INV Line#8 [Type]
      ,582 -- Procedures
    )
    and ce.value is not null
    and ce.charttime between ie.intime and ie.intime + interval '2' day
  where ie.dbsource = 'carevue'
  group by ie.icustay_id
)
, mv_ce as
(
  select ie.icustay_id
    , 1 as RRT
  from icustays ie
  inner join chartevents ce
    on ie.icustay_id = ce.icustay_id
    and ce.charttime between ie.intime and ie.intime + interval '2' day
    and itemid in
    (
      -- Checkboxes
        226118 -- | Dialysis Catheter placed in outside facility      | Access Lines - Invasive | chartevents        | Checkbox
      , 227357 -- | Dialysis Catheter Dressing Occlusive              | Access Lines - Invasive | chartevents        | Checkbox
      , 225725 -- | Dialysis Catheter Tip Cultured                    | Access Lines - Invasive | chartevents        | Checkbox
      -- Numeric values
      , 226499 -- | Hemodialysis Output                               | Dialysis                | chartevents        | Numeric
      , 224154 -- | Dialysate Rate                                    | Dialysis                | chartevents        | Numeric
      , 225810 -- | Dwell Time (Peritoneal Dialysis)                  | Dialysis                | chartevents        | Numeric
      , 227639 -- | Medication Added Amount  #2 (Peritoneal Dialysis) | Dialysis                | chartevents        | Numeric
      , 225183 -- | Current Goal                     | Dialysis | chartevents        | Numeric
      , 227438 -- | Volume not removed               | Dialysis | chartevents        | Numeric
      , 224191 -- | Hourly Patient Fluid Removal     | Dialysis | chartevents        | Numeric
      , 225806 -- | Volume In (PD)                   | Dialysis | chartevents        | Numeric
      , 225807 -- | Volume Out (PD)                  | Dialysis | chartevents        | Numeric
      , 228004 -- | Citrate (ACD-A)                  | Dialysis | chartevents        | Numeric
      , 228005 -- | PBP (Prefilter) Replacement Rate | Dialysis | chartevents        | Numeric
      , 228006 -- | Post Filter Replacement Rate     | Dialysis | chartevents        | Numeric
      , 224144 -- | Blood Flow (ml/min)              | Dialysis | chartevents        | Numeric
      , 224145 -- | Heparin Dose (per hour)          | Dialysis | chartevents        | Numeric
      , 224149 -- | Access Pressure                  | Dialysis | chartevents        | Numeric
      , 224150 -- | Filter Pressure                  | Dialysis | chartevents        | Numeric
      , 224151 -- | Effluent Pressure                | Dialysis | chartevents        | Numeric
      , 224152 -- | Return Pressure                  | Dialysis | chartevents        | Numeric
      , 224153 -- | Replacement Rate                 | Dialysis | chartevents        | Numeric
      , 224404 -- | ART Lumen Volume                 | Dialysis | chartevents        | Numeric
      , 224406 -- | VEN Lumen Volume                 | Dialysis | chartevents        | Numeric
      , 226457 -- | Ultrafiltrate Output             | Dialysis | chartevents        | Numeric
    )
    and valuenum > 0 -- also ensures it's not null
  group by ie.icustay_id
)
, mv_ie as
(
  select ie.icustay_id
    , 1 as RRT
  from icustays ie
  inner join inputevents_mv tt
    on ie.icustay_id = tt.icustay_id
    and tt.starttime between ie.intime and ie.intime + interval '2' day
    and itemid in
    (
        227536 --	KCl (CRRT)	Medications	inputevents_mv	Solution
      , 227525 --	Calcium Gluconate (CRRT)	Medications	inputevents_mv	Solution
    )
    and amount > 0 -- also ensures it's not null
  group by ie.icustay_id
)
, mv_de as
(
  select ie.icustay_id
    , 1 as RRT
  from icustays ie
  inner join datetimeevents tt
    on ie.icustay_id = tt.icustay_id
    and tt.charttime between ie.intime and ie.intime + interval '2' day
    and itemid in
    (
      -- TODO: unsure how to handle "Last dialysis"
      --  225128 -- | Last dialysis                                     | Adm History/FHPA        | datetimeevents     | Date time
        225318 -- | Dialysis Catheter Cap Change                      | Access Lines - Invasive | datetimeevents     | Date time
      , 225319 -- | Dialysis Catheter Change over Wire Date           | Access Lines - Invasive | datetimeevents     | Date time
      , 225321 -- | Dialysis Catheter Dressing Change                 | Access Lines - Invasive | datetimeevents     | Date time
      , 225322 -- | Dialysis Catheter Insertion Date                  | Access Lines - Invasive | datetimeevents     | Date time
      , 225324 -- | Dialysis CatheterTubing Change                    | Access Lines - Invasive | datetimeevents     | Date time
    )
  group by ie.icustay_id
)
, mv_pe as
(
    select ie.icustay_id
      , 1 as RRT
    from icustays ie
    inner join procedureevents_mv tt
      on ie.icustay_id = tt.icustay_id
      and tt.starttime between ie.intime and ie.intime + interval '2' day
      and itemid in
      (
          225441 -- | Hemodialysis                                      | 4-Procedures            | procedureevents_mv | Process
        , 225802 -- | Dialysis - CRRT                                   | Dialysis                | procedureevents_mv | Process
        , 225803 -- | Dialysis - CVVHD                                  | Dialysis                | procedureevents_mv | Process
        , 225805 -- | Peritoneal Dialysis                               | Dialysis                | procedureevents_mv | Process
        , 224270 -- | Dialysis Catheter                                 | Access Lines - Invasive | procedureevents_mv | Process
        , 225809 -- | Dialysis - CVVHDF                                 | Dialysis                | procedureevents_mv | Process
        , 225955 -- | Dialysis - SCUF                                   | Dialysis                | procedureevents_mv | Process
        , 225436 -- | CRRT Filter Change               | Dialysis | procedureevents_mv | Process
      )
    group by ie.icustay_id
)
select ie.subject_id, ie.hadm_id, ie.icustay_id
  , case
      when cv.RRT = 1 then 1
      when mv_ce.RRT = 1 then 1
      when mv_ie.RRT = 1 then 1
      when mv_de.RRT = 1 then 1
      when mv_pe.RRT = 1 then 1
      else 0
    end as RRT_2d
from icustays ie
left join cv
  on ie.icustay_id = cv.icustay_id
left join mv_ce
  on ie.icustay_id = mv_ce.icustay_id
left join mv_ie
  on ie.icustay_id = mv_ie.icustay_id
left join mv_de
  on ie.icustay_id = mv_de.icustay_id
left join mv_pe
  on ie.icustay_id = mv_pe.icustay_id
order by ie.icustay_id;
"""
cur.execute(query_rrt2day)
con.commit()

In [11]:
query_rrt7day= query_schema+"""
DROP MATERIALIZED VIEW IF EXISTS rrt7day CASCADE;
CREATE MATERIALIZED VIEW rrt7day as
with cv as
(
  select ie.icustay_id
    , max(
        case
          when ce.itemid in (152,148,149,146,147,151,150) and value is not null then 1
          when ce.itemid in (229,235,241,247,253,259,265,271) and value = 'Dialysis Line' then 1
          when ce.itemid = 582 and value in ('CAVH Start','CAVH D/C','CVVHD Start','CVVHD D/C','Hemodialysis st','Hemodialysis end') then 1
        else 0 end
        ) as RRT
  from icustays ie
  inner join chartevents ce
    on ie.icustay_id = ce.icustay_id
    and ce.itemid in
    (
       152 -- "Dialysis Type";61449
      ,148 -- "Dialysis Access Site";60335
      ,149 -- "Dialysis Access Type";60030
      ,146 -- "Dialysate Flow ml/hr";57445
      ,147 -- "Dialysate Infusing";56605
      ,151 -- "Dialysis Site Appear";37345
      ,150 -- "Dialysis Machine";27472
      ,229 -- INV Line#1 [Type]
      ,235 -- INV Line#2 [Type]
      ,241 -- INV Line#3 [Type]
      ,247 -- INV Line#4 [Type]
      ,253 -- INV Line#5 [Type]
      ,259 -- INV Line#6 [Type]
      ,265 -- INV Line#7 [Type]
      ,271 -- INV Line#8 [Type]
      ,582 -- Procedures
    )
    and ce.value is not null
    and ce.charttime between ie.intime and ie.intime + interval '7' day
  where ie.dbsource = 'carevue'
  group by ie.icustay_id
)
, mv_ce as
(
  select ie.icustay_id
    , 1 as RRT
  from icustays ie
  inner join chartevents ce
    on ie.icustay_id = ce.icustay_id
    and ce.charttime between ie.intime and ie.intime + interval '7' day
    and itemid in
    (
      -- Checkboxes
        226118 -- | Dialysis Catheter placed in outside facility      | Access Lines - Invasive | chartevents        | Checkbox
      , 227357 -- | Dialysis Catheter Dressing Occlusive              | Access Lines - Invasive | chartevents        | Checkbox
      , 225725 -- | Dialysis Catheter Tip Cultured                    | Access Lines - Invasive | chartevents        | Checkbox
      -- Numeric values
      , 226499 -- | Hemodialysis Output                               | Dialysis                | chartevents        | Numeric
      , 224154 -- | Dialysate Rate                                    | Dialysis                | chartevents        | Numeric
      , 225810 -- | Dwell Time (Peritoneal Dialysis)                  | Dialysis                | chartevents        | Numeric
      , 227639 -- | Medication Added Amount  #2 (Peritoneal Dialysis) | Dialysis                | chartevents        | Numeric
      , 225183 -- | Current Goal                     | Dialysis | chartevents        | Numeric
      , 227438 -- | Volume not removed               | Dialysis | chartevents        | Numeric
      , 224191 -- | Hourly Patient Fluid Removal     | Dialysis | chartevents        | Numeric
      , 225806 -- | Volume In (PD)                   | Dialysis | chartevents        | Numeric
      , 225807 -- | Volume Out (PD)                  | Dialysis | chartevents        | Numeric
      , 228004 -- | Citrate (ACD-A)                  | Dialysis | chartevents        | Numeric
      , 228005 -- | PBP (Prefilter) Replacement Rate | Dialysis | chartevents        | Numeric
      , 228006 -- | Post Filter Replacement Rate     | Dialysis | chartevents        | Numeric
      , 224144 -- | Blood Flow (ml/min)              | Dialysis | chartevents        | Numeric
      , 224145 -- | Heparin Dose (per hour)          | Dialysis | chartevents        | Numeric
      , 224149 -- | Access Pressure                  | Dialysis | chartevents        | Numeric
      , 224150 -- | Filter Pressure                  | Dialysis | chartevents        | Numeric
      , 224151 -- | Effluent Pressure                | Dialysis | chartevents        | Numeric
      , 224152 -- | Return Pressure                  | Dialysis | chartevents        | Numeric
      , 224153 -- | Replacement Rate                 | Dialysis | chartevents        | Numeric
      , 224404 -- | ART Lumen Volume                 | Dialysis | chartevents        | Numeric
      , 224406 -- | VEN Lumen Volume                 | Dialysis | chartevents        | Numeric
      , 226457 -- | Ultrafiltrate Output             | Dialysis | chartevents        | Numeric
    )
    and valuenum > 0 -- also ensures it's not null
  group by ie.icustay_id
)
, mv_ie as
(
  select ie.icustay_id
    , 1 as RRT
  from icustays ie
  inner join inputevents_mv tt
    on ie.icustay_id = tt.icustay_id
    and tt.starttime between ie.intime and ie.intime + interval '7' day
    and itemid in
    (
        227536 --	KCl (CRRT)	Medications	inputevents_mv	Solution
      , 227525 --	Calcium Gluconate (CRRT)	Medications	inputevents_mv	Solution
    )
    and amount > 0 -- also ensures it's not null
  group by ie.icustay_id
)
, mv_de as
(
  select ie.icustay_id
    , 1 as RRT
  from icustays ie
  inner join datetimeevents tt
    on ie.icustay_id = tt.icustay_id
    and tt.charttime between ie.intime and ie.intime + interval '7' day
    and itemid in
    (
      -- TODO: unsure how to handle "Last dialysis"
      --  225128 -- | Last dialysis                                     | Adm History/FHPA        | datetimeevents     | Date time
        225318 -- | Dialysis Catheter Cap Change                      | Access Lines - Invasive | datetimeevents     | Date time
      , 225319 -- | Dialysis Catheter Change over Wire Date           | Access Lines - Invasive | datetimeevents     | Date time
      , 225321 -- | Dialysis Catheter Dressing Change                 | Access Lines - Invasive | datetimeevents     | Date time
      , 225322 -- | Dialysis Catheter Insertion Date                  | Access Lines - Invasive | datetimeevents     | Date time
      , 225324 -- | Dialysis CatheterTubing Change                    | Access Lines - Invasive | datetimeevents     | Date time
    )
  group by ie.icustay_id
)
, mv_pe as
(
    select ie.icustay_id
      , 1 as RRT
    from icustays ie
    inner join procedureevents_mv tt
      on ie.icustay_id = tt.icustay_id
      and tt.starttime between ie.intime and ie.intime + interval '7' day
      and itemid in
      (
          225441 -- | Hemodialysis                                      | 4-Procedures            | procedureevents_mv | Process
        , 225802 -- | Dialysis - CRRT                                   | Dialysis                | procedureevents_mv | Process
        , 225803 -- | Dialysis - CVVHD                                  | Dialysis                | procedureevents_mv | Process
        , 225805 -- | Peritoneal Dialysis                               | Dialysis                | procedureevents_mv | Process
        , 224270 -- | Dialysis Catheter                                 | Access Lines - Invasive | procedureevents_mv | Process
        , 225809 -- | Dialysis - CVVHDF                                 | Dialysis                | procedureevents_mv | Process
        , 225955 -- | Dialysis - SCUF                                   | Dialysis                | procedureevents_mv | Process
        , 225436 -- | CRRT Filter Change               | Dialysis | procedureevents_mv | Process
      )
    group by ie.icustay_id
)
select ie.subject_id, ie.hadm_id, ie.icustay_id
  , case
      when cv.RRT = 1 then 1
      when mv_ce.RRT = 1 then 1
      when mv_ie.RRT = 1 then 1
      when mv_de.RRT = 1 then 1
      when mv_pe.RRT = 1 then 1
      else 0
    end as RRT_7d
from icustays ie
left join cv
  on ie.icustay_id = cv.icustay_id
left join mv_ce
  on ie.icustay_id = mv_ce.icustay_id
left join mv_ie
  on ie.icustay_id = mv_ie.icustay_id
left join mv_de
  on ie.icustay_id = mv_de.icustay_id
left join mv_pe
  on ie.icustay_id = mv_pe.icustay_id
order by ie.icustay_id;
"""
cur.execute(query_rrt7day)
con.commit()

In [12]:
# kdigo_stages_48hr table using kdigo_uo and kdigo_creat table
query_new_kdigo_stages_48hr =query_schema+"""
-- This query checks if the patient had AKI according to KDIGO on admission
-- AKI can be defined either using data from the first 2 days, or first 7 days

-- For urine output: the highest UO in hours 0-48 is used
-- For creatinine: the creatinine value from days 0-2 or 0-7 is used.
-- Baseline creatinine is defined as first measurement in hours [-6, 24] from ICU admit

DROP MATERIALIZED VIEW IF EXISTS new_kdigo_stages_48hr CASCADE;
CREATE MATERIALIZED VIEW new_kdigo_stages_48hr AS
with uo_6hr as
(
  select
        ie.icustay_id
      -- , uo.charttime
      -- , uo.urineoutput_6hr
      , min(uo.urineoutput_6hr / uo.weight / 6.0)::numeric as uo6
  from icustays ie
  inner join kdigo_uo uo
    on ie.icustay_id = uo.icustay_id
    and uo.charttime <= ie.intime + interval '42' hour
  group by ie.icustay_id
)
, uo_12hr as
(
  select
      ie.icustay_id
      -- , uo.charttime
      -- , uo.weight
      -- , uo.urineoutput_12hr
      , min(uo.urineoutput_12hr / uo.weight / 12.0)::numeric as uo12
  from icustays ie
  inner join kdigo_uo uo
    on ie.icustay_id = uo.icustay_id
    and uo.charttime <= ie.intime + interval '36' hour
  group by ie.icustay_id
)
, uo_24hr as
(
  select
      ie.icustay_id
      -- , uo.charttime
      -- , uo.weight
      -- , uo.urineoutput_24hr
      , min(uo.urineoutput_24hr / uo.weight / 24.0)::numeric as uo24
  from icustays ie
  inner join kdigo_uo uo
    on ie.icustay_id = uo.icustay_id
    and uo.charttime <= ie.intime + interval '24' hour
  group by ie.icustay_id
)
-- stages for UO / creat

, cre as
(
select
    ie.icustay_id
  , ie.intime, ie.outtime
  , la.valuenum as creatinine
  , la.charttime
  from icustays ie
  left join labo la
    on ie.subject_id = la.subject_id
    and la.label = 'CREATININE'
    and la.VALUENUM is not null
    and la.CHARTTIME between (ie.intime - interval '3' month) and (ie.intime - interval '1' day)
)

, cre_base as
(
select
    cre.icustay_id
  , cre.creatinine
  , cre.charttime
  , ROW_NUMBER ()
          OVER (PARTITION BY cre.icustay_id
                ORDER BY cre.charttime desc
              ) as rn_last
  from cre
)

, creat_base as 
(
select
    ie.subject_id, ie.hadm_id, ie.icustay_id,ie.intime
  , cre_base.creatinine as BaseCre
  , cre_base.charttime as BaseCreTime

from icustays ie
left join cre_base
  on ie.icustay_id = cre_base.icustay_id
  and cre_base.rn_last = 1
order by ie.icustay_id
)



, kdigo_stg as
(

  select ie.icustay_id
  , ie.intime, ie.outtime
  , case
    when HighCreat48hr >= (ba.BaseCre*3.0) then 3
    when HighCreat48hr >= 4 -- note the criteria specify an INCREASE to >=4
      and ba.BaseCre <= (3.7)  then 3 -- therefore we check that adm <= 3.7
    -- TODO: initiation of RRT
    when rrt2day.RRT_2d=1 then 3
    when HighCreat48hr >= (ba.BaseCre*2.0) then 2
    when HighCreat48hr >= (ba.BaseCre+0.3) then 1
    when HighCreat48hr >= (ba.BaseCre*1.5) then 1
    when HighCreat48hr is null then null
      when ba.BaseCre is null then null
    else 0 end as AKI_stage_48hr_creat

  -- AKI stages according to urine output
  , case
      when UO24 < 0.3 then 3
      when UO12 = 0 then 3
      when UO12 < 0.5 then 2
      when UO6  < 0.5 then 1
      when UO6  is null then null
    else 0 end as AKI_stage_48hr_uo

  -- Creatinine information
  , ba.BaseCreTime, ba.BaseCre
  , HighCreat48hr, HighCreat48hrTime

  -- Urine output information: the values and the time of their measurement
  , round(UO6,4) as UO6_48hr
  , round(UO12,4) as UO12_48hr
  , round(UO24,4) as UO24_48hr
  
  ---RRT
  , rrt2day.rrt_2d
  from icustays ie
  left join uo_6hr  on ie.icustay_id = uo_6hr.icustay_id
  left join uo_12hr on ie.icustay_id = uo_12hr.icustay_id
  left join uo_24hr on ie.icustay_id = uo_24hr.icustay_id
  left join KDIGO_CREAT cr on ie.icustay_id = cr.icustay_id
  LEFT JOIN creat_base ba on ie.icustay_id = ba.icustay_id
  LEFT JOIN rrt2day on ie.icustay_id = rrt2day.icustay_id
)
select
  kd.icustay_id

  -- Classify AKI using both creatinine/urine output criteria
  , case
      when coalesce(AKI_stage_48hr_creat,AKI_stage_48hr_uo) > 0 then 1
      else coalesce(AKI_stage_48hr_creat,AKI_stage_48hr_uo)
    end as AKI_48hr

  , case
      when AKI_stage_48hr_creat >= AKI_stage_48hr_uo then AKI_stage_48hr_creat
      when AKI_stage_48hr_uo > AKI_stage_48hr_creat then AKI_stage_48hr_uo
      else coalesce(AKI_stage_48hr_creat,AKI_stage_48hr_uo)
    end as AKI_stage_48hr

  -- components
  , AKI_stage_48hr_creat
  , AKI_stage_48hr_uo

  -- Creatinine information - convert absolute times to hours since admission
  , BaseCre
  , HighCreat48hr
  , ROUND(extract(epoch from (BaseCreTime-intime))::numeric / 60.0 / 60.0 / 24.0, 4) as LowCreat48hrTimeElapsed
  , ROUND(extract(epoch from (HighCreat48hrTime-intime))::numeric / 60.0 / 60.0 / 24.0, 4) as HighCreat48hrTimeElapsed
  , BaseCreTime
  , HighCreat48hrTime

  -- Urine output information: the values and the time of their measurement
  , UO6_48hr
  , UO12_48hr
  , UO24_48hr
  
  ,rrt_2d
from kdigo_stg kd
order by kd.icustay_id;
"""
cur.execute(query_new_kdigo_stages_48hr)
con.commit()

In [13]:
query_new_kdigo_stages_7day =query_schema+"""
DROP MATERIALIZED VIEW IF EXISTS new_kdigo_stages_7day CASCADE;
CREATE MATERIALIZED VIEW new_kdigo_stages_7day AS
with uo_6hr as
(
  select
        ie.icustay_id
      -- , uo.charttime
      -- , uo.urineoutput_6hr
      , min(uo.urineoutput_6hr / uo.weight / 6.0)::numeric as uo6
  from icustays ie
  inner join kdigo_uo uo
    on ie.icustay_id = uo.icustay_id
    and uo.charttime <= ie.intime + interval '7' day - interval '6' hour
  group by ie.icustay_id
)
, uo_12hr as
(
  select
      ie.icustay_id
      -- , uo.charttime
      -- , uo.weight
      -- , uo.urineoutput_12hr
      , min(uo.urineoutput_12hr / uo.weight / 12.0)::numeric as uo12
  from icustays ie
  inner join kdigo_uo uo
    on ie.icustay_id = uo.icustay_id
    and uo.charttime <= ie.intime + interval '7' day - interval '12' hour
  group by ie.icustay_id
)
, uo_24hr as
(
  select
      ie.icustay_id
      -- , uo.charttime
      -- , uo.weight
      -- , uo.urineoutput_24hr
      , min(uo.urineoutput_24hr / uo.weight / 24.0)::numeric as uo24
  from icustays ie
  inner join kdigo_uo uo
    on ie.icustay_id = uo.icustay_id
    and uo.charttime <= ie.intime + interval '7' day - interval '24' hour
  group by ie.icustay_id
)
-- stages for UO / creat


, cre as
(
select
    ie.icustay_id
  , ie.intime, ie.outtime
  , la.valuenum as creatinine
  , la.charttime
  from icustays ie
  left join labo la
    on ie.subject_id = la.subject_id
    and la.label = 'CREATININE'
    and la.VALUENUM is not null
    and la.CHARTTIME between (ie.intime - interval '3' month) and (ie.intime - interval '1' day)
)

, cre_base as
(
select
    cre.icustay_id
  , cre.creatinine
  , cre.charttime
  , ROW_NUMBER ()
          OVER (PARTITION BY cre.icustay_id
                ORDER BY cre.charttime desc
              ) as rn_last
  from cre
)

, creat_base as 
(
select
    ie.subject_id, ie.hadm_id, ie.icustay_id,ie.intime
  , cre_base.creatinine as BaseCre
  , cre_base.charttime as BaseCreTime

from icustays ie
left join cre_base
  on ie.icustay_id = cre_base.icustay_id
  and cre_base.rn_last = 1
order by ie.icustay_id
)


, kdigo_stg as
(

  select ie.icustay_id
  , ie.intime, ie.outtime
  , case
    when HighCreat7day >= (ba.BaseCre*3.0) then 3
    when HighCreat7day >= 4 -- note the criteria specify an INCREASE to >=4
      and ba.BaseCre <= (3.7)  then 3 -- therefore we check that adm <= 3.7
    -- TODO: initiation of RRT
    when rrt7day.RRT_7d=1 then 3
    when HighCreat7day >= (ba.BaseCre*2.0) then 2
    when HighCreat7day >= (ba.BaseCre+0.3) then 1
    when HighCreat7day >= (ba.BaseCre*1.5) then 1
    when HighCreat7day is null then null
      when ba.BaseCre is null then null
    else 0 end as AKI_stage_7day_creat

  -- AKI stages according to urine output
  , case
      when UO24 < 0.3 then 3
      when UO12 = 0 then 3
      when UO12 < 0.5 then 2
      when UO6  < 0.5 then 1
      when UO6  is null then null
    else 0 end as AKI_stage_7day_uo

  -- Creatinine information
  , ba.BaseCreTime, ba.BaseCre
  , HighCreat7dayTime, HighCreat7day

  -- Urine output information: the values and the time of their measurement
  , round(UO6,4) as UO6_48hr
  , round(UO12,4) as UO12_48hr
  , round(UO24,4) as UO24_48hr
  
  --- RRT
  , rrt7day.RRT_7d
  from icustays ie
  left join uo_6hr  on ie.icustay_id = uo_6hr.icustay_id
  left join uo_12hr on ie.icustay_id = uo_12hr.icustay_id
  left join uo_24hr on ie.icustay_id = uo_24hr.icustay_id
  left join KDIGO_CREAT cr on ie.icustay_id = cr.icustay_id
  LEFT JOIN creat_base ba on ie.icustay_id = ba.icustay_id
  LEFT JOIN rrt7day on ie.icustay_id = rrt7day.icustay_id
)
select
  kd.icustay_id

  -- Classify AKI using both creatinine/urine output criteria
  , case
      when coalesce(AKI_stage_7day_creat,AKI_stage_7day_uo) > 0 then 1
      else coalesce(AKI_stage_7day_creat,AKI_stage_7day_uo)
    end as AKI_7day

  , case
      when AKI_stage_7day_creat >= AKI_stage_7day_uo then AKI_stage_7day_creat
      when AKI_stage_7day_uo > AKI_stage_7day_creat then AKI_stage_7day_uo
      else coalesce(AKI_stage_7day_creat,AKI_stage_7day_uo)
    end as AKI_stage_7day

  , AKI_stage_7day_creat

  -- Creatinine information - convert absolute times to hours since admission
  , BaseCre
  , HighCreat7day
  , ROUND(extract(epoch from (BaseCreTime-intime))::numeric / 60.0 / 60.0 / 24.0, 4) as BaseCreTimeElapsed
  , ROUND(extract(epoch from (HighCreat7dayTime-intime))::numeric / 60.0 / 60.0 / 24.0, 4) as HighCreat7dayTimeElapsed
  , BaseCreTime
  , HighCreat7dayTime

  -- Urine output information: the values and the time of their measurement
  , UO6_48hr
  , UO12_48hr
  , UO24_48hr
  
  , rrt_7d
from kdigo_stg kd
order by kd.icustay_id;
"""
cur.execute(query_new_kdigo_stages_7day)
con.commit()

In [14]:
pd.read_sql(query_schema+"""
select *
from  new_kdigo_stages_7day
limit 10
""", con)

,icustay_id,aki_7day,aki_stage_7day,aki_stage_7day_creat,basecre,highcreat7day,basecretimeelapsed,highcreat7daytimeelapsed,basecretime,highcreat7daytime,uo6_48hr,uo12_48hr,uo24_48hr,rrt_7d
0,200001,1.0,3.0,3.0,2.2,4.7,-1.5043,4.4853,2181-11-24 07:00:00,2181-11-30 06:45:00,NaN,NaN,NaN,1
1,200003,1.0,3.0,1.0,0.6,1.0,-16.2570,0.2479,2199-07-17 13:40:00,2199-08-03 01:47:00,0.4601,0.2300,0.1150,0
2,200006,1.0,3.0,1.0,0.8,1.1,-24.9467,-0.1446,2159-08-09 12:45:00,2159-09-03 08:00:00,0.2427,0.1214,0.0607,0
3,200007,1.0,3.0,NaN,NaN,0.9,NaN,-0.0553,NaT,2109-02-17 08:44:00,0.3307,0.2811,0.1405,0
4,200009,0.0,3.0,0.0,0.6,0.5,-11.9684,0.3295,2189-11-18 11:20:00,2189-11-30 18:29:00,0.1810,0.0905,0.0452,0
5,200010,0.0,0.0,0.0,1.1,1.0,-1.8565,-0.0988,2132-08-03 02:30:00,2132-08-04 20:41:00,NaN,NaN,NaN,0
6,200011,0.0,0.0,0.0,2.2,2.0,-8.8475,0.0261,2188-07-28 05:19:00,2188-08-06 02:17:00,NaN,NaN,NaN,0
7,200012,0.0,0.0,NaN,NaN,0.7,NaN,-0.2451,NaT,2153-12-22 23:20:00,1.7904,0.8952,0.4476,0
8,200014,NaN,NaN,NaN,NaN,0.9,NaN,3.3286,NaT,2105-02-20 07:10:00,NaN,NaN,NaN,0
9,200016,NaN,NaN,NaN,NaN,0.7,NaN,-0.0204,NaT,2150-12-02 15:30:00,NaN,NaN,NaN,0


In [15]:
#Final diagnoses for hospital admissions are coded and can be found in the DIAGNOSES_ICD table
#Furthermore, even though laboratory values are captured elsewhere (LABEVENTS), they are frequently repeated within CHARTEVENTS. This occurs because it is desirable to display the laboratory values on the patient’s electronic chart, and so the values are copied from the database storing laboratory values to the database storing the CHARTEVENTS.

In [16]:
query_summary_lab= query_schema+"""
DROP MATERIALIZED VIEW IF EXISTS summary_lab cascade;
CREATE MATERIALIZED VIEW summary_lab AS
with cl as
(
select
    ie.icustay_id
  , ie.intime, ie.outtime
  , la.valuenum as chloride
  , la.charttime
  from icustays ie
  left join labo la
    on ie.subject_id = la.subject_id
    and la.label = 'CHLORIDE'
    and la.VALUENUM is not null
    and la.CHARTTIME between (ie.intime - interval '6' hour) and (ie.intime + interval '7' day)
)

, cl_48hr as
(
select
    cl.icustay_id
  , cl.chloride
  , cl.charttime
  , ROW_NUMBER ()
          OVER (PARTITION BY cl.icustay_id
                ORDER BY cl.charttime
              ) as rn_first
  , ROW_NUMBER ()
          OVER (PARTITION BY cl.icustay_id
                ORDER BY cl.chloride DESC
              ) as rn_highest
  , ROW_NUMBER ()
          OVER (PARTITION BY cl.icustay_id
                ORDER BY cl.chloride
              ) as rn_lowest
  from cl
  -- limit to the first 48 hours (source table has data up to 7 days)
  where cl.charttime <= cl.intime + interval '48' hour
)

, chloride as
(
select
    ie.subject_id, ie.hadm_id, ie.icustay_id
  , cl_48hr_admit.chloride as AdmCl
  , cl_48hr_admit.charttime as AdmClTime
  , cl_48hr_low.chloride as LowCl48hr
  , cl_48hr_low.charttime as LowCl48hrTime
  , cl_48hr_high.chloride as HighCl48hr
  , cl_48hr_high.charttime as HighCl48hrTime

from icustays ie
left join cl_48hr cl_48hr_admit
  on ie.icustay_id = cl_48hr_admit.icustay_id
  and cl_48hr_admit.rn_first = 1
left join cl_48hr cl_48hr_high
  on ie.icustay_id = cl_48hr_high.icustay_id
  and cl_48hr_high.rn_highest = 1
left join cl_48hr cl_48hr_low
  on ie.icustay_id = cl_48hr_low.icustay_id
  and cl_48hr_low.rn_lowest = 1
order by ie.icustay_id
)

, na as 
(
select
    ie.icustay_id
  , ie.intime, ie.outtime
  , la.valuenum as sodium
  , la.charttime
  from icustays ie
  left join labo la
    on ie.subject_id = la.subject_id
    and la.label = 'SODIUM'
    and la.VALUENUM is not null
    and la.CHARTTIME between (ie.intime - interval '6' hour) and (ie.intime + interval '7' day)
)

, na_48hr as
(
select
    na.icustay_id
  , na.sodium
  , na.charttime
  , ROW_NUMBER ()
          OVER (PARTITION BY na.icustay_id
                ORDER BY na.charttime
              ) as rn_first
  , ROW_NUMBER ()
          OVER (PARTITION BY na.icustay_id
                ORDER BY na.sodium DESC
              ) as rn_highest
  , ROW_NUMBER ()
          OVER (PARTITION BY na.icustay_id
                ORDER BY na.sodium
              ) as rn_lowest
  from na
  where na.charttime <= na.intime + interval '48' hour
)

, sodium as 
(
select
    ie.subject_id, ie.hadm_id, ie.icustay_id
  , na_48hr_admit.sodium as AdmNa
  , na_48hr_admit.charttime as AdmNaTime
  , na_48hr_low.sodium as LowNa48hr
  , na_48hr_low.charttime as LowNa48hrTime
  , na_48hr_high.sodium as HighNa48hr
  , na_48hr_high.charttime as HighNa48hrTime

from icustays ie
left join na_48hr na_48hr_admit
  on ie.icustay_id = na_48hr_admit.icustay_id
  and na_48hr_admit.rn_first = 1
left join na_48hr na_48hr_high
  on ie.icustay_id = na_48hr_high.icustay_id
  and na_48hr_high.rn_highest = 1
left join na_48hr na_48hr_low
  on ie.icustay_id = na_48hr_low.icustay_id
  and na_48hr_low.rn_lowest = 1
order by ie.icustay_id
)

, sid_48hr as
(
select
    na.icustay_id
  , na.sodium
  , na.charttime
  , cl.chloride
  , na.sodium - cl.chloride as sid
  , ROW_NUMBER ()
          OVER (PARTITION BY na.icustay_id
                ORDER BY na.charttime
              ) as rn_first
  , ROW_NUMBER ()
          OVER (PARTITION BY na.icustay_id
                ORDER BY (na.sodium - cl.chloride) DESC
              ) as rn_highest
  , ROW_NUMBER ()
          OVER (PARTITION BY na.icustay_id
                ORDER BY (na.sodium - cl.chloride)
              ) as rn_lowest
  from na
  INNER JOIN cl
    ON na.icustay_id= cl.icustay_id
    AND na.charttime= cl.charttime
  where na.charttime <= na.intime + interval '48' hour
)

, sid as 
(
select
    ie.subject_id, ie.hadm_id, ie.icustay_id
  , sid_48hr_admit.sid as AdmSID
  , sid_48hr_admit.charttime as AdmSIDTime
  , sid_48hr_low.sid as LowSID48hr
  , sid_48hr_low.charttime as LowSID48hrTime
  , sid_48hr_high.sid as HighSID48hr
  , sid_48hr_high.charttime as HighSID48hrTime

from icustays ie
left join sid_48hr sid_48hr_admit
  on ie.icustay_id = sid_48hr_admit.icustay_id
  and sid_48hr_admit.rn_first = 1
left join sid_48hr sid_48hr_high
  on ie.icustay_id = sid_48hr_high.icustay_id
  and sid_48hr_high.rn_highest = 1
left join sid_48hr sid_48hr_low
  on ie.icustay_id = sid_48hr_low.icustay_id
  and sid_48hr_low.rn_lowest = 1
order by ie.icustay_id
)

, becl_48hr as 
(
select
    na.icustay_id
  , 102 - (cl.chloride * 140 / na.sodium) as BEcl
  , na.charttime
  , ROW_NUMBER ()
          OVER (PARTITION BY na.icustay_id
                ORDER BY na.charttime
              ) as rn_first
  , ROW_NUMBER ()
          OVER (PARTITION BY na.icustay_id
                ORDER BY (102 - (cl.chloride * 140 / na.sodium) ) DESC
              ) as rn_highest
  , ROW_NUMBER ()
          OVER (PARTITION BY na.icustay_id
                ORDER BY (102 - (cl.chloride * 140 / na.sodium) )
              ) as rn_lowest
  from na
  INNER JOIN cl
    ON na.icustay_id= cl.icustay_id
    AND na.charttime= cl.charttime
  where na.charttime <= na.intime + interval '48' hour
)

, becl as 
(
select
    ie.subject_id, ie.hadm_id, ie.icustay_id
  , becl_48hr_admit.BEcl as AdmBECl
  , becl_48hr_admit.charttime as AdmBEClTime
  , becl_48hr_low.BEcl as LowBECl48hr
  , becl_48hr_low.charttime as LowBECl48hrTime
  , becl_48hr_high.BEcl as HighBECl48hr
  , becl_48hr_high.charttime as HighBECl48hrTime

from icustays ie
left join becl_48hr becl_48hr_admit
  on ie.icustay_id = becl_48hr_admit.icustay_id
  and becl_48hr_admit.rn_first = 1
left join becl_48hr becl_48hr_high
  on ie.icustay_id = becl_48hr_high.icustay_id
  and becl_48hr_high.rn_highest = 1
left join becl_48hr becl_48hr_low
  on ie.icustay_id = becl_48hr_low.icustay_id
  and becl_48hr_low.rn_lowest = 1
order by ie.icustay_id
)


, be as
(
select
    ie.icustay_id
  , ie.intime, ie.outtime
  , la.valuenum as baseexcess
  , la.charttime
  from icustays ie
  left join labo la
    on ie.subject_id = la.subject_id
    and la.label = 'BASEEXCESS'
    and la.VALUENUM is not null
    and la.CHARTTIME between (ie.intime - interval '6' hour) and (ie.intime + interval '7' day)
)

, be_48hr as
(
select
    be.icustay_id
  , be.baseexcess
  , be.charttime
  , ROW_NUMBER ()
          OVER (PARTITION BY be.icustay_id
                ORDER BY be.charttime
              ) as rn_first
  , ROW_NUMBER ()
          OVER (PARTITION BY be.icustay_id
                ORDER BY be.baseexcess DESC
              ) as rn_highest
  , ROW_NUMBER ()
          OVER (PARTITION BY be.icustay_id
                ORDER BY be.baseexcess
              ) as rn_lowest
  from be
  where be.charttime <= be.intime + interval '48' hour
)

, baseexcess as
(
select
    ie.subject_id, ie.hadm_id, ie.icustay_id
  , be_48hr_admit.baseexcess as AdmBE
  , be_48hr_admit.charttime as AdmBETime
  , be_48hr_low.baseexcess as LowBE48hr
  , be_48hr_low.charttime as LowBE48hrTime
  , be_48hr_high.baseexcess as HighBE48hr
  , be_48hr_high.charttime as HighBE48hrTime

from icustays ie
left join be_48hr be_48hr_admit
  on ie.icustay_id = be_48hr_admit.icustay_id
  and be_48hr_admit.rn_first = 1
left join be_48hr be_48hr_high
  on ie.icustay_id = be_48hr_high.icustay_id
  and be_48hr_high.rn_highest = 1
left join be_48hr be_48hr_low
  on ie.icustay_id = be_48hr_low.icustay_id
  and be_48hr_low.rn_lowest = 1
order by ie.icustay_id
)

, cre as
(
select
    ie.icustay_id
  , ie.intime, ie.outtime
  , la.valuenum as creatinine
  , la.charttime
  from icustays ie
  left join labo la
    on ie.subject_id = la.subject_id
    and la.label = 'CREATININE'
    and la.VALUENUM is not null
    and la.CHARTTIME between (ie.intime - interval '6' hour) and (ie.intime + interval '7' day)
)

, cre_48hr as
(
select
    cre.icustay_id
  , cre.creatinine
  , cre.charttime
  , ROW_NUMBER ()
          OVER (PARTITION BY cre.icustay_id
                ORDER BY cre.charttime
              ) as rn_first

  from cre
  -- limit to the first 48 hours (source table has data up to 7 days)
  where cre.charttime <= cre.intime + interval '48' hour
)

, creatinine as
(
select
    ie.subject_id, ie.hadm_id, ie.icustay_id
  , cre_48hr_admit.creatinine as AdmCre
  , cre_48hr_admit.charttime as AdmCreTime

from icustays ie
left join cre_48hr cre_48hr_admit
  on ie.icustay_id = cre_48hr_admit.icustay_id
  and cre_48hr_admit.rn_first = 1
order by ie.icustay_id
)


SELECT chloride.subject_id, chloride.hadm_id, chloride.icustay_id, chloride.admcl, chloride.lowcl48hr, chloride.highcl48hr, sodium.admna
,sodium.lowna48hr,sodium.highna48hr,sid.admsid,sid.lowsid48hr,sid.highsid48hr,baseexcess.admbe,baseexcess.lowbe48hr,baseexcess.highbe48hr
,becl.admbecl,becl.lowbecl48hr,becl.highbecl48hr, creatinine.admcre, creatinine.admcretime

FROM chloride
LEFT JOIN sodium
  on chloride.icustay_id=sodium.icustay_id
LEFT JOIN sid
  on chloride.icustay_id=sid.icustay_id
LEFT JOIN baseexcess
  on chloride.icustay_id=baseexcess.icustay_id
LEFT JOIN becl
  on chloride.icustay_id=becl.icustay_id
LEFT JOIN creatinine
  on chloride.icustay_id=creatinine.icustay_id
"""
cur.execute(query_summary_lab)
con.commit()

In [17]:
query_elixhauser_ahrq= query_schema+"""
DROP MATERIALIZED VIEW IF EXISTS ELIXHAUSER_AHRQ CASCADE;
CREATE MATERIALIZED VIEW ELIXHAUSER_AHRQ as
with icd as
(
  select hadm_id, subject_id, seq_num
    , cast(icd9_code as char(5)) as icd9_code
  from diagnoses_icd
  where seq_num != 1 -- we do not include the primary icd-9 code
)
,
eliflg as
(
select hadm_id, subject_id, seq_num, icd9_code
-- note that these codes will seem incomplete at first
-- for example, CHF is missing a lot of codes referenced in the literature (402.11, 402.91, etc)
-- these codes are captured by hypertension flags instead
-- later there are some complicated rules which confirm/reject those codes as CHF
, CASE
  when icd9_code = '39891' then 1
  when icd9_code between '4280 ' and '4289 ' then 1
		end as CHF       /* Congestive heart failure */

-- cardiac arrhythmias is removed in up to date versions
, case
    when icd9_code = '42610' then 1
    when icd9_code = '42611' then 1
    when icd9_code = '42613' then 1
    when icd9_code between '4262 ' and '42653' then 1
    when icd9_code between '4266 ' and '42689' then 1
    when icd9_code = '4270 ' then 1
    when icd9_code = '4272 ' then 1
    when icd9_code = '42731' then 1
    when icd9_code = '42760' then 1
    when icd9_code = '4279 ' then 1
    when icd9_code = '7850 ' then 1
    when icd9_code between 'V450 ' and 'V4509' then 1
    when icd9_code between 'V533 ' and 'V5339' then 1
  end as ARYTHM /* Cardiac arrhythmias */

, CASE
  when icd9_code between '09320' and '09324' then 1
  when icd9_code between '3940 ' and '3971 ' then 1
  when icd9_code = '3979 ' then 1
  when icd9_code between '4240 ' and '42499' then 1
  when icd9_code between '7463 ' and '7466 ' then 1
  when icd9_code = 'V422 ' then 1
  when icd9_code = 'V433 ' then 1
		end as VALVE     /* Valvular disease */

, CASE
  when icd9_code between '41511' and '41519' then 1
  when icd9_code between '4160 ' and '4169 ' then 1
  when icd9_code = '4179 ' then 1
		end as PULMCIRC  /* Pulmonary circulation disorder */

, CASE
  when icd9_code between '4400 ' and '4409 ' then 1
  when icd9_code between '44100' and '4419 ' then 1
  when icd9_code between '4420 ' and '4429 ' then 1
  when icd9_code between '4431 ' and '4439 ' then 1
  when icd9_code between '44421' and '44422' then 1
  when icd9_code = '4471 ' then 1
  when icd9_code = '449  ' then 1
  when icd9_code = '5571 ' then 1
  when icd9_code = '5579 ' then 1
  when icd9_code = 'V434 ' then 1
		end as PERIVASC  /* Peripheral vascular disorder */

, CASE
  when icd9_code = '4011 ' then 1
  when icd9_code = '4019 ' then 1
  when icd9_code between '64200' and '64204' then 1
		end as HTN       /* Hypertension, uncomplicated */

, CASE
  when icd9_code = '4010 ' then 1
  when icd9_code = '4372 ' then 1
		end as HTNCX     /* Hypertension, complicated */


      /******************************************************************/
      /* The following are special, temporary formats used in the       */
      /* creation of the hypertension complicated comorbidity when      */
      /* overlapping with congestive heart failure or renal failure     */
      /* occurs. These temporary formats are referenced in the program  */
      /* called comoanaly2009.txt.                                      */
      /******************************************************************/
, CASE
  when icd9_code between '64220' and '64224' then 1
		end as HTNPREG   /* Pre-existing hypertension complicating pregnancy */

, CASE
  when icd9_code = '40200' then 1
  when icd9_code = '40210' then 1
  when icd9_code = '40290' then 1
  when icd9_code = '40509' then 1
  when icd9_code = '40519' then 1
  when icd9_code = '40599'         then 1
		end as HTNWOCHF  /* Hypertensive heart disease without heart failure */

, CASE
  when icd9_code = '40201' then 1
  when icd9_code = '40211' then 1
  when icd9_code = '40291'         then 1
		end as HTNWCHF   /* Hypertensive heart disease with heart failure */

, CASE
  when icd9_code = '40300' then 1
  when icd9_code = '40310' then 1
  when icd9_code = '40390' then 1
  when icd9_code = '40501' then 1
  when icd9_code = '40511' then 1
  when icd9_code = '40591' then 1
  when icd9_code between '64210' and '64214' then 1
		end as HRENWORF  /* Hypertensive renal disease without renal failure */

, CASE
  when icd9_code = '40301' then 1
  when icd9_code = '40311' then 1
  when icd9_code = '40391'         then 1
		end as HRENWRF   /* Hypertensive renal disease with renal failure */

, CASE
  when icd9_code = '40400' then 1
  when icd9_code = '40410' then 1
  when icd9_code = '40490'         then 1
		end as HHRWOHRF  /* Hypertensive heart and renal disease without heart or renal failure */

, CASE
  when icd9_code = '40401' then 1
  when icd9_code = '40411' then 1
  when icd9_code = '40491'         then 1
		end as HHRWCHF   /* Hypertensive heart and renal disease with heart failure */

, CASE
  when icd9_code = '40402' then 1
  when icd9_code = '40412' then 1
  when icd9_code = '40492'         then 1
		end as HHRWRF    /* Hypertensive heart and renal disease with renal failure */

, CASE
  when icd9_code = '40403' then 1
  when icd9_code = '40413' then 1
  when icd9_code = '40493'         then 1
		end as HHRWHRF   /* Hypertensive heart and renal disease with heart and renal failure */

, CASE
  when icd9_code between '64270' and '64274' then 1
  when icd9_code between '64290' and '64294' then 1
		end as OHTNPREG  /* Other hypertension in pregnancy */

      /******************** End Temporary Formats ***********************/

, CASE
  when icd9_code between '3420 ' and '3449 ' then 1
  when icd9_code between '43820' and '43853' then 1
  when icd9_code = '78072'         then 1
		end as PARA      /* Paralysis */

, CASE
  when icd9_code between '3300 ' and '3319 ' then 1
  when icd9_code = '3320 ' then 1
  when icd9_code = '3334 ' then 1
  when icd9_code = '3335 ' then 1
  when icd9_code = '3337 ' then 1
  when icd9_code in ('33371','33372','33379','33385','33394') then 1
  when icd9_code between '3340 ' and '3359 ' then 1
  when icd9_code = '3380 ' then 1
  when icd9_code = '340  ' then 1
  when icd9_code between '3411 ' and '3419 ' then 1
  when icd9_code between '34500' and '34511' then 1
  when icd9_code between '3452 ' and '3453 ' then 1
  when icd9_code between '34540' and '34591' then 1
  when icd9_code between '34700' and '34701' then 1
  when icd9_code between '34710' and '34711' then 1
  when icd9_code = '3483' then 1 -- discontinued icd-9
  when icd9_code between '64940' and '64944' then 1
  when icd9_code = '7687 ' then 1
  when icd9_code between '76870' and '76873' then 1
  when icd9_code = '7803 ' then 1
  when icd9_code = '78031' then 1
  when icd9_code = '78032' then 1
  when icd9_code = '78033' then 1
  when icd9_code = '78039' then 1
  when icd9_code = '78097' then 1
  when icd9_code = '7843 '         then 1
		end as NEURO     /* Other neurological */

, CASE
  when icd9_code between '490  ' and '4928 ' then 1
  when icd9_code between '49300' and '49392' then 1
  when icd9_code between '494  ' and '4941 ' then 1
  when icd9_code between '4950 ' and '505  ' then 1
  when icd9_code = '5064 '         then 1
		end as CHRNLUNG  /* Chronic pulmonary disease */

, CASE
  when icd9_code between '25000' and '25033' then 1
  when icd9_code between '64800' and '64804' then 1
  when icd9_code between '24900' and '24931' then 1
		end as DM        /* Diabetes w/o chronic complications*/

, CASE
  when icd9_code between '25040' and '25093' then 1
  when icd9_code = '7751 ' then 1
  when icd9_code between '24940' and '24991' then 1
		end as DMCX      /* Diabetes w/ chronic complications */

, CASE
  when icd9_code between '243  ' and '2442 ' then 1
  when icd9_code = '2448 ' then 1
  when icd9_code = '2449 '         then 1
		end as HYPOTHY   /* Hypothyroidism */

, CASE
  when icd9_code = '585  ' then 1 -- discontinued code
  when icd9_code = '5853 ' then 1
  when icd9_code = '5854 ' then 1
  when icd9_code = '5855 ' then 1
  when icd9_code = '5856 ' then 1
  when icd9_code = '5859 ' then 1
  when icd9_code = '586  ' then 1
  when icd9_code = 'V420 ' then 1
  when icd9_code = 'V451 ' then 1
  when icd9_code between 'V560 ' and 'V5632' then 1
  when icd9_code = 'V568 ' then 1
  when icd9_code between 'V4511' and 'V4512' then 1
		end as RENLFAIL  /* Renal failure */

, CASE
  when icd9_code = '07022' then 1
  when icd9_code = '07023' then 1
  when icd9_code = '07032' then 1
  when icd9_code = '07033' then 1
  when icd9_code = '07044' then 1
  when icd9_code = '07054' then 1
  when icd9_code = '4560 ' then 1
  when icd9_code = '4561 ' then 1
  when icd9_code = '45620' then 1
  when icd9_code = '45621' then 1
  when icd9_code = '5710 ' then 1
  when icd9_code = '5712 ' then 1
  when icd9_code = '5713 ' then 1
  when icd9_code between '57140' and '57149' then 1
  when icd9_code = '5715 ' then 1
  when icd9_code = '5716 ' then 1
  when icd9_code = '5718 ' then 1
  when icd9_code = '5719 ' then 1
  when icd9_code = '5723 ' then 1
  when icd9_code = '5728 ' then 1
  when icd9_code = '5735 ' then 1
  when icd9_code = 'V427 '         then 1
		end as LIVER     /* Liver disease */

, CASE
  when icd9_code = '53141' then 1
  when icd9_code = '53151' then 1
  when icd9_code = '53161' then 1
  when icd9_code = '53170' then 1
  when icd9_code = '53171' then 1
  when icd9_code = '53191' then 1
  when icd9_code = '53241' then 1
  when icd9_code = '53251' then 1
  when icd9_code = '53261' then 1
  when icd9_code = '53270' then 1
  when icd9_code = '53271' then 1
  when icd9_code = '53291' then 1
  when icd9_code = '53341' then 1
  when icd9_code = '53351' then 1
  when icd9_code = '53361' then 1
  when icd9_code = '53370' then 1
  when icd9_code = '53371' then 1
  when icd9_code = '53391' then 1
  when icd9_code = '53441' then 1
  when icd9_code = '53451' then 1
  when icd9_code = '53461' then 1
  when icd9_code = '53470' then 1
  when icd9_code = '53471' then 1
  when icd9_code = '53491'         then 1
		end as ULCER     /* Chronic Peptic ulcer disease (includes bleeding only if obstruction is also present) */

, CASE
  when icd9_code between '042  ' and '0449 ' then 1
		end as AIDS      /* HIV and AIDS */

, CASE
  when icd9_code between '20000' and '20238' then 1
  when icd9_code between '20250' and '20301' then 1
  when icd9_code = '2386 ' then 1
  when icd9_code = '2733 ' then 1
  when icd9_code between '20302' and '20382' then 1
		end as LYMPH     /* Lymphoma */

, CASE
  when icd9_code between '1960 ' and '1991 ' then 1
  when icd9_code between '20970' and '20975' then 1
  when icd9_code = '20979' then 1
  when icd9_code = '78951'         then 1
		end as METS      /* Metastatic cancer */

, CASE
  when icd9_code between '1400 ' and '1729 ' then 1
  when icd9_code between '1740 ' and '1759 ' then 1
  when icd9_code between '179  ' and '1958 ' then 1
  when icd9_code between '20900' and '20924' then 1
  when icd9_code between '20925' and '2093 ' then 1
  when icd9_code between '20930' and '20936' then 1
  when icd9_code between '25801' and '25803' then 1
		end as TUMOR     /* Solid tumor without metastasis */

, CASE
  when icd9_code = '7010 ' then 1
  when icd9_code between '7100 ' and '7109 ' then 1
  when icd9_code between '7140 ' and '7149 ' then 1
  when icd9_code between '7200 ' and '7209 ' then 1
  when icd9_code = '725  ' then 1
		end as ARTH              /* Rheumatoid arthritis/collagen vascular diseases */

, CASE
  when icd9_code between '2860 ' and '2869 ' then 1
  when icd9_code = '2871 ' then 1
  when icd9_code between '2873 ' and '2875 ' then 1
  when icd9_code between '64930' and '64934' then 1
  when icd9_code = '28984'         then 1
		end as COAG      /* Coagulation deficiency */

, CASE
  when icd9_code = '2780 ' then 1
  when icd9_code = '27800' then 1
  when icd9_code = '27801' then 1
  when icd9_code = '27803' then 1
  when icd9_code between '64910' and '64914' then 1
  when icd9_code between 'V8530' and 'V8539' then 1
  when icd9_code = 'V854 ' then 1 -- hierarchy used for AHRQ v3.6 and earlier
  when icd9_code between 'V8541' and 'V8545' then 1
  when icd9_code = 'V8554' then 1
  when icd9_code = '79391'         then 1
		end as OBESE     /* Obesity      */

, CASE
  when icd9_code between '260  ' and '2639 ' then 1
  when icd9_code between '78321' and '78322' then 1
		end as WGHTLOSS  /* Weight loss */

, CASE
  when icd9_code between '2760 ' and '2769 ' then 1
		end as LYTES     /* Fluid and electrolyte disorders - note:
                                      this comorbidity should be dropped when
                                      used with the AHRQ Patient Safety Indicators*/
, CASE
  when icd9_code = '2800 ' then 1
  when icd9_code between '64820' and '64824' then 1
		end as BLDLOSS   /* Blood loss anemia */

, CASE
  when icd9_code between '2801 ' and '2819 ' then 1
  when icd9_code between '28521' and '28529' then 1
  when icd9_code = '2859 '         then 1
		end as ANEMDEF  /* Deficiency anemias */

, CASE
  when icd9_code between '2910 ' and '2913 ' then 1
  when icd9_code = '2915 ' then 1
  when icd9_code = '2918 ' then 1
  when icd9_code = '29181' then 1
  when icd9_code = '29182' then 1
  when icd9_code = '29189' then 1
  when icd9_code = '2919 ' then 1
  when icd9_code between '30300' and '30393' then 1
  when icd9_code between '30500' and '30503' then 1
		end as ALCOHOL   /* Alcohol abuse */

, CASE
  when icd9_code = '2920 ' then 1
  when icd9_code between '29282' and '29289' then 1
  when icd9_code = '2929 ' then 1
  when icd9_code between '30400' and '30493' then 1
  when icd9_code between '30520' and '30593' then 1
  when icd9_code between '64830' and '64834' then 1
		end as DRUG      /* Drug abuse */

, CASE
  when icd9_code between '29500' and '2989 ' then 1
  when icd9_code = '29910' then 1
  when icd9_code = '29911'         then 1
		end as PSYCH    /* Psychoses */

, CASE
  when icd9_code = '3004 ' then 1
  when icd9_code = '30112' then 1
  when icd9_code = '3090 ' then 1
  when icd9_code = '3091 ' then 1
  when icd9_code = '311  '         then 1
		end as DEPRESS  /* Depression */
from icd
)
-- collapse the icd9_code specific flags into hadm_id specific flags
-- this groups comorbidities together for a single patient admission
, eligrp as
(
  select hadm_id, subject_id
  , max(chf) as chf
  , max(arythm) as arythm
  , max(valve) as valve
  , max(pulmcirc) as pulmcirc
  , max(perivasc) as perivasc
  , max(htn) as htn
  , max(htncx) as htncx
  , max(htnpreg) as htnpreg
  , max(htnwochf) as htnwochf
  , max(htnwchf) as htnwchf
  , max(hrenworf) as hrenworf
  , max(hrenwrf) as hrenwrf
  , max(hhrwohrf) as hhrwohrf
  , max(hhrwchf) as hhrwchf
  , max(hhrwrf) as hhrwrf
  , max(hhrwhrf) as hhrwhrf
  , max(ohtnpreg) as ohtnpreg
  , max(para) as para
  , max(neuro) as neuro
  , max(chrnlung) as chrnlung
  , max(dm) as dm
  , max(dmcx) as dmcx
  , max(hypothy) as hypothy
  , max(renlfail) as renlfail
  , max(liver) as liver
  , max(ulcer) as ulcer
  , max(aids) as aids
  , max(lymph) as lymph
  , max(mets) as mets
  , max(tumor) as tumor
  , max(arth) as arth
  , max(coag) as coag
  , max(obese) as obese
  , max(wghtloss) as wghtloss
  , max(lytes) as lytes
  , max(bldloss) as bldloss
  , max(anemdef) as anemdef
  , max(alcohol) as alcohol
  , max(drug) as drug
  , max(psych) as psych
  , max(depress) as depress
from eliflg
group by hadm_id, subject_id
)

-- DRG FILTER --
, msdrg as
(
select
  hadm_id, subject_id
/**** V29 MS-DRG Formats ****/

/* Cardiac */
, case
    when d.drg_code between 001 and 002 then 1
    when d.drg_code between 215 and 238 then 1
    when d.drg_code between 242 and 252 then 1
    when d.drg_code between 253 and 254 then 1
    when d.drg_code between 258 and 262 then 1
    when d.drg_code between 265 and 267 then 1
    when d.drg_code between 280 and 293 then 1
    when d.drg_code between 296 and 298 then 1
    when d.drg_code between 302 and 303 then 1
    when d.drg_code between 306 and 313 then 1
else 0 end as CARDDRG

/* Peripheral vascular */
, case
    when d.drg_code between 299 and 301 then 1
else 0 end as PERIDRG

/* Renal */
, case
    when d.drg_code = 652 then 1
    when d.drg_code between 656 and 661 then 1
    when d.drg_code between 673 and 675 then 1
    when d.drg_code between 682 and 700 then 1
else 0 end as RENALDRG

/* Nervous system */
, case
    when d.drg_code between 020 and 042 then 1
    when d.drg_code between 052 and 103 then 1
else 0 end as NERVDRG

/* Cerebrovascular */
, case
    when d.drg_code between 020 and 022 then 1
    when d.drg_code between 034 and 039 then 1
    when d.drg_code between 064 and 072 then 1
else 0 end as CEREDRG

/* COPD asthma */
, case
    when d.drg_code between 190 and 192 then 1
    when d.drg_code between 202 and 203 then 1
else 0 end as PULMDRG

/* Diabetes */
, case
    when d.drg_code between 637 and 639 then 1
else 0 end as  DIABDRG

/* Thyroid endocrine */
, case
    when d.drg_code between 625 and 627 then 1
    when d.drg_code between 643 and 645 then 1
else 0 end as HYPODRG

/* Kidney transp, renal fail/dialysis */
, case
    when d.drg_code = 652 then 1
    when d.drg_code between 682 and 685 then 1
else 0 end as RENFDRG

/* Liver */
, case
    when d.drg_code between 420 and 425 then 1
    when d.drg_code between 432 and 434 then 1
    when d.drg_code between 441 and 446 then 1
else 0 end as LIVERDRG

/* GI hemorrhage or ulcer */
, case
    when d.drg_code between 377 and 384 then 1
else 0 end as ULCEDRG

/* Human immunodeficiency virus */
, case
    when d.drg_code between 969 and 970 then 1
    when d.drg_code between 974 and 977 then 1
else 0 end as HIVDRG

/* Leukemia/lymphoma */
, case
    when d.drg_code between 820 and 830 then 1
    when d.drg_code between 834 and 849 then 1
else 0 end as LEUKDRG

/* Cancer, lymphoma */
, case
    when d.drg_code = 054 then 1
    when d.drg_code = 055 then 1
    when d.drg_code between 146 and 148 then 1
    when d.drg_code between 180 and 182 then 1
    when d.drg_code between 374 and 376 then 1
    when d.drg_code between 435 and 437 then 1
    when d.drg_code between 542 and 544 then 1
    when d.drg_code between 582 and 585 then 1
    when d.drg_code between 597 and 599 then 1
    when d.drg_code between 656 and 658 then 1
    when d.drg_code between 686 and 688 then 1
    when d.drg_code between 715 and 716 then 1
    when d.drg_code between 722 and 724 then 1
    when d.drg_code between 736 and 741 then 1
    when d.drg_code between 754 and 756 then 1
    when d.drg_code between 826 and 830 then 1
    when d.drg_code between 843 and 849 then 1
else 0 end as CANCDRG

/* Connective tissue */
, case
    when d.drg_code between 545 and 547 then 1
else 0 end as ARTHDRG

/* Nutrition/metabolic */
, case
    when d.drg_code between 640 and 641 then 1
else 0 end as NUTRDRG

/* Anemia */
, case
    when d.drg_code between 808 and 812 then 1
else 0 end as ANEMDRG

/* Alcohol drug */
, case
    when d.drg_code between 894 and 897 then 1
else 0 end as ALCDRG

/*Coagulation disorders*/
, case
    when d.drg_code = 813 then 1
else 0 end as COAGDRG

/*Hypertensive Complicated  */
, case
    when d.drg_code = 077 then 1
    when d.drg_code = 078 then 1
    when d.drg_code = 304 then 1
else 0 end as HTNCXDRG

/*Hypertensive Uncomplicated  */
, case
    when d.drg_code = 079 then 1
    when d.drg_code = 305 then 1
else 0 end as HTNDRG

/* Psychoses */
, case
    when d.drg_code = 885 then 1
else 0 end as PSYDRG

/* Obesity */
, case
    when d.drg_code between 619 and 621 then 1
else 0 end as OBESEDRG

/* Depressive Neuroses */
, case
    when d.drg_code = 881 then 1
else 0 end as DEPRSDRG

from
(
  select hadm_id, subject_id, drg_type, cast(drg_code as numeric) as drg_code from drgcodes where drg_type = 'MS'
) d

)
, hcfadrg as
(
select
  hadm_id, subject_id

  /** V24 DRG Formats  **/

  /* Cardiac */
  , case
      when d.drg_code between 103 and 112 then 1
      when d.drg_code between 115 and 118 then 1
      when d.drg_code between 121 and 127 then 1
      when d.drg_code = 129 then 1
      when d.drg_code = 132 then 1
      when d.drg_code = 133 then 1
      when d.drg_code between 135 and 143 then 1
      when d.drg_code between 514 and 518 then 1
      when d.drg_code between 525 and 527 then 1
      when d.drg_code between 535 and 536 then 1
      when d.drg_code between 547 and 550 then 1
      when d.drg_code between 551 and 558 then 1
  else 0 end as CARDDRG

  /* Peripheral vascular */
  , case
      when d.drg_code = 130 then 1
      when d.drg_code = 131 then 1
  else 0 end as PERIDRG

  /* Renal */
  , case
      when d.drg_code between 302 and 305 then 1
      when d.drg_code between 315 and 333 then 1

  else 0 end as RENALDRG

  /* Nervous system */
  , case
      when d.drg_code between 1 and 35 then 1
      when d.drg_code = 524 then 1
      when d.drg_code between 528 and 534 then 1
      when d.drg_code = 543 then 1
      when d.drg_code between 559 and 564 then 1
      when d.drg_code = 577 then 1

  else 0 end as NERVDRG

   /* Cerebrovascular */
  , case
      when d.drg_code = 5 then 1
      when d.drg_code between 14 and 17 then 1
      when d.drg_code = 524 then 1
      when d.drg_code = 528 then 1
      when d.drg_code between 533 and 534 then 1
      when d.drg_code = 577 then 1
  else 0 end as CEREDRG

  /* COPD asthma */
  , case
      when d.drg_code = 88 then 1
      when d.drg_code between 96 and 98 then 1

  else 0 end as PULMDRG

  /* Diabetes */
  , case
      when d.drg_code = 294 then 1
      when d.drg_code = 295 then 1
  else 0 end as DIABDRG

  /* Thyroid endocrine */
  , case
      when d.drg_code = 290 then 1
      when d.drg_code = 300 then 1
      when d.drg_code = 301 then 1

  else 0 end as HYPODRG

  /* Kidney transp, renal fail/dialysis */
  , case
      when d.drg_code = 302 then 1
      when d.drg_code = 316 then 1
      when d.drg_code = 317 then 1
  else 0 end as RENFDRG

  /* Liver */
  , case
      when d.drg_code between 199 and 202 then 1
      when d.drg_code between 205 and 208 then 1

  else 0 end as LIVERDRG

  /* GI hemorrhage or ulcer */
  , case
      when d.drg_code between 174 and 178 then 1
  else 0 end as ULCEDRG

  /* Human immunodeficiency virus */
  , case
      when d.drg_code = 488 then 1
      when d.drg_code = 489 then 1
      when d.drg_code = 490 then 1

  else 0 end as HIVDRG

  /* Leukemia/lymphoma */
  , case
      when d.drg_code between 400 and 414 then 1
      when d.drg_code = 473 then 1
      when d.drg_code = 492 then 1
      when d.drg_code between 539 and 540 then 1

  else 0 end as LEUKDRG

  /* Cancer, lymphoma */
  , case
      when d.drg_code = 10 then 1
      when d.drg_code = 11 then 1
      when d.drg_code = 64 then 1
      when d.drg_code = 82 then 1
      when d.drg_code = 172 then 1
      when d.drg_code = 173 then 1
      when d.drg_code = 199 then 1
      when d.drg_code = 203 then 1
      when d.drg_code = 239 then 1

      when d.drg_code between 257 and 260 then 1
      when d.drg_code = 274 then 1
      when d.drg_code = 275 then 1
      when d.drg_code = 303 then 1
      when d.drg_code = 318 then 1
      when d.drg_code = 319 then 1

      when d.drg_code = 338 then 1
      when d.drg_code = 344 then 1
      when d.drg_code = 346 then 1
      when d.drg_code = 347 then 1
      when d.drg_code = 354 then 1
      when d.drg_code = 355 then 1
      when d.drg_code = 357 then 1
      when d.drg_code = 363 then 1
      when d.drg_code = 366 then 1

      when d.drg_code = 367 then 1
      when d.drg_code between 406 and 414 then 1
  else 0 end as CANCDRG

  /* Connective tissue */
  , case
      when d.drg_code = 240 then 1
      when d.drg_code = 241 then 1
  else 0 end as ARTHDRG

  /* Nutrition/metabolic */
  , case
      when d.drg_code between 296 and 298 then 1
  else 0 end as NUTRDRG

  /* Anemia */
  , case
      when d.drg_code = 395 then 1
      when d.drg_code = 396 then 1
      when d.drg_code = 574 then 1
  else 0 end as ANEMDRG

  /* Alcohol drug */
  , case
      when d.drg_code between 433 and 437 then 1
      when d.drg_code between 521 and 523 then 1
  else 0 end as ALCDRG

  /* Coagulation disorders */
  , case
      when d.drg_code = 397 then 1
  else 0 end as COAGDRG

  /* Hypertensive Complicated */
  , case
      when d.drg_code = 22 then 1
      when d.drg_code = 134 then 1
  else 0 end as HTNCXDRG

  /* Hypertensive Uncomplicated */
  , case
      when d.drg_code = 134 then 1
  else 0 end as HTNDRG

  /* Psychoses */
  , case
      when d.drg_code = 430 then 1
  else 0 end as PSYDRG

  /* Obesity */
  , case
      when d.drg_code = 288 then 1
  else 0 end as OBESEDRG

  /* Depressive Neuroses */
  , case
      when d.drg_code = 426 then 1
  else 0 end as DEPRSDRG

  from
  (
    select hadm_id, subject_id, drg_type, cast(drg_code as numeric) as drg_code from drgcodes where drg_type = 'HCFA'
  ) d
)
-- merge DRG groups together
, drggrp as
(
  select hadm_id, subject_id
, max(carddrg) as carddrg
, max(peridrg) as peridrg
, max(renaldrg) as renaldrg
, max(nervdrg) as nervdrg
, max(ceredrg) as ceredrg
, max(pulmdrg) as pulmdrg
, max(diabdrg) as diabdrg
, max(hypodrg) as hypodrg
, max(renfdrg) as renfdrg
, max(liverdrg) as liverdrg
, max(ulcedrg) as ulcedrg
, max(hivdrg) as hivdrg
, max(leukdrg) as leukdrg
, max(cancdrg) as cancdrg
, max(arthdrg) as arthdrg
, max(nutrdrg) as nutrdrg
, max(anemdrg) as anemdrg
, max(alcdrg) as alcdrg
, max(coagdrg) as coagdrg
, max(htncxdrg) as htncxdrg
, max(htndrg) as htndrg
, max(psydrg) as psydrg
, max(obesedrg) as obesedrg
, max(deprsdrg) as deprsdrg
from
(
  select d1.* from msdrg d1
  UNION
  select d1.* from hcfadrg d1
) d
group by d.hadm_id, d.subject_id
)
-- now merge these flags together to define elixhauser
-- most are straightforward.. but hypertension flags are a bit more complicated
select adm.subject_id, adm.hadm_id
, case
    when carddrg = 1 then 0 -- DRG filter

    when chf     = 1 then 1
    when htnwchf = 1 then 1
    when hhrwchf = 1 then 1
    when hhrwhrf = 1 then 1
  else 0 end as CONGESTIVE_HEART_FAILURE
, case
    when carddrg = 1 then 0 -- DRG filter
    when arythm = 1 then 1
  else 0 end as CARDIAC_ARRHYTHMIAS
, case
    when carddrg = 1 then 0
    when valve = 1 then 1
  else 0 end as VALVULAR_DISEASE
, case
    when carddrg = 1 or pulmdrg = 1 then 0
    when pulmcirc = 1 then 1
    else 0 end as PULMONARY_CIRCULATION
, case
    when peridrg  = 1 then 0
    when perivasc = 1 then 1
    else 0 end as PERIPHERAL_VASCULAR

-- we combine 'htn' and 'htncx' into 'HYPERTENSION'
-- note 'htn' (hypertension) is only 1 if 'htncx' (complicated hypertension) is 0
-- also if htncxdrg = 1, then htndrg = 1

-- In the original SAS code, it appears that:
--  HTN can be 1
--  HTNCX is set to 0 by DRGs
--  but HTN_C is still 1, because HTN is 1
-- so we have to do this complex addition.
,
case
  when
(
-- first hypertension
case
  when htndrg = 0 then 0
  when htn = 1 then 1
else 0 end
)
+
(
-- next complicated hypertension
case
    when htncx    = 1 and htncxdrg = 1 then 0

    when htnpreg  = 1 and htncxdrg = 1 then 0
    when htnwochf = 1 and (htncxdrg = 1 OR carddrg = 1) then 0
    when htnwchf  = 1 and htncxdrg = 1 then 0
    when htnwchf  = 1 and carddrg = 1 then 0
    when hrenworf = 1 and (htncxdrg = 1 or renaldrg = 1) then 0
    when hrenwrf  = 1 and htncxdrg = 1 then 0
    when hrenwrf  = 1 and renaldrg = 1 then 0
    when hhrwohrf = 1 and (htncxdrg = 1 or carddrg = 1 or renaldrg = 1) then 0
    when hhrwchf  = 1 and (htncxdrg = 1 or carddrg = 1 or renaldrg = 1) then 0
    when hhrwrf   = 1 and (htncxdrg = 1 or carddrg = 1 or renaldrg = 1) then 0
    when hhrwhrf  = 1 and (htncxdrg = 1 or carddrg = 1 or renaldrg = 1) then 0
    when ohtnpreg = 1 and (htncxdrg = 1 or carddrg = 1 or renaldrg = 1) then 0

    when htncx = 1 then 1
    when htnpreg = 1 then 1
    when htnwochf = 1 then 1
    when htnwchf = 1 then 1
    when hrenworf = 1 then 1
    when hrenwrf = 1 then 1
    when hhrwohrf = 1 then 1
    when hhrwchf = 1 then 1
    when hhrwrf = 1 then 1
    when hhrwhrf = 1 then 1
    when ohtnpreg = 1 then 1
  else 0 end
)
  > 0 then 1 else 0 end as HYPERTENSION

, case when ceredrg = 1 then 0 when para      = 1 then 1 else 0 end as PARALYSIS
, case when nervdrg = 1 then 0 when neuro     = 1 then 1 else 0 end as OTHER_NEUROLOGICAL
, case when pulmdrg = 1 then 0 when chrnlung  = 1 then 1 else 0 end as CHRONIC_PULMONARY
, case
    -- only the more severe comorbidity (complicated diabetes) is kept
    when diabdrg = 1 then 0
    when dmcx = 1 then 0
    when dm = 1 then 1
  else 0 end as DIABETES_UNCOMPLICATED
, case when diabdrg = 1 then 0 when dmcx    = 1 then 1 else 0 end as DIABETES_COMPLICATED
, case when hypodrg = 1 then 0 when hypothy = 1 then 1 else 0 end as HYPOTHYROIDISM
, case
    when renaldrg = 1 then 0
    when renlfail = 1 then 1
    when hrenwrf  = 1 then 1
    when hhrwrf   = 1 then 1
    when hhrwhrf  = 1 then 1
  else 0 end as RENAL_FAILURE

, case when liverdrg  = 1 then 0 when liver = 1 then 1 else 0 end as LIVER_DISEASE
, case when ulcedrg   = 1 then 0 when ulcer = 1 then 1 else 0 end as PEPTIC_ULCER
, case when hivdrg    = 1 then 0 when aids = 1 then 1 else 0 end as AIDS
, case when leukdrg   = 1 then 0 when lymph = 1 then 1 else 0 end as LYMPHOMA
, case when cancdrg   = 1 then 0 when mets = 1 then 1 else 0 end as METASTATIC_CANCER
, case
    when cancdrg = 1 then 0
    -- only the more severe comorbidity (metastatic cancer) is kept
    when mets = 1 then 0
    when tumor = 1 then 1
  else 0 end as SOLID_TUMOR
, case when arthdrg   = 1 then 0 when arth = 1 then 1 else 0 end as RHEUMATOID_ARTHRITIS
, case when coagdrg   = 1 then 0 when coag = 1 then 1 else 0 end as COAGULOPATHY
, case when nutrdrg   = 1
         OR obesedrg  = 1 then 0 when obese = 1 then 1 else 0 end as OBESITY
, case when nutrdrg   = 1 then 0 when wghtloss = 1 then 1 else 0 end as WEIGHT_LOSS
, case when nutrdrg   = 1 then 0 when lytes = 1 then 1 else 0 end as FLUID_ELECTROLYTE
, case when anemdrg   = 1 then 0 when bldloss = 1 then 1 else 0 end as BLOOD_LOSS_ANEMIA
, case when anemdrg   = 1 then 0 when anemdef = 1 then 1 else 0 end as DEFICIENCY_ANEMIAS
, case when alcdrg    = 1 then 0 when alcohol = 1 then 1 else 0 end as ALCOHOL_ABUSE
, case when alcdrg    = 1 then 0 when drug = 1 then 1 else 0 end as DRUG_ABUSE
, case when psydrg    = 1 then 0 when psych = 1 then 1 else 0 end as PSYCHOSES
, case when deprsdrg  = 1 then 0 when depress = 1 then 1 else 0 end as DEPRESSION


from admissions adm
left join eligrp eli
  on adm.hadm_id = eli.hadm_id and adm.subject_id = eli.subject_id
left join drggrp d
  on adm.hadm_id = d.hadm_id and adm.subject_id = d.subject_id
order by adm.hadm_id;
"""
cur.execute(query_elixhauser_ahrq)

In [18]:
query_comorbidity= query_schema+"""
drop materialized view if exists comorbidity CASCADE; 
create materialized view comorbidity as
select subject_id,hadm_id,congestive_heart_failure,hypertension
,diabetes_uncomplicated,diabetes_complicated,renal_failure,liver_disease
,lymphoma,metastatic_cancer,solid_tumor
from elixhauser_ahrq
"""
cur.execute(query_comorbidity)

In [19]:
query_fluid_intake = query_schema+"""
drop materialized view if exists fluid_intake CASCADE; 
create materialized view fluid_intake as
with t3 as
(
select
        co.icustay_id,
        
        cast(DATE_PART('day', mv.starttime-co.intime) as integer)*24+
        cast(DATE_PART('hour', mv.starttime-co.intime) as integer)+
        cast(DATE_PART('minute', mv.starttime-co.intime) as integer)/60+
        cast(DATE_PART('second', mv.starttime-co.intime) as integer)/60/60 as hr
        
--      , DATETIME_DIFF(mv.starttime, co.intime, SECOND)/60.0/60.0 as hr
      -- standardize the units to millilitres
      -- also metavision has floating point precision.. but we only care down to the mL
      , round(case
          when mv.amountuom = 'L'
            then mv.amount * 1000.0
          when mv.amountuom = 'ml'
            then mv.amount
        else 0 end) as amount
        , case
            when itemid = 225158 then 'NaCl 0.9%'
            when itemid = 225828 then 'LR'
            when itemid = 225944 then 'Sterile Water'
            when itemid = 225797 then 'Free Water'
          else null end as label
      from icustays co
      inner join inputevents_mv mv
      on co.icustay_id = mv.icustay_id
      and mv.itemid in
      (
        -- 225943 Solution
        225158, -- NaCl 0.9%
        225828, -- LR
        225944, -- Sterile Water
        225797  -- Free Water
      )
      where mv.statusdescription != 'Rewritten'
      and mv.starttime-co.intime<interval '2' day  and mv.starttime>co.intime
      and
      -- in MetaVision, these ITEMIDs appear with a null rate IFF endtime=starttime + 1 minute
      -- so it is sufficient to:
      --    (1) check the rate is > 240 if it exists or
      --    (2) ensure the rate is null and amount > 240 ml
        (
          (mv.rate is not null and mv.rateuom = 'mL/hour' and mv.rate > 248)
          OR (mv.rate is not null and mv.rateuom = 'mL/min' and mv.rate > (248/60.0))
          OR (mv.rate is null and mv.amountuom = 'L' and mv.amount > 0.248)
          OR (mv.rate is null and mv.amountuom = 'ml' and mv.amount > 248)
          )
)

, t1 as
(
select * 
, case
            when label = 'NaCl 0.9%' then amount*154
            when label = 'LR' then amount* 109
            else 0 end as chloride_meq
from t3
)

, t4 as
(
select
        co.icustay_id,
        
        cast(DATE_PART('day', cv.charttime-co.intime) as integer)*24+
        cast(DATE_PART('hour', cv.charttime-co.intime) as integer)+
        cast(DATE_PART('minute', cv.charttime-co.intime) as integer)/60+
        cast(DATE_PART('second', cv.charttime-co.intime) as integer)/60/60 as hr
        
 --     , DATETIME_DIFF(cv.charttime, co.intime, SECOND)/60.0/60.0 as hr
      -- carevue always has units in millilitres
      , round(cv.amount) as amount
      , case
            when itemid = 30018 then '.9% Normal Saline'
            when itemid = 30021 then 'Lactated Ringers'
            when itemid = 30058 then 'Free Water Bolus'
            when itemid = 40850 then 'ns bolus'
            when itemid = 41491 then 'fluid bolus'
            when itemid = 42639 then 'bolus'
            when itemid = 30065 then 'Sterile Water'
            when itemid = 42187 then 'free h20'
            when itemid = 43819 then '1:1 NS Repletion'
            when itemid = 30063 then 'IV Piggyback'
            when itemid = 41430 then 'free water boluses'
            when itemid = 40712 then 'free H20'
            when itemid = 44160 then 'BOLUS'
            when itemid = 42383 then 'cc for cc replace'
            when itemid = 30169 then 'Sterile H20_GU'
            when itemid = 42297 then 'Fluid bolus'
            when itemid = 42453 then 'Fluid Bolus'
            when itemid = 40872 then 'free water'
            when itemid = 41915 then 'FREE WATER'
            when itemid = 41490 then 'NS bolus'
            when itemid = 46501 then 'H2O Bolus'
            when itemid = 45045 then 'WaterBolus'
            when itemid = 41984 then 'FREE H20'
            when itemid = 41371 then 'ns fluid bolus'
            when itemid = 41582 then 'free h20 bolus'
            when itemid = 41322 then 'rl bolus'
            when itemid = 40778 then 'Free H2O'
            when itemid = 41896 then 'ivf boluses'
            when itemid = 41428 then 'ns .9% bolus'
            when itemid = 43936 then 'FREE WATER BOLUSES'
            when itemid = 44200 then 'FLUID BOLUS'
            when itemid = 41619 then 'frfee water boluses'
            when itemid = 40424 then 'free H2O'
            when itemid = 41457 then 'Free H20 intake'
            when itemid = 41581 then 'Water bolus'
            when itemid = 42844 then 'NS fluid bolus'
            when itemid = 42429 then 'Free water'
            when itemid = 41356 then 'IV Bolus'
            when itemid = 40532 then 'FREE H2O'
            when itemid = 42548 then 'NS Bolus'
            when itemid = 44184 then 'LR Bolus'
            when itemid = 44521 then 'LR bolus'
            when itemid = 44741 then 'NS FLUID BOLUS'
            when itemid = 44126 then 'fl bolus'
            when itemid = 44110 then 'RL BOLUS'
            when itemid = 44633 then 'ns boluses'
            when itemid = 44983 then 'Bolus NS'
            when itemid = 44815 then 'LR BOLUS'
            when itemid = 43986 then 'iv bolus'
            when itemid = 45079 then '500 cc ns bolus'
            when itemid = 46781 then 'lr bolus'
            when itemid = 45155 then 'ns cc/cc replacement'
            when itemid = 43909 then 'H20 BOlus'
            when itemid = 41467 then 'NS IV bolus'
            when itemid = 44367 then 'LR'
            when itemid = 41743 then 'water bolus'
            when itemid = 40423 then 'Bolus'
            when itemid = 44263 then 'fluid bolus ns'
            when itemid = 42749 then 'fluid bolus NS'
            when itemid = 45480 then '500cc ns bolus'
            when itemid = 44491 then '.9NS bolus'
            when itemid = 41695 then 'NS fluid boluses'
            when itemid = 46169 then 'free water bolus'
            when itemid = 41580 then 'free h2o bolus'
            when itemid = 41392 then 'ns b'
            when itemid = 45989 then 'NS Fluid Bolus'
            when itemid = 45137 then 'NS cc/cc'
            when itemid = 45154 then 'Free H20 bolus'
            when itemid = 44053 then 'normal saline bolus'
            when itemid = 41416 then 'free h2o boluses'
            when itemid = 44761 then 'Free H20'
            when itemid = 41237 then 'ns fluid boluses'
            when itemid = 44426 then 'bolus ns'
            when itemid = 43975 then 'FREE H20 BOLUSES'
            when itemid = 44894 then 'N/s 500 ml bolus'
            when itemid = 41380 then 'nsbolus'
            when itemid = 42671 then 'free h2o'
          else null end as label
      from icustays co
      inner join inputevents_cv cv
      on co.icustay_id = cv.icustay_id
      and cv.itemid in
      (
        30018 --	.9% Normal Saline
      , 30021 --	Lactated Ringers
      , 30058 --	Free Water Bolus
      , 40850 --	ns bolus
      , 41491 --	fluid bolus
      , 42639 --	bolus
      , 30065 --	Sterile Water
      , 42187 --	free h20
      , 43819 --	1:1 NS Repletion.
      , 30063 --	IV Piggyback
      , 41430 --	free water boluses
      , 40712 --	free H20
      , 44160 --	BOLUS
      , 42383 --	cc for cc replace
      , 30169 --	Sterile H20_GU
      , 42297 --	Fluid bolus
      , 42453 --	Fluid Bolus
      , 40872 --	free water
      , 41915 --	FREE WATER
      , 41490 --	NS bolus
      , 46501 --	H2O Bolus
      , 45045 --	WaterBolus
      , 41984 --	FREE H20
      , 41371 --	ns fluid bolus
      , 41582 --	free h20 bolus
      , 41322 --	rl bolus
      , 40778 --	Free H2O
      , 41896 --	ivf boluses
      , 41428 --	ns .9% bolus
      , 43936 --	FREE WATER BOLUSES
      , 44200 --	FLUID BOLUS
      , 41619 --	frfee water boluses
      , 40424 --	free H2O
      , 41457 --	Free H20 intake
      , 41581 --	Water bolus
      , 42844 --	NS fluid bolus
      , 42429 --	Free water
      , 41356 --	IV Bolus
      , 40532 --	FREE H2O
      , 42548 --	NS Bolus
      , 44184 --	LR Bolus
      , 44521 --	LR bolus
      , 44741 --	NS FLUID BOLUS
      , 44126 --	fl bolus
      , 44110 --	RL BOLUS
      , 44633 --	ns boluses
      , 44983 --	Bolus NS
      , 44815 --	LR BOLUS
      , 43986 --	iv bolus
      , 45079 --	500 cc ns bolus
      , 46781 --	lr bolus
      , 45155 --	ns cc/cc replacement
      , 43909 --	H20 BOlus
      , 41467 --	NS IV bolus
      , 44367 --	LR
      , 41743 --	water bolus
      , 40423 --	Bolus
      , 44263 --	fluid bolus ns
      , 42749 --	fluid bolus NS
      , 45480 --	500cc ns bolus
      , 44491 --	.9NS bolus
      , 41695 --	NS fluid boluses
      , 46169 --	free water bolus.
      , 41580 --	free h2o bolus
      , 41392 --	ns b
      , 45989 --	NS Fluid Bolus
      , 45137 --	NS cc/cc
      , 45154 --	Free H20 bolus
      , 44053 --	normal saline bolus
      , 41416 --	free h2o boluses
      , 44761 --	Free H20
      , 41237 --	ns fluid boluses
      , 44426 --	bolus ns
      , 43975 --	FREE H20 BOLUSES
      , 44894 --	N/s 500 ml bolus
      , 41380 --	nsbolus
      , 42671 --	free h2o
      )
      where cv.amount > 248
      and cv.amount < 2000
      and cv.amountuom = 'ml'
      and cv.charttime-co.intime < interval '2' day and cv.charttime>co.intime
)
, t2 as
(
select *
,case
            when label = '.9% Normal Saline' OR label = 'ns bolus' OR label = 'NS bolus'
            OR label = 'ns fluid bolus' OR label = 'ns .9% bolus' OR label = 'NS fluid bolus' OR label = 'NS Bolus'
            OR label = 'NS FLUID BOLUS' OR label = 'ns boluses' OR label = 'Bolus NS' OR label = '500 cc ns bolus'
            OR label = 'ns cc/cc replacement' OR label = 'NS IV bolus' OR label = 'fluid bolus ns'
            OR label = 'fluid bolus NS' OR label = '500cc ns bolus' OR label = '.9NS bolus' OR label = 'NS fluid boluses'
            OR label = 'ns b' OR label = 'NS Fluid Bolus' OR label = 'NS cc/cc' OR label = 'normal saline bolus'
            OR label = 'ns fluid boluses' OR label = 'bolus ns' OR label = 'N/s 500 ml bolus' 
            OR label = 'nsbolus' 
            then amount*154
            when label = 'Lactated Ringers' OR label = 'rl bolus' OR label = 'LR Bolus' OR label = 'LR bolus'
            OR label = 'RL BOLUS' OR label = 'LR BOLUS' OR label = 'lr bolus' OR label = 'LR' 
            then amount *109
            when label = '1:1 NS Repletion' then amount*154/2   
            else 0 end as chloride_meq
FROM t4
)

select
    icustay_id
  , hr
  , sum(amount) as crystalloid_bolus
  , sum(chloride_meq) as chloride_meq_sum
from t1
-- just because the rate was high enough, does *not* mean the final amount was
where amount > 248
group by t1.icustay_id, t1.hr
UNION ALL
select
    icustay_id
  , hr
  , sum(amount) as crystalloid_bolus
  , sum(chloride_meq) as chloride_meq_sum
from t2
group by t2.icustay_id, t2.hr
order by icustay_id, hr;
"""
cur.execute(query_fluid_intake)
con.commit()

In [48]:
query_new_fluid_intake = query_schema+"""
drop materialized view if exists new_fluid_intake CASCADE; 
create materialized view new_fluid_intake as
with t3 as
(
select
        co.icustay_id,
        
        cast(DATE_PART('day', mv.starttime-co.intime) as integer)*24+
        cast(DATE_PART('hour', mv.starttime-co.intime) as integer)+
        cast(DATE_PART('minute', mv.starttime-co.intime) as integer)/60+
        cast(DATE_PART('second', mv.starttime-co.intime) as integer)/60/60 as hr
        
--      , DATETIME_DIFF(mv.starttime, co.intime, SECOND)/60.0/60.0 as hr
      -- standardize the units to millilitres
      -- also metavision has floating point precision.. but we only care down to the mL
      , round(case
          when mv.amountuom = 'L'
            then mv.amount * 1000.0
          when mv.amountuom = 'ml'
            then mv.amount
        else 0 end) as amount
        , case
            when itemid = 225158 then 'NaCl 0.9%'
            when itemid = 225828 then 'LR'
            when itemid = 225944 then 'Sterile Water'
            when itemid = 225797 then 'Free Water'
          else null end as label
      from icustays co
      inner join inputevents_mv mv
      on co.icustay_id = mv.icustay_id
      and mv.itemid in
      (
        -- 225943 Solution
        225158, -- NaCl 0.9%
        225828, -- LR
        225944, -- Sterile Water
        225797  -- Free Water
      )
      where mv.statusdescription != 'Rewritten'
      and mv.starttime-co.intime<interval '2' day  and mv.starttime>co.intime
      and
      -- in MetaVision, these ITEMIDs appear with a null rate IFF endtime=starttime + 1 minute
      -- so it is sufficient to:
      --    (1) check the rate is > 240 if it exists or
      --    (2) ensure the rate is null and amount > 240 ml
        (
          (mv.rate is not null and mv.rateuom = 'mL/hour' and mv.rate > 248)
          OR (mv.rate is not null and mv.rateuom = 'mL/min' and mv.rate > (248/60.0))
          OR (mv.rate is null and mv.amountuom = 'L' and mv.amount > 0.248)
          OR (mv.rate is null and mv.amountuom = 'ml' and mv.amount > 248)
          )
)

, t1 as
(
select * 
, case
            when label = 'NaCl 0.9%' then amount*154
            when label = 'LR' then amount* 109
            else 0 end as chloride_meq
, case
            when label = 'NaCl 0.9%' then amount*154
            when label = 'LR' then amount* 130
            else 0 end as sodium_meq
from t3
)

, t4 as
(
select
        co.icustay_id,
        
        cast(DATE_PART('day', cv.charttime-co.intime) as integer)*24+
        cast(DATE_PART('hour', cv.charttime-co.intime) as integer)+
        cast(DATE_PART('minute', cv.charttime-co.intime) as integer)/60+
        cast(DATE_PART('second', cv.charttime-co.intime) as integer)/60/60 as hr
        
 --     , DATETIME_DIFF(cv.charttime, co.intime, SECOND)/60.0/60.0 as hr
      -- carevue always has units in millilitres
      , round(cv.amount) as amount
      , case
            when itemid = 30018 then '.9% Normal Saline'
            when itemid = 30021 then 'Lactated Ringers'
            when itemid = 30058 then 'Free Water Bolus'
            when itemid = 40850 then 'ns bolus'
            when itemid = 41491 then 'fluid bolus'
            when itemid = 42639 then 'bolus'
            when itemid = 30065 then 'Sterile Water'
            when itemid = 42187 then 'free h20'
            when itemid = 43819 then '1:1 NS Repletion'
            when itemid = 30063 then 'IV Piggyback'
            when itemid = 41430 then 'free water boluses'
            when itemid = 40712 then 'free H20'
            when itemid = 44160 then 'BOLUS'
            when itemid = 42383 then 'cc for cc replace'
            when itemid = 30169 then 'Sterile H20_GU'
            when itemid = 42297 then 'Fluid bolus'
            when itemid = 42453 then 'Fluid Bolus'
            when itemid = 40872 then 'free water'
            when itemid = 41915 then 'FREE WATER'
            when itemid = 41490 then 'NS bolus'
            when itemid = 46501 then 'H2O Bolus'
            when itemid = 45045 then 'WaterBolus'
            when itemid = 41984 then 'FREE H20'
            when itemid = 41371 then 'ns fluid bolus'
            when itemid = 41582 then 'free h20 bolus'
            when itemid = 41322 then 'rl bolus'
            when itemid = 40778 then 'Free H2O'
            when itemid = 41896 then 'ivf boluses'
            when itemid = 41428 then 'ns .9% bolus'
            when itemid = 43936 then 'FREE WATER BOLUSES'
            when itemid = 44200 then 'FLUID BOLUS'
            when itemid = 41619 then 'frfee water boluses'
            when itemid = 40424 then 'free H2O'
            when itemid = 41457 then 'Free H20 intake'
            when itemid = 41581 then 'Water bolus'
            when itemid = 42844 then 'NS fluid bolus'
            when itemid = 42429 then 'Free water'
            when itemid = 41356 then 'IV Bolus'
            when itemid = 40532 then 'FREE H2O'
            when itemid = 42548 then 'NS Bolus'
            when itemid = 44184 then 'LR Bolus'
            when itemid = 44521 then 'LR bolus'
            when itemid = 44741 then 'NS FLUID BOLUS'
            when itemid = 44126 then 'fl bolus'
            when itemid = 44110 then 'RL BOLUS'
            when itemid = 44633 then 'ns boluses'
            when itemid = 44983 then 'Bolus NS'
            when itemid = 44815 then 'LR BOLUS'
            when itemid = 43986 then 'iv bolus'
            when itemid = 45079 then '500 cc ns bolus'
            when itemid = 46781 then 'lr bolus'
            when itemid = 45155 then 'ns cc/cc replacement'
            when itemid = 43909 then 'H20 BOlus'
            when itemid = 41467 then 'NS IV bolus'
            when itemid = 44367 then 'LR'
            when itemid = 41743 then 'water bolus'
            when itemid = 40423 then 'Bolus'
            when itemid = 44263 then 'fluid bolus ns'
            when itemid = 42749 then 'fluid bolus NS'
            when itemid = 45480 then '500cc ns bolus'
            when itemid = 44491 then '.9NS bolus'
            when itemid = 41695 then 'NS fluid boluses'
            when itemid = 46169 then 'free water bolus'
            when itemid = 41580 then 'free h2o bolus'
            when itemid = 41392 then 'ns b'
            when itemid = 45989 then 'NS Fluid Bolus'
            when itemid = 45137 then 'NS cc/cc'
            when itemid = 45154 then 'Free H20 bolus'
            when itemid = 44053 then 'normal saline bolus'
            when itemid = 41416 then 'free h2o boluses'
            when itemid = 44761 then 'Free H20'
            when itemid = 41237 then 'ns fluid boluses'
            when itemid = 44426 then 'bolus ns'
            when itemid = 43975 then 'FREE H20 BOLUSES'
            when itemid = 44894 then 'N/s 500 ml bolus'
            when itemid = 41380 then 'nsbolus'
            when itemid = 42671 then 'free h2o'
          else null end as label
      from icustays co
      inner join inputevents_cv cv
      on co.icustay_id = cv.icustay_id
      and cv.itemid in
      (
        30018 --	.9% Normal Saline
      , 30021 --	Lactated Ringers
      , 30058 --	Free Water Bolus
      , 40850 --	ns bolus
      , 41491 --	fluid bolus
      , 42639 --	bolus
      , 30065 --	Sterile Water
      , 42187 --	free h20
      , 43819 --	1:1 NS Repletion.
      , 30063 --	IV Piggyback
      , 41430 --	free water boluses
      , 40712 --	free H20
      , 44160 --	BOLUS
      , 42383 --	cc for cc replace
      , 30169 --	Sterile H20_GU
      , 42297 --	Fluid bolus
      , 42453 --	Fluid Bolus
      , 40872 --	free water
      , 41915 --	FREE WATER
      , 41490 --	NS bolus
      , 46501 --	H2O Bolus
      , 45045 --	WaterBolus
      , 41984 --	FREE H20
      , 41371 --	ns fluid bolus
      , 41582 --	free h20 bolus
      , 41322 --	rl bolus
      , 40778 --	Free H2O
      , 41896 --	ivf boluses
      , 41428 --	ns .9% bolus
      , 43936 --	FREE WATER BOLUSES
      , 44200 --	FLUID BOLUS
      , 41619 --	frfee water boluses
      , 40424 --	free H2O
      , 41457 --	Free H20 intake
      , 41581 --	Water bolus
      , 42844 --	NS fluid bolus
      , 42429 --	Free water
      , 41356 --	IV Bolus
      , 40532 --	FREE H2O
      , 42548 --	NS Bolus
      , 44184 --	LR Bolus
      , 44521 --	LR bolus
      , 44741 --	NS FLUID BOLUS
      , 44126 --	fl bolus
      , 44110 --	RL BOLUS
      , 44633 --	ns boluses
      , 44983 --	Bolus NS
      , 44815 --	LR BOLUS
      , 43986 --	iv bolus
      , 45079 --	500 cc ns bolus
      , 46781 --	lr bolus
      , 45155 --	ns cc/cc replacement
      , 43909 --	H20 BOlus
      , 41467 --	NS IV bolus
      , 44367 --	LR
      , 41743 --	water bolus
      , 40423 --	Bolus
      , 44263 --	fluid bolus ns
      , 42749 --	fluid bolus NS
      , 45480 --	500cc ns bolus
      , 44491 --	.9NS bolus
      , 41695 --	NS fluid boluses
      , 46169 --	free water bolus.
      , 41580 --	free h2o bolus
      , 41392 --	ns b
      , 45989 --	NS Fluid Bolus
      , 45137 --	NS cc/cc
      , 45154 --	Free H20 bolus
      , 44053 --	normal saline bolus
      , 41416 --	free h2o boluses
      , 44761 --	Free H20
      , 41237 --	ns fluid boluses
      , 44426 --	bolus ns
      , 43975 --	FREE H20 BOLUSES
      , 44894 --	N/s 500 ml bolus
      , 41380 --	nsbolus
      , 42671 --	free h2o
      )
      where cv.amount > 248
      and cv.amount < 2000
      and cv.amountuom = 'ml'
      and cv.charttime-co.intime < interval '2' day and cv.charttime>co.intime
)
, t2 as
(
select *
,case
            when label = '.9% Normal Saline' OR label = 'ns bolus' OR label = 'NS bolus'
            OR label = 'ns fluid bolus' OR label = 'ns .9% bolus' OR label = 'NS fluid bolus' OR label = 'NS Bolus'
            OR label = 'NS FLUID BOLUS' OR label = 'ns boluses' OR label = 'Bolus NS' OR label = '500 cc ns bolus'
            OR label = 'ns cc/cc replacement' OR label = 'NS IV bolus' OR label = 'fluid bolus ns'
            OR label = 'fluid bolus NS' OR label = '500cc ns bolus' OR label = '.9NS bolus' OR label = 'NS fluid boluses'
            OR label = 'ns b' OR label = 'NS Fluid Bolus' OR label = 'NS cc/cc' OR label = 'normal saline bolus'
            OR label = 'ns fluid boluses' OR label = 'bolus ns' OR label = 'N/s 500 ml bolus' 
            OR label = 'nsbolus' 
            then amount*154
            when label = 'Lactated Ringers' OR label = 'rl bolus' OR label = 'LR Bolus' OR label = 'LR bolus'
            OR label = 'RL BOLUS' OR label = 'LR BOLUS' OR label = 'lr bolus' OR label = 'LR' 
            then amount *109
            when label = '1:1 NS Repletion' then amount*154/2   
            else 0 end as chloride_meq
,case
            when label = '.9% Normal Saline' OR label = 'ns bolus' OR label = 'NS bolus'
            OR label = 'ns fluid bolus' OR label = 'ns .9% bolus' OR label = 'NS fluid bolus' OR label = 'NS Bolus'
            OR label = 'NS FLUID BOLUS' OR label = 'ns boluses' OR label = 'Bolus NS' OR label = '500 cc ns bolus'
            OR label = 'ns cc/cc replacement' OR label = 'NS IV bolus' OR label = 'fluid bolus ns'
            OR label = 'fluid bolus NS' OR label = '500cc ns bolus' OR label = '.9NS bolus' OR label = 'NS fluid boluses'
            OR label = 'ns b' OR label = 'NS Fluid Bolus' OR label = 'NS cc/cc' OR label = 'normal saline bolus'
            OR label = 'ns fluid boluses' OR label = 'bolus ns' OR label = 'N/s 500 ml bolus' 
            OR label = 'nsbolus' 
            then amount*154
            when label = 'Lactated Ringers' OR label = 'rl bolus' OR label = 'LR Bolus' OR label = 'LR bolus'
            OR label = 'RL BOLUS' OR label = 'LR BOLUS' OR label = 'lr bolus' OR label = 'LR' 
            then amount *130
            when label = '1:1 NS Repletion' then amount*154/2   
            else 0 end as sodium_meq
FROM t4
)

select
    icustay_id
  , hr
  , sum(amount) as crystalloid_bolus
  , sum(chloride_meq) as chloride_meq_sum
  , sum(sodium_meq) as sodium_meq_sum
from t1
-- just because the rate was high enough, does *not* mean the final amount was
where amount > 248
group by t1.icustay_id, t1.hr
UNION ALL
select
    icustay_id
  , hr
  , sum(amount) as crystalloid_bolus
  , sum(chloride_meq) as sodium_meq_sum
  , sum(sodium_meq) as sodium_meq_sum
from t2
group by t2.icustay_id, t2.hr
order by icustay_id, hr;
"""
cur.execute(query_new_fluid_intake)
con.commit()

In [51]:
query_new_fluid_intake_sum = query_schema+"""
drop materialized view if exists new_fluid_intake_sum cascade; 
create materialized view new_fluid_intake_sum as
select icustay_id,sum(crystalloid_bolus) as total_crystalloid_bolus
, sum(chloride_meq_sum) as total_chloride_meq_sum
, sum(sodium_meq_sum) as total_sodium_meq_sum
from new_fluid_intake
group by icustay_id
"""
cur.execute(query_new_fluid_intake_sum)

In [27]:
query_icustay_detail_1 = query_schema+"""
drop materialized view if exists icustay_detail_1 cascade; 
create materialized view icustay_detail_1 as
with sur as
(
select subject_id, hadm_id, transfertime
, case when prev_service like '%SURG%' or curr_service like '%SURG%' then 1
       else 0 end as surgery
from services
)

, id_sur as
(
select *
from icustay_detail id
left join sur
    on id.hadm_id = sur.hadm_id
)

, t1 as 
(
select *
, case when 
id_sur.transfertime between id_sur.admittime and (id_sur.intime + interval '7' day) AND id_sur.surgery = 1 then 1
else 0 end as icu_surgery
from id_sur
)

, t2 as
(
select icustay_id, sum(icu_surgery) as sum_icu_surge
-- `sum_icu_surge=0` means no surgery pre- or during ICUD 7
from t1
group by t1.icustay_id
)

select subject_id,hadm_id,id.icustay_id,gender,dod,dod_hosp,expire_flag
,admittime,dischtime,los_hospital,admission_age,ethnicity,admission_type
,hospital_expire_flag,hospstay_seq,first_hosp_stay,intime,outtime
,los_icu,icustay_seq,first_icu_stay,sum_icu_surge
from icustay_detail id
left join t2
     on id.icustay_id = t2.icustay_id
"""
cur.execute(query_icustay_detail_1)
con.commit()
#`sum_icu_surge=0` means no surgery pre- or during ICUD 7

In [32]:
# New
# Combine icustay_detail and kdigo tables
# uo6_48hr et al are different between kdigo_stages_48hr and kdigo_stages_7day
# : because minimum of urine output in 6 hour in any phase in 48h or 7d
query_new_kdigo= query_schema+"""
DROP MATERIALIZED VIEW IF EXISTS new_kdigo CASCADE;
CREATE MATERIALIZED VIEW new_kdigo AS
SELECT icu.subject_id, icu.hadm_id, icu.icustay_id, icu.gender
,icu.dod, icu.dod_hosp, icu.expire_flag, icu.admittime, icu.dischtime
,icu.los_hospital, icu.admission_age, icu.ethnicity, icu.hospital_expire_flag
,icu.hospstay_seq, icu.first_hosp_stay, icu.intime, icu.outtime, icu.los_icu, icu.icustay_seq
,icu.first_icu_stay,icu.sum_icu_surge
,k48.aki_48hr, k48.aki_stage_48hr, k48.highcreat48hr,k48.rrt_2d
,k7.aki_7day, k7.aki_stage_7day, k7.highcreat7day, k7.BaseCre, k7.BaseCreTime,k7.rrt_7d
FROM icustay_detail_1 icu
LEFT JOIN new_kdigo_stages_48hr k48
on icu.icustay_id = k48.icustay_id
LEFT JOIN new_kdigo_stages_7day k7
on icu.icustay_id = k7.icustay_id
"""
cur.execute(query_new_kdigo)
con.commit()

# Creating APACHE score

In [33]:
query_bloodgasfirstday=query_schema+"""
DROP MATERIALIZED VIEW IF EXISTS bloodgasfirstday CASCADE;
create materialized view bloodgasfirstday as
with pvt as
( -- begin query that extracts the data
  select ie.subject_id, ie.hadm_id, ie.icustay_id
  -- here we assign labels to ITEMIDs
  -- this also fuses together multiple ITEMIDs containing the same data
      , case
        when itemid = 50800 then 'SPECIMEN'
        when itemid = 50801 then 'AADO2'
        when itemid = 50802 then 'BASEEXCESS'
        when itemid = 50803 then 'BICARBONATE'
        when itemid = 50804 then 'TOTALCO2'
        when itemid = 50805 then 'CARBOXYHEMOGLOBIN'
        when itemid = 50806 then 'CHLORIDE'
        when itemid = 50808 then 'CALCIUM'
        when itemid = 50809 then 'GLUCOSE'
        when itemid = 50810 then 'HEMATOCRIT'
        when itemid = 50811 then 'HEMOGLOBIN'
        when itemid = 50812 then 'INTUBATED'
        when itemid = 50813 then 'LACTATE'
        when itemid = 50814 then 'METHEMOGLOBIN'
        when itemid = 50815 then 'O2FLOW'
        when itemid = 50816 then 'FIO2'
        when itemid = 50817 then 'SO2' -- OXYGENSATURATION
        when itemid = 50818 then 'PCO2'
        when itemid = 50819 then 'PEEP'
        when itemid = 50820 then 'PH'
        when itemid = 50821 then 'PO2'
        when itemid = 50822 then 'POTASSIUM'
        when itemid = 50823 then 'REQUIREDO2'
        when itemid = 50824 then 'SODIUM'
        when itemid = 50825 then 'TEMPERATURE'
        when itemid = 50826 then 'TIDALVOLUME'
        when itemid = 50827 then 'VENTILATIONRATE'
        when itemid = 50828 then 'VENTILATOR'
        else null
        end as label
        , charttime
        , value
        -- add in some sanity checks on the values
        , case
          when valuenum <= 0 then null
          when itemid = 50810 and valuenum > 100 then null -- hematocrit
          -- ensure FiO2 is a valid number between 21-100
          -- mistakes are rare (<100 obs out of ~100,000)
          -- there are 862 obs of valuenum == 20 - some people round down!
          -- rather than risk imputing garbage data for FiO2, we simply NULL invalid values
          when itemid = 50816 and valuenum < 20 then null
          when itemid = 50816 and valuenum > 100 then null
          when itemid = 50817 and valuenum > 100 then null -- O2 sat
          when itemid = 50815 and valuenum >  70 then null -- O2 flow
          when itemid = 50821 and valuenum > 800 then null -- PO2
           -- conservative upper limit
        else valuenum
        end as valuenum

    from icustays ie
    left join labevents le
      on le.subject_id = ie.subject_id and le.hadm_id = ie.hadm_id
      and le.charttime between (ie.intime - interval '6' hour) and (ie.intime + interval '1' day)
      and le.ITEMID in
      -- blood gases
      (
        50800, 50801, 50802, 50803, 50804, 50805, 50806, 50807, 50808, 50809
        , 50810, 50811, 50812, 50813, 50814, 50815, 50816, 50817, 50818, 50819
        , 50820, 50821, 50822, 50823, 50824, 50825, 50826, 50827, 50828
        , 51545
      )
)
select pvt.SUBJECT_ID, pvt.HADM_ID, pvt.ICUSTAY_ID, pvt.CHARTTIME
, max(case when label = 'SPECIMEN' then value else null end) as SPECIMEN
, max(case when label = 'AADO2' then valuenum else null end) as AADO2
, max(case when label = 'BASEEXCESS' then valuenum else null end) as BASEEXCESS
, max(case when label = 'BICARBONATE' then valuenum else null end) as BICARBONATE
, max(case when label = 'TOTALCO2' then valuenum else null end) as TOTALCO2
, max(case when label = 'CARBOXYHEMOGLOBIN' then valuenum else null end) as CARBOXYHEMOGLOBIN
, max(case when label = 'CHLORIDE' then valuenum else null end) as CHLORIDE
, max(case when label = 'CALCIUM' then valuenum else null end) as CALCIUM
, max(case when label = 'GLUCOSE' then valuenum else null end) as GLUCOSE
, max(case when label = 'HEMATOCRIT' then valuenum else null end) as HEMATOCRIT
, max(case when label = 'HEMOGLOBIN' then valuenum else null end) as HEMOGLOBIN
, max(case when label = 'INTUBATED' then valuenum else null end) as INTUBATED
, max(case when label = 'LACTATE' then valuenum else null end) as LACTATE
, max(case when label = 'METHEMOGLOBIN' then valuenum else null end) as METHEMOGLOBIN
, max(case when label = 'O2FLOW' then valuenum else null end) as O2FLOW
, max(case when label = 'FIO2' then valuenum else null end) as FIO2
, max(case when label = 'SO2' then valuenum else null end) as SO2 -- OXYGENSATURATION
, max(case when label = 'PCO2' then valuenum else null end) as PCO2
, max(case when label = 'PEEP' then valuenum else null end) as PEEP
, max(case when label = 'PH' then valuenum else null end) as PH
, max(case when label = 'PO2' then valuenum else null end) as PO2
, max(case when label = 'POTASSIUM' then valuenum else null end) as POTASSIUM
, max(case when label = 'REQUIREDO2' then valuenum else null end) as REQUIREDO2
, max(case when label = 'SODIUM' then valuenum else null end) as SODIUM
, max(case when label = 'TEMPERATURE' then valuenum else null end) as TEMPERATURE
, max(case when label = 'TIDALVOLUME' then valuenum else null end) as TIDALVOLUME
, max(case when label = 'VENTILATIONRATE' then valuenum else null end) as VENTILATIONRATE
, max(case when label = 'VENTILATOR' then valuenum else null end) as VENTILATOR
from pvt
group by pvt.subject_id, pvt.hadm_id, pvt.icustay_id, pvt.CHARTTIME
order by pvt.subject_id, pvt.hadm_id, pvt.icustay_id, pvt.CHARTTIME;
"""
cur.execute(query_bloodgasfirstday)

In [34]:
query_bloodgasfirstdayarterial= query_schema+"""
DROP MATERIALIZED VIEW IF EXISTS bloodgasfirstdayarterial CASCADE;
CREATE MATERIALIZED VIEW bloodgasfirstdayarterial AS
with stg_spo2 as
(
  select SUBJECT_ID, HADM_ID, ICUSTAY_ID, CHARTTIME
    -- max here is just used to group SpO2 by charttime
    , max(case when valuenum <= 0 or valuenum > 100 then null else valuenum end) as SpO2
  from CHARTEVENTS
  -- o2 sat
  where ITEMID in
  (
    646 -- SpO2
  , 220277 -- O2 saturation pulseoxymetry
  )
  group by SUBJECT_ID, HADM_ID, ICUSTAY_ID, CHARTTIME
)
, stg_fio2 as
(
  select SUBJECT_ID, HADM_ID, ICUSTAY_ID, CHARTTIME
    -- pre-process the FiO2s to ensure they are between 21-100%
    , max(
        case
          when itemid = 223835
            then case
              when valuenum > 0 and valuenum <= 1
                then valuenum * 100
              -- improperly input data - looks like O2 flow in litres
              when valuenum > 1 and valuenum < 21
                then null
              when valuenum >= 21 and valuenum <= 100
                then valuenum
              else null end -- unphysiological
        when itemid in (3420, 3422)
        -- all these values are well formatted
            then valuenum
        when itemid = 190 and valuenum > 0.20 and valuenum < 1
        -- well formatted but not in %
            then valuenum * 100
      else null end
    ) as fio2_chartevents
  from CHARTEVENTS
  where ITEMID in
  (
    3420 -- FiO2
  , 190 -- FiO2 set
  , 223835 -- Inspired O2 Fraction (FiO2)
  , 3422 -- FiO2 [measured]
  )
  -- exclude rows marked as error
  and error IS DISTINCT FROM 1
  group by SUBJECT_ID, HADM_ID, ICUSTAY_ID, CHARTTIME
)
, stg2 as
(
select bg.*
  , ROW_NUMBER() OVER (partition by bg.icustay_id, bg.charttime order by s1.charttime DESC) as lastRowSpO2
  , s1.spo2
from bloodgasfirstday bg
left join stg_spo2 s1
  -- same patient
  on  bg.icustay_id = s1.icustay_id
  -- spo2 occurred at most 2 hours before this blood gas
  and s1.charttime between bg.charttime - interval '2' hour and bg.charttime
where bg.po2 is not null
)
, stg3 as
(
select bg.*
  , ROW_NUMBER() OVER (partition by bg.icustay_id, bg.charttime order by s2.charttime DESC) as lastRowFiO2
  , s2.fio2_chartevents

  -- create our specimen prediction
  ,  1/(1+exp(-(-0.02544
  +    0.04598 * po2
  + coalesce(-0.15356 * spo2             , -0.15356 *   97.49420 +    0.13429)
  + coalesce( 0.00621 * fio2_chartevents ,  0.00621 *   51.49550 +   -0.24958)
  + coalesce( 0.10559 * hemoglobin       ,  0.10559 *   10.32307 +    0.05954)
  + coalesce( 0.13251 * so2              ,  0.13251 *   93.66539 +   -0.23172)
  + coalesce(-0.01511 * pco2             , -0.01511 *   42.08866 +   -0.01630)
  + coalesce( 0.01480 * fio2             ,  0.01480 *   63.97836 +   -0.31142)
  + coalesce(-0.00200 * aado2            , -0.00200 *  442.21186 +   -0.01328)
  + coalesce(-0.03220 * bicarbonate      , -0.03220 *   22.96894 +   -0.06535)
  + coalesce( 0.05384 * totalco2         ,  0.05384 *   24.72632 +   -0.01405)
  + coalesce( 0.08202 * lactate          ,  0.08202 *    3.06436 +    0.06038)
  + coalesce( 0.10956 * ph               ,  0.10956 *    7.36233 +   -0.00617)
  + coalesce( 0.00848 * o2flow           ,  0.00848 *    7.59362 +   -0.35803)
  ))) as SPECIMEN_PROB
from stg2 bg
left join stg_fio2 s2
  -- same patient
  on  bg.icustay_id = s2.icustay_id
  -- fio2 occurred at most 4 hours before this blood gas
  and s2.charttime between bg.charttime - interval '4' hour and bg.charttime
where bg.lastRowSpO2 = 1 -- only the row with the most recent SpO2 (if no SpO2 found lastRowSpO2 = 1)
)

select subject_id, hadm_id,
icustay_id, charttime
, SPECIMEN -- raw data indicating sample type, only present 80% of the time

-- prediction of specimen for missing data
, case
      when SPECIMEN is not null then SPECIMEN
      when SPECIMEN_PROB > 0.75 then 'ART'
    else null end as SPECIMEN_PRED
, SPECIMEN_PROB

-- oxygen related parameters
, SO2, spo2 -- note spo2 is from chartevents
, PO2, PCO2
, fio2_chartevents, FIO2
, AADO2
-- also calculate AADO2
, case
    when  PO2 is not null
      and pco2 is not null
      and coalesce(FIO2, fio2_chartevents) is not null
     -- multiple by 100 because FiO2 is in a % but should be a fraction
      then (coalesce(FIO2, fio2_chartevents)/100) * (760 - 47) - (pco2/0.8) - po2
    else null
  end as AADO2_calc
, case
    when PO2 is not null and coalesce(FIO2, fio2_chartevents) is not null
     -- multiply by 100 because FiO2 is in a % but should be a fraction
      then 100*PO2/(coalesce(FIO2, fio2_chartevents))
    else null
  end as PaO2FiO2
-- acid-base parameters
, PH, BASEEXCESS
, BICARBONATE, TOTALCO2

-- blood count parameters
, HEMATOCRIT
, HEMOGLOBIN
, CARBOXYHEMOGLOBIN
, METHEMOGLOBIN

-- chemistry
, CHLORIDE, CALCIUM
, TEMPERATURE
, POTASSIUM, SODIUM
, LACTATE
, GLUCOSE

-- ventilation stuff that's sometimes input
, INTUBATED, TIDALVOLUME, VENTILATIONRATE, VENTILATOR
, PEEP, O2Flow
, REQUIREDO2

from stg3
where lastRowFiO2 = 1 -- only the most recent FiO2
-- restrict it to *only* arterial samples
and (SPECIMEN = 'ART' or SPECIMEN_PROB > 0.75)
order by icustay_id, charttime;
"""
cur.execute(query_bloodgasfirstdayarterial)

In [35]:
query_ventilation_durations=query_schema+"""
DROP MATERIALIZED VIEW IF EXISTS ventsettings CASCADE;
CREATE MATERIALIZED VIEW ventsettings AS
select
  icustay_id, charttime
  -- case statement determining whether it is an instance of mech vent
  , max(
    case
      when itemid is null or value is null then 0 -- can't have null values
      when itemid = 720 and value != 'Other/Remarks' THEN 1  -- VentTypeRecorded
      when itemid = 223848 and value != 'Other' THEN 1
      when itemid = 223849 then 1 -- ventilator mode
      when itemid = 467 and value = 'Ventilator' THEN 1 -- O2 delivery device == ventilator
      when itemid in
        (
        445, 448, 449, 450, 1340, 1486, 1600, 224687 -- minute volume
        , 639, 654, 681, 682, 683, 684,224685,224684,224686 -- tidal volume
        , 218,436,535,444,459,224697,224695,224696,224746,224747 -- High/Low/Peak/Mean/Neg insp force ("RespPressure")
        , 221,1,1211,1655,2000,226873,224738,224419,224750,227187 -- Insp pressure
        , 543 -- PlateauPressure
        , 5865,5866,224707,224709,224705,224706 -- APRV pressure
        , 60,437,505,506,686,220339,224700 -- PEEP
        , 3459 -- high pressure relief
        , 501,502,503,224702 -- PCV
        , 223,667,668,669,670,671,672 -- TCPCV
        , 224701 -- PSVlevel
        )
        THEN 1
      else 0
    end
    ) as MechVent
    , max(
      case
        -- initiation of oxygen therapy indicates the ventilation has ended
        when itemid = 226732 and value in
        (
          'Nasal cannula', -- 153714 observations
          'Face tent', -- 24601 observations
          'Aerosol-cool', -- 24560 observations
          'Trach mask ', -- 16435 observations
          'High flow neb', -- 10785 observations
          'Non-rebreather', -- 5182 observations
          'Venti mask ', -- 1947 observations
          'Medium conc mask ', -- 1888 observations
          'T-piece', -- 1135 observations
          'High flow nasal cannula', -- 925 observations
          'Ultrasonic neb', -- 9 observations
          'Vapomist' -- 3 observations
        ) then 1
        when itemid = 467 and value in
        (
          'Cannula', -- 278252 observations
          'Nasal Cannula', -- 248299 observations
          -- 'None', -- 95498 observations
          'Face Tent', -- 35766 observations
          'Aerosol-Cool', -- 33919 observations
          'Trach Mask', -- 32655 observations
          'Hi Flow Neb', -- 14070 observations
          'Non-Rebreather', -- 10856 observations
          'Venti Mask', -- 4279 observations
          'Medium Conc Mask', -- 2114 observations
          'Vapotherm', -- 1655 observations
          'T-Piece', -- 779 observations
          'Hood', -- 670 observations
          'Hut', -- 150 observations
          'TranstrachealCat', -- 78 observations
          'Heated Neb', -- 37 observations
          'Ultrasonic Neb' -- 2 observations
        ) then 1
      else 0
      end
    ) as OxygenTherapy
    , max(
      case when itemid is null or value is null then 0
        -- extubated indicates ventilation event has ended
        when itemid = 640 and value = 'Extubated' then 1
        when itemid = 640 and value = 'Self Extubation' then 1
      else 0
      end
      )
      as Extubated
    , max(
      case when itemid is null or value is null then 0
        when itemid = 640 and value = 'Self Extubation' then 1
      else 0
      end
      )
      as SelfExtubated
from chartevents ce
where ce.value is not null
-- exclude rows marked as error
and ce.error IS DISTINCT FROM 1
and itemid in
(
    -- the below are settings used to indicate ventilation
      720, 223849 -- vent mode
    , 223848 -- vent type
    , 445, 448, 449, 450, 1340, 1486, 1600, 224687 -- minute volume
    , 639, 654, 681, 682, 683, 684,224685,224684,224686 -- tidal volume
    , 218,436,535,444,224697,224695,224696,224746,224747 -- High/Low/Peak/Mean ("RespPressure")
    , 221,1,1211,1655,2000,226873,224738,224419,224750,227187 -- Insp pressure
    , 543 -- PlateauPressure
    , 5865,5866,224707,224709,224705,224706 -- APRV pressure
    , 60,437,505,506,686,220339,224700 -- PEEP
    , 3459 -- high pressure relief
    , 501,502,503,224702 -- PCV
    , 223,667,668,669,670,671,672 -- TCPCV
    , 224701 -- PSVlevel

    -- the below are settings used to indicate extubation
    , 640 -- extubated

    -- the below indicate oxygen/NIV, i.e. the end of a mechanical vent event
    , 468 -- O2 Delivery Device#2
    , 469 -- O2 Delivery Mode
    , 470 -- O2 Flow (lpm)
    , 471 -- O2 Flow (lpm) #2
    , 227287 -- O2 Flow (additional cannula)
    , 226732 -- O2 Delivery Device(s)
    , 223834 -- O2 Flow

    -- used in both oxygen + vent calculation
    , 467 -- O2 Delivery Device
)
group by icustay_id, charttime
UNION
-- add in the extubation flags from procedureevents_mv
-- note that we only need the start time for the extubation
-- (extubation is always charted as ending 1 minute after it started)
select
  icustay_id, starttime as charttime
  , 0 as MechVent
  , 0 as OxygenTherapy
  , 1 as Extubated
  , case when itemid = 225468 then 1 else 0 end as SelfExtubated
from procedureevents_mv
where itemid in
(
  227194 -- "Extubation"
, 225468 -- "Unplanned Extubation (patient-initiated)"
, 225477 -- "Unplanned Extubation (non-patient initiated)"
);


--DROP MATERIALIZED VIEW IF EXISTS VENTDURATIONS CASCADE;
DROP MATERIALIZED VIEW IF EXISTS VENTDURATIONS CASCADE;
create MATERIALIZED VIEW ventdurations as
with vd0 as
(
  select
    icustay_id
    -- this carries over the previous charttime which had a mechanical ventilation event
    , case
        when MechVent=1 then
          LAG(CHARTTIME, 1) OVER (partition by icustay_id, MechVent order by charttime)
        else null
      end as charttime_lag
    , charttime
    , MechVent
    , OxygenTherapy
    , Extubated
    , SelfExtubated
  from ventsettings
)
, vd1 as
(
  select
      icustay_id
      , charttime_lag
      , charttime
      , MechVent
      , OxygenTherapy
      , Extubated
      , SelfExtubated

      -- if this is a mechanical ventilation event, we calculate the time since the last event
      , case
          -- if the current observation indicates mechanical ventilation is present
          -- calculate the time since the last vent event
          when MechVent=1 then
            CHARTTIME - charttime_lag
          else null
        end as ventduration

      , LAG(Extubated,1)
      OVER
      (
      partition by icustay_id, case when MechVent=1 or Extubated=1 then 1 else 0 end
      order by charttime
      ) as ExtubatedLag

      -- now we determine if the current mech vent event is a "new", i.e. they've just been intubated
      , case
        -- if there is an extubation flag, we mark any subsequent ventilation as a new ventilation event
          --when Extubated = 1 then 0 -- extubation is *not* a new ventilation event, the *subsequent* row is
          when
            LAG(Extubated,1)
            OVER
            (
            partition by icustay_id, case when MechVent=1 or Extubated=1 then 1 else 0 end
            order by charttime
            )
            = 1 then 1
          -- if patient has initiated oxygen therapy, and is not currently vented, start a newvent
          when MechVent = 0 and OxygenTherapy = 1 then 1
            -- if there is less than 8 hours between vent settings, we do not treat this as a new ventilation event
          when (CHARTTIME - charttime_lag) > interval '8' hour
            then 1
        else 0
        end as newvent
  -- use the staging table with only vent settings from chart events
  FROM vd0 ventsettings
)
, vd2 as
(
  select vd1.*
  -- create a cumulative sum of the instances of new ventilation
  -- this results in a monotonic integer assigned to each instance of ventilation
  , case when MechVent=1 or Extubated = 1 then
      SUM( newvent )
      OVER ( partition by icustay_id order by charttime )
    else null end
    as ventnum
  --- now we convert CHARTTIME of ventilator settings into durations
  from vd1
)
-- create the durations for each mechanical ventilation instance
select icustay_id
  -- regenerate ventnum so it's sequential
  , ROW_NUMBER() over (partition by icustay_id order by ventnum) as ventnum
  , min(charttime) as starttime
  , max(charttime) as endtime
  , extract(epoch from max(charttime)-min(charttime))/60/60 AS duration_hours
from vd2
group by icustay_id, ventnum
having min(charttime) != max(charttime)
-- patient had to be mechanically ventilated at least once
-- i.e. max(mechvent) should be 1
-- this excludes a frequent situation of NIV/oxygen before intub
-- in these cases, ventnum=0 and max(mechvent)=0, so they are ignored
and max(mechvent) = 1
order by icustay_id, ventnum;
"""
cur.execute(query_ventilation_durations)
con.commit()

In [36]:
query_uofirstday= query_schema+"""
DROP MATERIALIZED VIEW IF EXISTS uofirstday CASCADE;
create materialized view uofirstday as
select
  -- patient identifiers
  ie.subject_id, ie.hadm_id, ie.icustay_id

  -- volumes associated with urine output ITEMIDs
  , sum(
      -- we consider input of GU irrigant as a negative volume
      case
        when oe.itemid = 227488 and oe.value > 0 then -1*oe.value
        else oe.value
    end) as UrineOutput
from icustays ie
-- Join to the outputevents table to get urine output
left join outputevents oe
-- join on all patient identifiers
on ie.subject_id = oe.subject_id and ie.hadm_id = oe.hadm_id and ie.icustay_id = oe.icustay_id
-- and ensure the data occurs during the first day
and oe.charttime between ie.intime and (ie.intime + interval '1' day) -- first ICU day
where itemid in
(
-- these are the most frequently occurring urine output observations in CareVue
40055, -- "Urine Out Foley"
43175, -- "Urine ."
40069, -- "Urine Out Void"
40094, -- "Urine Out Condom Cath"
40715, -- "Urine Out Suprapubic"
40473, -- "Urine Out IleoConduit"
40085, -- "Urine Out Incontinent"
40057, -- "Urine Out Rt Nephrostomy"
40056, -- "Urine Out Lt Nephrostomy"
40405, -- "Urine Out Other"
40428, -- "Urine Out Straight Cath"
40086,--	Urine Out Incontinent
40096, -- "Urine Out Ureteral Stent #1"
40651, -- "Urine Out Ureteral Stent #2"

-- these are the most frequently occurring urine output observations in MetaVision
226559, -- "Foley"
226560, -- "Void"
226561, -- "Condom Cath"
226584, -- "Ileoconduit"
226563, -- "Suprapubic"
226564, -- "R Nephrostomy"
226565, -- "L Nephrostomy"
226567, --	Straight Cath
226557, -- R Ureteral Stent
226558, -- L Ureteral Stent
227488, -- GU Irrigant Volume In
227489  -- GU Irrigant/Urine Volume Out
)
group by ie.subject_id, ie.hadm_id, ie.icustay_id
order by ie.subject_id, ie.hadm_id, ie.icustay_id;
"""
cur.execute(query_uofirstday)
con.commit()

In [37]:
query_labsfirstday= query_schema+"""
DROP MATERIALIZED VIEW IF EXISTS labsfirstday CASCADE;
CREATE materialized VIEW labsfirstday AS
SELECT
  pvt.subject_id, pvt.hadm_id, pvt.icustay_id

  , min(CASE WHEN label = 'ANION GAP' THEN valuenum ELSE null END) as ANIONGAP_min
  , max(CASE WHEN label = 'ANION GAP' THEN valuenum ELSE null END) as ANIONGAP_max
  , min(CASE WHEN label = 'ALBUMIN' THEN valuenum ELSE null END) as ALBUMIN_min
  , max(CASE WHEN label = 'ALBUMIN' THEN valuenum ELSE null END) as ALBUMIN_max
  , min(CASE WHEN label = 'BANDS' THEN valuenum ELSE null END) as BANDS_min
  , max(CASE WHEN label = 'BANDS' THEN valuenum ELSE null END) as BANDS_max
  , min(CASE WHEN label = 'BICARBONATE' THEN valuenum ELSE null END) as BICARBONATE_min
  , max(CASE WHEN label = 'BICARBONATE' THEN valuenum ELSE null END) as BICARBONATE_max
  , min(CASE WHEN label = 'BILIRUBIN' THEN valuenum ELSE null END) as BILIRUBIN_min
  , max(CASE WHEN label = 'BILIRUBIN' THEN valuenum ELSE null END) as BILIRUBIN_max
  , min(CASE WHEN label = 'CREATININE' THEN valuenum ELSE null END) as CREATININE_min
  , max(CASE WHEN label = 'CREATININE' THEN valuenum ELSE null END) as CREATININE_max
  , min(CASE WHEN label = 'CHLORIDE' THEN valuenum ELSE null END) as CHLORIDE_min
  , max(CASE WHEN label = 'CHLORIDE' THEN valuenum ELSE null END) as CHLORIDE_max
  , min(CASE WHEN label = 'GLUCOSE' THEN valuenum ELSE null END) as GLUCOSE_min
  , max(CASE WHEN label = 'GLUCOSE' THEN valuenum ELSE null END) as GLUCOSE_max
  , min(CASE WHEN label = 'HEMATOCRIT' THEN valuenum ELSE null END) as HEMATOCRIT_min
  , max(CASE WHEN label = 'HEMATOCRIT' THEN valuenum ELSE null END) as HEMATOCRIT_max
  , min(CASE WHEN label = 'HEMOGLOBIN' THEN valuenum ELSE null END) as HEMOGLOBIN_min
  , max(CASE WHEN label = 'HEMOGLOBIN' THEN valuenum ELSE null END) as HEMOGLOBIN_max
  , min(CASE WHEN label = 'LACTATE' THEN valuenum ELSE null END) as LACTATE_min
  , max(CASE WHEN label = 'LACTATE' THEN valuenum ELSE null END) as LACTATE_max
  , min(CASE WHEN label = 'PLATELET' THEN valuenum ELSE null END) as PLATELET_min
  , max(CASE WHEN label = 'PLATELET' THEN valuenum ELSE null END) as PLATELET_max
  , min(CASE WHEN label = 'POTASSIUM' THEN valuenum ELSE null END) as POTASSIUM_min
  , max(CASE WHEN label = 'POTASSIUM' THEN valuenum ELSE null END) as POTASSIUM_max
  , min(CASE WHEN label = 'PTT' THEN valuenum ELSE null END) as PTT_min
  , max(CASE WHEN label = 'PTT' THEN valuenum ELSE null END) as PTT_max
  , min(CASE WHEN label = 'INR' THEN valuenum ELSE null END) as INR_min
  , max(CASE WHEN label = 'INR' THEN valuenum ELSE null END) as INR_max
  , min(CASE WHEN label = 'PT' THEN valuenum ELSE null END) as PT_min
  , max(CASE WHEN label = 'PT' THEN valuenum ELSE null END) as PT_max
  , min(CASE WHEN label = 'SODIUM' THEN valuenum ELSE null END) as SODIUM_min
  , max(CASE WHEN label = 'SODIUM' THEN valuenum ELSE null end) as SODIUM_max
  , min(CASE WHEN label = 'BUN' THEN valuenum ELSE null end) as BUN_min
  , max(CASE WHEN label = 'BUN' THEN valuenum ELSE null end) as BUN_max
  , min(CASE WHEN label = 'WBC' THEN valuenum ELSE null end) as WBC_min
  , max(CASE WHEN label = 'WBC' THEN valuenum ELSE null end) as WBC_max


FROM
( -- begin query that extracts the data
  SELECT ie.subject_id, ie.hadm_id, ie.icustay_id
  -- here we assign labels to ITEMIDs
  -- this also fuses together multiple ITEMIDs containing the same data
  , CASE
        WHEN itemid = 50868 THEN 'ANION GAP'
        WHEN itemid = 50862 THEN 'ALBUMIN'
        WHEN itemid = 51144 THEN 'BANDS'
        WHEN itemid = 50882 THEN 'BICARBONATE'
        WHEN itemid = 50885 THEN 'BILIRUBIN'
        WHEN itemid = 50912 THEN 'CREATININE'
        WHEN itemid = 50806 THEN 'CHLORIDE'
        WHEN itemid = 50902 THEN 'CHLORIDE'
        WHEN itemid = 50809 THEN 'GLUCOSE'
        WHEN itemid = 50931 THEN 'GLUCOSE'
        WHEN itemid = 50810 THEN 'HEMATOCRIT'
        WHEN itemid = 51221 THEN 'HEMATOCRIT'
        WHEN itemid = 50811 THEN 'HEMOGLOBIN'
        WHEN itemid = 51222 THEN 'HEMOGLOBIN'
        WHEN itemid = 50813 THEN 'LACTATE'
        WHEN itemid = 51265 THEN 'PLATELET'
        WHEN itemid = 50822 THEN 'POTASSIUM'
        WHEN itemid = 50971 THEN 'POTASSIUM'
        WHEN itemid = 51275 THEN 'PTT'
        WHEN itemid = 51237 THEN 'INR'
        WHEN itemid = 51274 THEN 'PT'
        WHEN itemid = 50824 THEN 'SODIUM'
        WHEN itemid = 50983 THEN 'SODIUM'
        WHEN itemid = 51006 THEN 'BUN'
        WHEN itemid = 51300 THEN 'WBC'
        WHEN itemid = 51301 THEN 'WBC'
      ELSE null
    END AS label
  , -- add in some sanity checks on the values
  -- the where clause below requires all valuenum to be > 0, so these are only upper limit checks
    CASE
      WHEN itemid = 50862 and valuenum >    10 THEN null -- g/dL 'ALBUMIN'
      WHEN itemid = 50868 and valuenum > 10000 THEN null -- mEq/L 'ANION GAP'
      WHEN itemid = 51144 and valuenum <     0 THEN null -- immature band forms, %
      WHEN itemid = 51144 and valuenum >   100 THEN null -- immature band forms, %
      WHEN itemid = 50882 and valuenum > 10000 THEN null -- mEq/L 'BICARBONATE'
      WHEN itemid = 50885 and valuenum >   150 THEN null -- mg/dL 'BILIRUBIN'
      WHEN itemid = 50806 and valuenum > 10000 THEN null -- mEq/L 'CHLORIDE'
      WHEN itemid = 50902 and valuenum > 10000 THEN null -- mEq/L 'CHLORIDE'
      WHEN itemid = 50912 and valuenum >   150 THEN null -- mg/dL 'CREATININE'
      WHEN itemid = 50809 and valuenum > 10000 THEN null -- mg/dL 'GLUCOSE'
      WHEN itemid = 50931 and valuenum > 10000 THEN null -- mg/dL 'GLUCOSE'
      WHEN itemid = 50810 and valuenum >   100 THEN null -- % 'HEMATOCRIT'
      WHEN itemid = 51221 and valuenum >   100 THEN null -- % 'HEMATOCRIT'
      WHEN itemid = 50811 and valuenum >    50 THEN null -- g/dL 'HEMOGLOBIN'
      WHEN itemid = 51222 and valuenum >    50 THEN null -- g/dL 'HEMOGLOBIN'
      WHEN itemid = 50813 and valuenum >    50 THEN null -- mmol/L 'LACTATE'
      WHEN itemid = 51265 and valuenum > 10000 THEN null -- K/uL 'PLATELET'
      WHEN itemid = 50822 and valuenum >    30 THEN null -- mEq/L 'POTASSIUM'
      WHEN itemid = 50971 and valuenum >    30 THEN null -- mEq/L 'POTASSIUM'
      WHEN itemid = 51275 and valuenum >   150 THEN null -- sec 'PTT'
      WHEN itemid = 51237 and valuenum >    50 THEN null -- 'INR'
      WHEN itemid = 51274 and valuenum >   150 THEN null -- sec 'PT'
      WHEN itemid = 50824 and valuenum >   200 THEN null -- mEq/L == mmol/L 'SODIUM'
      WHEN itemid = 50983 and valuenum >   200 THEN null -- mEq/L == mmol/L 'SODIUM'
      WHEN itemid = 51006 and valuenum >   300 THEN null -- 'BUN'
      WHEN itemid = 51300 and valuenum >  1000 THEN null -- 'WBC'
      WHEN itemid = 51301 and valuenum >  1000 THEN null -- 'WBC'
    ELSE le.valuenum
    END AS valuenum

  FROM icustays ie

  LEFT JOIN labevents le
    ON le.subject_id = ie.subject_id AND le.hadm_id = ie.hadm_id
    AND le.charttime BETWEEN (ie.intime - interval '6' hour) AND (ie.intime + interval '1' day)
    AND le.ITEMID in
    (
      -- comment is: LABEL | CATEGORY | FLUID | NUMBER OF ROWS IN LABEVENTS
      50868, -- ANION GAP | CHEMISTRY | BLOOD | 769895
      50862, -- ALBUMIN | CHEMISTRY | BLOOD | 146697
      51144, -- BANDS - hematology
      50882, -- BICARBONATE | CHEMISTRY | BLOOD | 780733
      50885, -- BILIRUBIN, TOTAL | CHEMISTRY | BLOOD | 238277
      50912, -- CREATININE | CHEMISTRY | BLOOD | 797476
      50902, -- CHLORIDE | CHEMISTRY | BLOOD | 795568
      50806, -- CHLORIDE, WHOLE BLOOD | BLOOD GAS | BLOOD | 48187
      50931, -- GLUCOSE | CHEMISTRY | BLOOD | 748981
      50809, -- GLUCOSE | BLOOD GAS | BLOOD | 196734
      51221, -- HEMATOCRIT | HEMATOLOGY | BLOOD | 881846
      50810, -- HEMATOCRIT, CALCULATED | BLOOD GAS | BLOOD | 89715
      51222, -- HEMOGLOBIN | HEMATOLOGY | BLOOD | 752523
      50811, -- HEMOGLOBIN | BLOOD GAS | BLOOD | 89712
      50813, -- LACTATE | BLOOD GAS | BLOOD | 187124
      51265, -- PLATELET COUNT | HEMATOLOGY | BLOOD | 778444
      50971, -- POTASSIUM | CHEMISTRY | BLOOD | 845825
      50822, -- POTASSIUM, WHOLE BLOOD | BLOOD GAS | BLOOD | 192946
      51275, -- PTT | HEMATOLOGY | BLOOD | 474937
      51237, -- INR(PT) | HEMATOLOGY | BLOOD | 471183
      51274, -- PT | HEMATOLOGY | BLOOD | 469090
      50983, -- SODIUM | CHEMISTRY | BLOOD | 808489
      50824, -- SODIUM, WHOLE BLOOD | BLOOD GAS | BLOOD | 71503
      51006, -- UREA NITROGEN | CHEMISTRY | BLOOD | 791925
      51301, -- WHITE BLOOD CELLS | HEMATOLOGY | BLOOD | 753301
      51300  -- WBC COUNT | HEMATOLOGY | BLOOD | 2371
    )
    AND valuenum IS NOT null AND valuenum > 0 -- lab values cannot be 0 and cannot be negative
) pvt
GROUP BY pvt.subject_id, pvt.hadm_id, pvt.icustay_id
ORDER BY pvt.subject_id, pvt.hadm_id, pvt.icustay_id;
"""
cur.execute(query_labsfirstday)

In [38]:
query_ventfirstday= query_schema+"""
DROP MATERIALIZED VIEW IF EXISTS ventfirstday CASCADE;
CREATE MATERIALIZED VIEW ventfirstday AS
select
  ie.subject_id, ie.hadm_id, ie.icustay_id
  -- if vd.icustay_id is not null, then they have a valid ventilation event
  -- in this case, we say they are ventilated
  -- otherwise, they are not
  , max(case
      when vd.icustay_id is not null then 1
    else 0 end) as vent
from icustays ie
left join ventdurations vd
  on ie.icustay_id = vd.icustay_id
  and
  (
    -- ventilation duration overlaps with ICU admission -> vented on admission
    (vd.starttime <= ie.intime and vd.endtime >= ie.intime)
    -- ventilation started during the first day
    OR (vd.starttime >= ie.intime and vd.starttime <= ie.intime + interval '1' day)
  )
group by ie.subject_id, ie.hadm_id, ie.icustay_id
order by ie.subject_id, ie.hadm_id, ie.icustay_id;
"""
cur.execute(query_ventfirstday)
con.commit()

In [39]:
query_vitalsfirstday = query_schema+"""
DROP MATERIALIZED VIEW IF EXISTS vitalsfirstday CASCADE;
create materialized view vitalsfirstday as
SELECT pvt.subject_id, pvt.hadm_id, pvt.icustay_id

-- Easier names
, min(case when VitalID = 1 then valuenum else null end) as HeartRate_Min
, max(case when VitalID = 1 then valuenum else null end) as HeartRate_Max
, avg(case when VitalID = 1 then valuenum else null end) as HeartRate_Mean
, min(case when VitalID = 2 then valuenum else null end) as SysBP_Min
, max(case when VitalID = 2 then valuenum else null end) as SysBP_Max
, avg(case when VitalID = 2 then valuenum else null end) as SysBP_Mean
, min(case when VitalID = 3 then valuenum else null end) as DiasBP_Min
, max(case when VitalID = 3 then valuenum else null end) as DiasBP_Max
, avg(case when VitalID = 3 then valuenum else null end) as DiasBP_Mean
, min(case when VitalID = 4 then valuenum else null end) as MeanBP_Min
, max(case when VitalID = 4 then valuenum else null end) as MeanBP_Max
, avg(case when VitalID = 4 then valuenum else null end) as MeanBP_Mean
, min(case when VitalID = 5 then valuenum else null end) as RespRate_Min
, max(case when VitalID = 5 then valuenum else null end) as RespRate_Max
, avg(case when VitalID = 5 then valuenum else null end) as RespRate_Mean
, min(case when VitalID = 6 then valuenum else null end) as TempC_Min
, max(case when VitalID = 6 then valuenum else null end) as TempC_Max
, avg(case when VitalID = 6 then valuenum else null end) as TempC_Mean
, min(case when VitalID = 7 then valuenum else null end) as SpO2_Min
, max(case when VitalID = 7 then valuenum else null end) as SpO2_Max
, avg(case when VitalID = 7 then valuenum else null end) as SpO2_Mean
, min(case when VitalID = 8 then valuenum else null end) as Glucose_Min
, max(case when VitalID = 8 then valuenum else null end) as Glucose_Max
, avg(case when VitalID = 8 then valuenum else null end) as Glucose_Mean

FROM  (
  select ie.subject_id, ie.hadm_id, ie.icustay_id
  , case
    when itemid in (211,220045) and valuenum > 0 and valuenum < 300 then 1 -- HeartRate
    when itemid in (51,442,455,6701,220179,220050) and valuenum > 0 and valuenum < 400 then 2 -- SysBP
    when itemid in (8368,8440,8441,8555,220180,220051) and valuenum > 0 and valuenum < 300 then 3 -- DiasBP
    when itemid in (456,52,6702,443,220052,220181,225312) and valuenum > 0 and valuenum < 300 then 4 -- MeanBP
    when itemid in (615,618,220210,224690) and valuenum > 0 and valuenum < 70 then 5 -- RespRate
    when itemid in (223761,678) and valuenum > 70 and valuenum < 120  then 6 -- TempF, converted to degC in valuenum call
    when itemid in (223762,676) and valuenum > 10 and valuenum < 50  then 6 -- TempC
    when itemid in (646,220277) and valuenum > 0 and valuenum <= 100 then 7 -- SpO2
    when itemid in (807,811,1529,3745,3744,225664,220621,226537) and valuenum > 0 then 8 -- Glucose

    else null end as VitalID
      -- convert F to C
  , case when itemid in (223761,678) then (valuenum-32)/1.8 else valuenum end as valuenum

  from icustays ie
  left join chartevents ce
  on ie.subject_id = ce.subject_id and ie.hadm_id = ce.hadm_id and ie.icustay_id = ce.icustay_id
  and ce.charttime between ie.intime and ie.intime + interval '1' day
  -- exclude rows marked as error
  and ce.error IS DISTINCT FROM 1
  where ce.itemid in
  (
  -- HEART RATE
  211, --"Heart Rate"
  220045, --"Heart Rate"

  -- Systolic/diastolic

  51, --	Arterial BP [Systolic]
  442, --	Manual BP [Systolic]
  455, --	NBP [Systolic]
  6701, --	Arterial BP #2 [Systolic]
  220179, --	Non Invasive Blood Pressure systolic
  220050, --	Arterial Blood Pressure systolic

  8368, --	Arterial BP [Diastolic]
  8440, --	Manual BP [Diastolic]
  8441, --	NBP [Diastolic]
  8555, --	Arterial BP #2 [Diastolic]
  220180, --	Non Invasive Blood Pressure diastolic
  220051, --	Arterial Blood Pressure diastolic


  -- MEAN ARTERIAL PRESSURE
  456, --"NBP Mean"
  52, --"Arterial BP Mean"
  6702, --	Arterial BP Mean #2
  443, --	Manual BP Mean(calc)
  220052, --"Arterial Blood Pressure mean"
  220181, --"Non Invasive Blood Pressure mean"
  225312, --"ART BP mean"

  -- RESPIRATORY RATE
  618,--	Respiratory Rate
  615,--	Resp Rate (Total)
  220210,--	Respiratory Rate
  224690, --	Respiratory Rate (Total)


  -- SPO2, peripheral
  646, 220277,

  -- GLUCOSE, both lab and fingerstick
  807,--	Fingerstick Glucose
  811,--	Glucose (70-105)
  1529,--	Glucose
  3745,--	BloodGlucose
  3744,--	Blood Glucose
  225664,--	Glucose finger stick
  220621,--	Glucose (serum)
  226537,--	Glucose (whole blood)

  -- TEMPERATURE
  223762, -- "Temperature Celsius"
  676,	-- "Temperature C"
  223761, -- "Temperature Fahrenheit"
  678 --	"Temperature F"

  )
) pvt
group by pvt.subject_id, pvt.hadm_id, pvt.icustay_id
order by pvt.subject_id, pvt.hadm_id, pvt.icustay_id;
"""
cur.execute(query_vitalsfirstday)

In [40]:
query_gcsfirstday = query_schema+"""
DROP MATERIALIZED VIEW IF EXISTS gcsfirstday CASCADE;
create materialized view gcsfirstday as
with base as
(
  SELECT pvt.ICUSTAY_ID
  , pvt.charttime

  -- Easier names - note we coalesced Metavision and CareVue IDs below
  , max(case when pvt.itemid = 454 then pvt.valuenum else null end) as GCSMotor
  , max(case when pvt.itemid = 723 then pvt.valuenum else null end) as GCSVerbal
  , max(case when pvt.itemid = 184 then pvt.valuenum else null end) as GCSEyes

  -- If verbal was set to 0 in the below select, then this is an intubated patient
  , case
      when max(case when pvt.itemid = 723 then pvt.valuenum else null end) = 0
    then 1
    else 0
    end as EndoTrachFlag

  , ROW_NUMBER ()
          OVER (PARTITION BY pvt.ICUSTAY_ID ORDER BY pvt.charttime ASC) as rn

  FROM  (
  select l.ICUSTAY_ID
  -- merge the ITEMIDs so that the pivot applies to both metavision/carevue data
  , case
      when l.ITEMID in (723,223900) then 723
      when l.ITEMID in (454,223901) then 454
      when l.ITEMID in (184,220739) then 184
      else l.ITEMID end
    as ITEMID

  -- convert the data into a number, reserving a value of 0 for ET/Trach
  , case
      -- endotrach/vent is assigned a value of 0, later parsed specially
      when l.ITEMID = 723 and l.VALUE = '1.0 ET/Trach' then 0 -- carevue
      when l.ITEMID = 223900 and l.VALUE = 'No Response-ETT' then 0 -- metavision

      else VALUENUM
      end
    as VALUENUM
  , l.CHARTTIME
  from CHARTEVENTS l

  -- get intime for charttime subselection
  inner join icustays b
    on l.icustay_id = b.icustay_id

  -- Isolate the desired GCS variables
  where l.ITEMID in
  (
    -- 198 -- GCS
    -- GCS components, CareVue
    184, 454, 723
    -- GCS components, Metavision
    , 223900, 223901, 220739
  )
  -- Only get data for the first 24 hours
  and l.charttime between b.intime and b.intime + interval '1' day
  -- exclude rows marked as error
  and l.error IS DISTINCT FROM 1
  ) pvt
  group by pvt.ICUSTAY_ID, pvt.charttime
)
, gcs as (
  select b.*
  , b2.GCSVerbal as GCSVerbalPrev
  , b2.GCSMotor as GCSMotorPrev
  , b2.GCSEyes as GCSEyesPrev
  -- Calculate GCS, factoring in special case when they are intubated and prev vals
  -- note that the coalesce are used to implement the following if:
  --  if current value exists, use it
  --  if previous value exists, use it
  --  otherwise, default to normal
  , case
      -- replace GCS during sedation with 15
      when b.GCSVerbal = 0
        then 15
      when b.GCSVerbal is null and b2.GCSVerbal = 0
        then 15
      -- if previously they were intub, but they aren't now, do not use previous GCS values
      when b2.GCSVerbal = 0
        then
            coalesce(b.GCSMotor,6)
          + coalesce(b.GCSVerbal,5)
          + coalesce(b.GCSEyes,4)
      -- otherwise, add up score normally, imputing previous value if none available at current time
      else
            coalesce(b.GCSMotor,coalesce(b2.GCSMotor,6))
          + coalesce(b.GCSVerbal,coalesce(b2.GCSVerbal,5))
          + coalesce(b.GCSEyes,coalesce(b2.GCSEyes,4))
      end as GCS

  from base b
  -- join to itself within 6 hours to get previous value
  left join base b2
    on b.ICUSTAY_ID = b2.ICUSTAY_ID and b.rn = b2.rn+1 and b2.charttime > b.charttime - interval '6' hour
)
, gcs_final as (
  select gcs.*
  -- This sorts the data by GCS, so rn=1 is the the lowest GCS values to keep
  , ROW_NUMBER ()
          OVER (PARTITION BY gcs.ICUSTAY_ID
                ORDER BY gcs.GCS
               ) as IsMinGCS
  from gcs
)
select ie.SUBJECT_ID, ie.HADM_ID, ie.ICUSTAY_ID
-- The minimum GCS is determined by the above row partition, we only join if IsMinGCS=1
, GCS as MinGCS
, coalesce(GCSMotor,GCSMotorPrev) as GCSMotor
, coalesce(GCSVerbal,GCSVerbalPrev) as GCSVerbal
, coalesce(GCSEyes,GCSEyesPrev) as GCSEyes
, EndoTrachFlag as EndoTrachFlag

-- subselect down to the cohort of eligible patients
from icustays ie
left join gcs_final gs
  on ie.ICUSTAY_ID = gs.ICUSTAY_ID and gs.IsMinGCS = 1
ORDER BY ie.ICUSTAY_ID;
"""
cur.execute(query_gcsfirstday)
con.commit()

In [41]:
query_apsIII= query_schema+"""
DROP MATERIALIZED VIEW IF EXISTS APSIII CASCADE;
CREATE MATERIALIZED VIEW APSIII as
with bg as
(
  -- join blood gas to ventilation durations to determine if patient was vent
  -- also join to cpap table for the same purpose
  select bg.icustay_id, bg.charttime
  , PO2 as PaO2
  , AADO2
  -- row number indicating the highest AaDO2
  , case
      when  coalesce(FIO2, fio2_chartevents) is not null
        and vd.icustay_id is not null -- patient is ventilated
        and coalesce(FIO2, fio2_chartevents) >= 50
        then ROW_NUMBER() over (partition by bg.ICUSTAY_ID ORDER BY AADO2 DESC)
      else null end
      as aado2_rn
  , case
      when  coalesce(FIO2, fio2_chartevents) >= 50
          then null
      when vd.icustay_id is not null
          then null
      else ROW_NUMBER() over (partition by bg.ICUSTAY_ID ORDER BY PO2 DESC)
    end as pao2_rn

  from bloodgasfirstdayarterial bg
  left join ventdurations vd
    on bg.icustay_id = vd.icustay_id
    and bg.charttime >= vd.starttime
    and bg.charttime <= vd.endtime
)
-- because ph/pco2 rules are an interaction *within* a blood gas, we calculate them here
-- the worse score is then taken for the final calculation
, acidbase as
(
  select bg.icustay_id
  , ph, pco2 as PACO2
  , case
      when ph is null or pco2 is null then null
      when ph < 7.20 then
        case
          when pco2 < 50 then 12
          else 4
        end
      when ph < 7.30 then
        case
          when pco2 < 30 then 9
          when pco2 < 40 then 6
          when pco2 < 50 then 3
          else 2
        end
      when ph < 7.35 then
        case
          when pco2 < 30 then 9
          when pco2 < 45 then 0
          else 1
        end
      when ph < 7.45 then
        case
          when pco2 < 30 then 5
          when pco2 < 45 then 0
          else 1
        end
      when ph < 7.50 then
        case
          when pco2 < 30 then 5
          when pco2 < 35 then 0
          when pco2 < 45 then 2
          else 12
        end
      when ph < 7.60 then
        case
          when pco2 < 40 then 3
          else 12
        end
      else -- ph >= 7.60
        case
          when pco2 < 25 then 0
          when pco2 < 40 then 3
          else 12
        end
    end as acidbase_score
  from bloodgasfirstdayarterial bg
  where ph is not null and pco2 is not null
)
, acidbase_max as
(
  select icustay_id, acidbase_score, ph, paco2
    -- create integer which indexes maximum value of score with 1
  , case
      when ph is not null and paco2 is not null
        then ROW_NUMBER() over (partition by ICUSTAY_ID ORDER BY ACIDBASE_SCORE DESC)
      else null
    end as acidbase_rn
  from acidbase
)
-- define acute renal failure (ARF) as:
--  creatinine >=1.5 mg/dl
--  and urine output <410 cc/day
--  and no chronic dialysis
, arf as
(
  select ie.icustay_id
    , case
        when labs.creatinine_max >= 1.5
        and  uo.urineoutput < 410
        -- acute renal failure is only coded if the patient is not on chronic dialysis
        -- we use ICD-9 coding of ESRD as a proxy for chronic dialysis
        and  icd.ckd = 0
          then 1
      else 0 end as arf
  from icustays ie
  left join uofirstday uo
    on ie.icustay_id = uo.icustay_id
  left join labsfirstday labs
    on ie.icustay_id = labs.icustay_id
  left join
  (
    select hadm_id
      , max(case
          -- severe kidney failure requiring use of dialysis
          when icd9_code in  ('5854','5855','5856') then 1
          -- we do not include 5859 as that is sometimes coded for acute-on-chronic ARF
        else 0 end)
      as ckd
    from diagnoses_icd
    group by hadm_id
  ) icd
    on ie.hadm_id = icd.hadm_id
)
, cohort as
(
select ie.subject_id, ie.hadm_id, ie.icustay_id
      , ie.intime
      , ie.outtime

      , vital.heartrate_min
      , vital.heartrate_max
      , vital.meanbp_min
      , vital.meanbp_max
      , vital.tempc_min
      , vital.tempc_max
      , vital.resprate_min
      , vital.resprate_max

      , pa.PaO2
      , aa.AaDO2

      , ab.ph
      , ab.paco2
      , ab.acidbase_score

      , labs.hematocrit_min
      , labs.hematocrit_max
      , labs.wbc_min
      , labs.wbc_max
      , labs.creatinine_min
      , labs.creatinine_max
      , labs.bun_min
      , labs.bun_max
      , labs.sodium_min
      , labs.sodium_max
      , labs.albumin_min
      , labs.albumin_max
      , labs.bilirubin_min
      , labs.bilirubin_max

      , case
          when labs.glucose_max is null and vital.glucose_max is null
            then null
          when labs.glucose_max is null or vital.glucose_max > labs.glucose_max
            then vital.glucose_max
          when vital.glucose_max is null or labs.glucose_max > vital.glucose_max
            then labs.glucose_max
          else labs.glucose_max -- if equal, just pick labs
        end as glucose_max

      , case
          when labs.glucose_min is null and vital.glucose_min is null
            then null
          when labs.glucose_min is null or vital.glucose_min < labs.glucose_min
            then vital.glucose_min
          when vital.glucose_min is null or labs.glucose_min < vital.glucose_min
            then labs.glucose_min
          else labs.glucose_min -- if equal, just pick labs
        end as glucose_min

      -- , labs.bicarbonate_min
      -- , labs.bicarbonate_max
      , vent.vent
      , uo.urineoutput
      -- gcs and its components
      , gcs.mingcs
      , gcs.gcsmotor, gcs.gcsverbal,  gcs.gcseyes, gcs.endotrachflag
      -- acute renal failure
      , arf.arf as arf

from icustays ie
inner join admissions adm
  on ie.hadm_id = adm.hadm_id
inner join patients pat
  on ie.subject_id = pat.subject_id

-- join to above views - the row number filters to 1 row per ICUSTAY_ID
left join bg pa
  on  ie.icustay_id = pa.icustay_id
  and pa.pao2_rn = 1
left join bg aa
  on  ie.icustay_id = aa.icustay_id
  and aa.aado2_rn = 1
left join acidbase_max ab
  on  ie.icustay_id = ab.icustay_id
  and ab.acidbase_rn = 1
left join arf
  on ie.icustay_id = arf.icustay_id

-- join to custom tables to get more data....
left join ventfirstday vent
  on ie.icustay_id = vent.icustay_id
left join gcsfirstday gcs
  on ie.icustay_id = gcs.icustay_id
left join vitalsfirstday vital
  on ie.icustay_id = vital.icustay_id
left join uofirstday uo
  on ie.icustay_id = uo.icustay_id
left join labsfirstday labs
  on ie.icustay_id = labs.icustay_id
)
-- First, we calculate the score for the minimum values
, score_min as
(
  select cohort.subject_id, cohort.hadm_id, cohort.icustay_id
  , case
      when heartrate_min is null then null
      when heartrate_min <   40 then 8
      when heartrate_min <   50 then 5
      when heartrate_min <  100 then 0
      when heartrate_min <  110 then 1
      when heartrate_min <  120 then 5
      when heartrate_min <  140 then 7
      when heartrate_min <  155 then 13
      when heartrate_min >= 155 then 17
    end as hr_score

  , case
      when meanbp_min is null then null
      when meanbp_min <   40 then 23
      when meanbp_min <   60 then 15
      when meanbp_min <   70 then 7
      when meanbp_min <   80 then 6
      when meanbp_min <  100 then 0
      when meanbp_min <  120 then 4
      when meanbp_min <  130 then 7
      when meanbp_min <  140 then 9
      when meanbp_min >= 140 then 10
    end as meanbp_score

  -- TODO: add 1 degree to axillary measurements
  , case
      when tempc_min is null then null
      when tempc_min <  33.0 then 20
      when tempc_min <  33.5 then 16
      when tempc_min <  34.0 then 13
      when tempc_min <  35.0 then 8
      when tempc_min <  36.0 then 2
      when tempc_min <  40.0 then 0
      when tempc_min >= 40.0 then 4
    end as temp_score

  , case
      when resprate_min is null then null
      -- special case for ventilated patients
      when vent = 1 and resprate_min < 14 then 0
      when resprate_min <   6 then 17
      when resprate_min <  12 then 8
      when resprate_min <  14 then 7
      when resprate_min <  25 then 0
      when resprate_min <  35 then 6
      when resprate_min <  40 then 9
      when resprate_min <  50 then 11
      when resprate_min >= 50 then 18
    end as resprate_score

  , case
      when hematocrit_min is null then null
      when hematocrit_min <   41.0 then 3
      when hematocrit_min <   50.0 then 0
      when hematocrit_min >=  50.0 then 3
    end as hematocrit_score

  , case
      when wbc_min is null then null
      when wbc_min <   1.0 then 19
      when wbc_min <   3.0 then 5
      when wbc_min <  20.0 then 0
      when wbc_min <  25.0 then 1
      when wbc_min >= 25.0 then 5
    end as wbc_score

  , case
      when creatinine_min is null then null
      when arf = 1 and creatinine_min <  1.5 then 0
      when arf = 1 and creatinine_min >= 1.5 then 10
      when creatinine_min <   0.5 then 3
      when creatinine_min <   1.5 then 0
      when creatinine_min <  1.95 then 4
      when creatinine_min >= 1.95 then 7
    end as creatinine_score

  , case
      when bun_min is null then null
      when bun_min <  17.0 then 0
      when bun_min <  20.0 then 2
      when bun_min <  40.0 then 7
      when bun_min <  80.0 then 11
      when bun_min >= 80.0 then 12
    end as bun_score

  , case
      when sodium_min is null then null
      when sodium_min <  120 then 3
      when sodium_min <  135 then 2
      when sodium_min <  155 then 0
      when sodium_min >= 155 then 4
    end as sodium_score

  , case
      when albumin_min is null then null
      when albumin_min <  2.0 then 11
      when albumin_min <  2.5 then 6
      when albumin_min <  4.5 then 0
      when albumin_min >= 4.5 then 4
    end as albumin_score

  , case
      when bilirubin_min is null then null
      when bilirubin_min <  2.0 then 0
      when bilirubin_min <  3.0 then 5
      when bilirubin_min <  5.0 then 6
      when bilirubin_min <  8.0 then 8
      when bilirubin_min >= 8.0 then 16
    end as bilirubin_score

  , case
      when glucose_min is null then null
      when glucose_min <   40 then 8
      when glucose_min <   60 then 9
      when glucose_min <  200 then 0
      when glucose_min <  350 then 3
      when glucose_min >= 350 then 5
    end as glucose_score

from cohort
)
, score_max as
(
  select cohort.subject_id, cohort.hadm_id, cohort.icustay_id
    , case
        when heartrate_max is null then null
        when heartrate_max <   40 then 8
        when heartrate_max <   50 then 5
        when heartrate_max <  100 then 0
        when heartrate_max <  110 then 1
        when heartrate_max <  120 then 5
        when heartrate_max <  140 then 7
        when heartrate_max <  155 then 13
        when heartrate_max >= 155 then 17
      end as hr_score

    , case
        when meanbp_max is null then null
        when meanbp_max <   40 then 23
        when meanbp_max <   60 then 15
        when meanbp_max <   70 then 7
        when meanbp_max <   80 then 6
        when meanbp_max <  100 then 0
        when meanbp_max <  120 then 4
        when meanbp_max <  130 then 7
        when meanbp_max <  140 then 9
        when meanbp_max >= 140 then 10
      end as meanbp_score

    -- TODO: add 1 degree to axillary measurements
    , case
        when tempc_max is null then null
        when tempc_max <  33.0 then 20
        when tempc_max <  33.5 then 16
        when tempc_max <  34.0 then 13
        when tempc_max <  35.0 then 8
        when tempc_max <  36.0 then 2
        when tempc_max <  40.0 then 0
        when tempc_max >= 40.0 then 4
      end as temp_score

    , case
        when resprate_max is null then null
        -- special case for ventilated patients
        when vent = 1 and resprate_max < 14 then 0
        when resprate_max <   6 then 17
        when resprate_max <  12 then 8
        when resprate_max <  14 then 7
        when resprate_max <  25 then 0
        when resprate_max <  35 then 6
        when resprate_max <  40 then 9
        when resprate_max <  50 then 11
        when resprate_max >= 50 then 18
      end as resprate_score

    , case
        when hematocrit_max is null then null
        when hematocrit_max <   41.0 then 3
        when hematocrit_max <   50.0 then 0
        when hematocrit_max >=  50.0 then 3
      end as hematocrit_score

    , case
        when wbc_max is null then null
        when wbc_max <   1.0 then 19
        when wbc_max <   3.0 then 5
        when wbc_max <  20.0 then 0
        when wbc_max <  25.0 then 1
        when wbc_max >= 25.0 then 5
      end as wbc_score

    , case
        when creatinine_max is null then null
        when arf = 1 and creatinine_max <  1.5 then 0
        when arf = 1 and creatinine_max >= 1.5 then 10
        when creatinine_max <   0.5 then 3
        when creatinine_max <   1.5 then 0
        when creatinine_max <  1.95 then 4
        when creatinine_max >= 1.95 then 7
      end as creatinine_score

    , case
        when bun_max is null then null
        when bun_max <  17.0 then 0
        when bun_max <  20.0 then 2
        when bun_max <  40.0 then 7
        when bun_max <  80.0 then 11
        when bun_max >= 80.0 then 12
      end as bun_score

    , case
        when sodium_max is null then null
        when sodium_max <  120 then 3
        when sodium_max <  135 then 2
        when sodium_max <  155 then 0
        when sodium_max >= 155 then 4
      end as sodium_score

    , case
        when albumin_max is null then null
        when albumin_max <  2.0 then 11
        when albumin_max <  2.5 then 6
        when albumin_max <  4.5 then 0
        when albumin_max >= 4.5 then 4
      end as albumin_score

    , case
        when bilirubin_max is null then null
        when bilirubin_max <  2.0 then 0
        when bilirubin_max <  3.0 then 5
        when bilirubin_max <  5.0 then 6
        when bilirubin_max <  8.0 then 8
        when bilirubin_max >= 8.0 then 16
      end as bilirubin_score

    , case
        when glucose_max is null then null
        when glucose_max <   40 then 8
        when glucose_max <   60 then 9
        when glucose_max <  200 then 0
        when glucose_max <  350 then 3
        when glucose_max >= 350 then 5
      end as glucose_score

from cohort
)
-- Combine together the scores for min/max, using the following rules:
--  1) select the value furthest from a predefined normal value
--  2) if both equidistant, choose the one which gives a worse score
--  3) calculate score for acid-base abnormalities as it requires interactions
-- sometimes the code is a bit redundant, i.e. we know the max would always be furthest from 0
, scorecomp as
(
  select co.*
  -- The rules for APS III require the definition of a "worst" value
  -- This value is defined as whatever value is furthest from a predefined normal
  -- e.g., for heart rate, worst is defined as furthest from 75
  , case
      when heartrate_max is null then null
      when abs(heartrate_max-75) > abs(heartrate_min-75)
        then smax.hr_score
      when abs(heartrate_max-75) < abs(heartrate_min-75)
        then smin.hr_score
      when abs(heartrate_max-75) = abs(heartrate_min-75)
      and  smax.hr_score >= smin.hr_score
        then smax.hr_score
      when abs(heartrate_max-75) = abs(heartrate_min-75)
      and  smax.hr_score < smin.hr_score
        then smin.hr_score
    end as hr_score

  , case
      when meanbp_max is null then null
      when abs(meanbp_max-90) > abs(meanbp_min-90)
        then smax.meanbp_score
      when abs(meanbp_max-90) < abs(meanbp_min-90)
        then smin.meanbp_score
      -- values are equidistant - pick the larger score
      when abs(meanbp_max-90) = abs(meanbp_min-90)
      and  smax.meanbp_score >= smin.meanbp_score
        then smax.meanbp_score
      when abs(meanbp_max-90) = abs(meanbp_min-90)
      and  smax.meanbp_score < smin.meanbp_score
        then smin.meanbp_score
    end as meanbp_score

  , case
      when tempc_max is null then null
      when abs(tempc_max-38) > abs(tempc_min-38)
        then smax.temp_score
      when abs(tempc_max-38) < abs(tempc_min-38)
        then smin.temp_score
      -- values are equidistant - pick the larger score
      when abs(tempc_max-38) = abs(tempc_min-38)
      and  smax.temp_score >= smin.temp_score
        then smax.temp_score
      when abs(tempc_max-38) = abs(tempc_min-38)
      and  smax.temp_score < smin.temp_score
        then smin.temp_score
    end as temp_score

  , case
      when resprate_max is null then null
      when abs(resprate_max-19) > abs(resprate_min-19)
        then smax.resprate_score
      when abs(resprate_max-19) < abs(resprate_min-19)
        then smin.resprate_score
      -- values are equidistant - pick the larger score
      when abs(resprate_max-19) = abs(resprate_max-19)
      and  smax.resprate_score >= smin.resprate_score
        then smax.resprate_score
      when abs(resprate_max-19) = abs(resprate_max-19)
      and  smax.resprate_score < smin.resprate_score
        then smin.resprate_score
    end as resprate_score

  , case
      when hematocrit_max is null then null
      when abs(hematocrit_max-45.5) > abs(hematocrit_min-45.5)
        then smax.hematocrit_score
      when abs(hematocrit_max-45.5) < abs(hematocrit_min-45.5)
        then smin.hematocrit_score
      -- values are equidistant - pick the larger score
      when abs(hematocrit_max-45.5) = abs(hematocrit_max-45.5)
      and  smax.hematocrit_score >= smin.hematocrit_score
        then smax.hematocrit_score
      when abs(hematocrit_max-45.5) = abs(hematocrit_max-45.5)
      and  smax.hematocrit_score < smin.hematocrit_score
        then smin.hematocrit_score
    end as hematocrit_score

  , case
      when wbc_max is null then null
      when abs(wbc_max-11.5) > abs(wbc_min-11.5)
        then smax.wbc_score
      when abs(wbc_max-11.5) < abs(wbc_min-11.5)
        then smin.wbc_score
      -- values are equidistant - pick the larger score
      when abs(wbc_max-11.5) = abs(wbc_max-11.5)
      and  smax.wbc_score >= smin.wbc_score
        then smax.wbc_score
      when abs(wbc_max-11.5) = abs(wbc_max-11.5)
      and  smax.wbc_score < smin.wbc_score
        then smin.wbc_score
    end as wbc_score


  -- For some labs, "furthest from normal" doesn't make sense
  -- e.g. creatinine w/ ARF, the minimum could be 0.3, and the max 1.6
  -- while the minimum of 0.3 is "further from 1", seems like the max should be scored

  , case
      when creatinine_max is null then null
      -- if they have arf then use the max to score
      when arf = 1 then smax.creatinine_score
      -- otherwise furthest from 1
      when abs(creatinine_max-1) > abs(creatinine_min-1)
        then smax.creatinine_score
      when abs(creatinine_max-1) < abs(creatinine_min-1)
        then smin.creatinine_score
      -- values are equidistant
      when smax.creatinine_score >= smin.creatinine_score
        then smax.creatinine_score
      when smax.creatinine_score < smin.creatinine_score
        then smin.creatinine_score
    end as creatinine_score

  -- the rule for BUN is the furthest from 0.. equivalent to the max value
  , case
      when bun_max is null then null
      else smax.bun_score
    end as bun_score

  , case
      when sodium_max is null then null
      when abs(sodium_max-145.5) > abs(sodium_min-145.5)
        then smax.sodium_score
      when abs(sodium_max-145.5) < abs(sodium_min-145.5)
        then smin.sodium_score
      -- values are equidistant - pick the larger score
      when abs(sodium_max-145.5) = abs(sodium_max-145.5)
      and  smax.sodium_score >= smin.sodium_score
        then smax.sodium_score
      when abs(sodium_max-145.5) = abs(sodium_max-145.5)
      and  smax.sodium_score < smin.sodium_score
        then smin.sodium_score
    end as sodium_score

  , case
      when albumin_max is null then null
      when abs(albumin_max-3.5) > abs(albumin_min-3.5)
        then smax.albumin_score
      when abs(albumin_max-3.5) < abs(albumin_min-3.5)
        then smin.albumin_score
      -- values are equidistant - pick the larger score
      when abs(albumin_max-3.5) = abs(albumin_max-3.5)
      and  smax.albumin_score >= smin.albumin_score
        then smax.albumin_score
      when abs(albumin_max-3.5) = abs(albumin_max-3.5)
      and  smax.albumin_score < smin.albumin_score
        then smin.albumin_score
    end as albumin_score

  , case
      when bilirubin_max is null then null
      else smax.bilirubin_score
    end as bilirubin_score

  , case
      when glucose_max is null then null
      when abs(glucose_max-130) > abs(glucose_min-130)
        then smax.glucose_score
      when abs(glucose_max-130) < abs(glucose_min-130)
        then smin.glucose_score
      -- values are equidistant - pick the larger score
      when abs(glucose_max-130) = abs(glucose_max-130)
      and  smax.glucose_score >= smin.glucose_score
        then smax.glucose_score
      when abs(glucose_max-130) = abs(glucose_max-130)
      and  smax.glucose_score < smin.glucose_score
        then smin.glucose_score
    end as glucose_score


  -- Below are interactions/special cases where only 1 value is important
  , case
      when urineoutput is null then null
      when urineoutput <   400 then 15
      when urineoutput <   600 then 8
      when urineoutput <   900 then 7
      when urineoutput <  1500 then 5
      when urineoutput <  2000 then 4
      when urineoutput <  4000 then 0
      when urineoutput >= 4000 then 1
  end as uo_score

  , case
      when endotrachflag = 1
        -- here they are intubated, so their verbal score is inappropriate
        -- normally you are supposed to use "clinical judgement"
        -- we don't have that, so we just assume normal (as was done in the original study)
        then 0
      when gcseyes = 1
        then case
          when gcsverbal = 1 and gcsmotor in (1,2)
            then 48
          when gcsverbal = 1 and gcsmotor in (3,4)
            then 33
          when gcsverbal = 1 and gcsmotor in (5,6)
            then 16
          when gcsverbal in (2,3) and gcsmotor in (1,2)
            then 29
          when gcsverbal in (2,3) and gcsmotor in (3,4)
            then 24
          when gcsverbal in (2,3) and gcsmotor >= 5
            -- highly unlikely clinical combination
            then null
          when gcsverbal >= 4
            then null
          end
      when gcseyes > 1
        then case
          when gcsverbal = 1 and gcsmotor in (1,2)
            then 29
          when gcsverbal = 1 and gcsmotor in (3,4)
            then 24
          when gcsverbal = 1 and gcsmotor in (5,6)
            then 15
          when gcsverbal in (2,3) and gcsmotor in (1,2)
            then 29
          when gcsverbal in (2,3) and gcsmotor in (3,4)
            then 24
          when gcsverbal in (2,3) and gcsmotor = 5
            then 13
          when gcsverbal in (2,3) and gcsmotor = 6
            then 10
          when gcsverbal = 4 and gcsmotor in (1,2,3,4)
            then 13
          when gcsverbal = 4 and gcsmotor = 5
            then 8
          when gcsverbal = 4 and gcsmotor = 6
            then 3
          when gcsverbal = 5 and gcsmotor in (1,2,3,4,5)
            then 3
          when gcsverbal = 5 and gcsmotor = 6
            then 0
          end
      else null
    end as gcs_score

  , case
      when PaO2 is null and AaDO2 is null
        then null
      when PaO2 is not null then
        case
          when PaO2 < 50 then 15
          when PaO2 < 70 then 5
          when PaO2 < 80 then 2
        else 0 end
      when AaDO2 is not null then
        case
          when AaDO2 <  100 then 0
          when AaDO2 <  250 then 7
          when AaDO2 <  350 then 9
          when AaDO2 <  500 then 11
          when AaDO2 >= 500 then 14
        else 0 end
      end as pao2_aado2_score

from cohort co
left join score_min smin
  on co.icustay_id = smin.icustay_id
left join score_max smax
  on co.icustay_id = smax.icustay_id
)
-- tabulate the APS III using the scores from the worst values
, score as
(
  select s.*
  -- coalesce statements impute normal score of zero if data element is missing
  , coalesce(hr_score,0)
  + coalesce(meanbp_score,0)
  + coalesce(temp_score,0)
  + coalesce(resprate_score,0)
  + coalesce(pao2_aado2_score,0)
  + coalesce(hematocrit_score,0)
  + coalesce(wbc_score,0)
  + coalesce(creatinine_score,0)
  + coalesce(uo_score,0)
  + coalesce(bun_score,0)
  + coalesce(sodium_score,0)
  + coalesce(albumin_score,0)
  + coalesce(bilirubin_score,0)
  + coalesce(glucose_score,0)
  + coalesce(acidbase_score,0)
  + coalesce(gcs_score,0)
    as APSIII
  from scorecomp s
)
select ie.subject_id, ie.hadm_id, ie.icustay_id
, APSIII
-- Calculate probability of hospital mortality using equation from Johnson 2014.
, 1 / (1 + exp(- (-4.4360 + 0.04726*(APSIII) ))) as APSIII_PROB
, hr_score
, meanbp_score
, temp_score
, resprate_score
, pao2_aado2_score
, hematocrit_score
, wbc_score
, creatinine_score
, uo_score
, bun_score
, sodium_score
, albumin_score
, bilirubin_score
, glucose_score
, acidbase_score
, gcs_score
from icustays ie
left join score s
  on ie.icustay_id = s.icustay_id
order by ie.icustay_id;
"""
cur.execute(query_apsIII)
con.commit()

In [42]:
query_pre_rrt = query_schema+"""
DROP MATERIALIZED VIEW IF EXISTS pre_rrt CASCADE;
create materialized view pre_rrt as
with cv as
(
  select ie.icustay_id
    , max(
        case
          when ce.itemid in (152,148,149,146,147,151,150) and value is not null then 1
          when ce.itemid in (229,235,241,247,253,259,265,271) and value = 'Dialysis Line' then 1
          when ce.itemid = 582 and value in ('CAVH Start','CAVH D/C','CVVHD Start','CVVHD D/C','Hemodialysis st','Hemodialysis end') then 1
        else 0 end
        ) as RRT
  from icustays ie
  inner join chartevents ce
    on ie.icustay_id = ce.icustay_id
    and ce.itemid in
    (
       152 -- "Dialysis Type";61449
      ,148 -- "Dialysis Access Site";60335
      ,149 -- "Dialysis Access Type";60030
      ,146 -- "Dialysate Flow ml/hr";57445
      ,147 -- "Dialysate Infusing";56605
      ,151 -- "Dialysis Site Appear";37345
      ,150 -- "Dialysis Machine";27472
      ,229 -- INV Line#1 [Type]
      ,235 -- INV Line#2 [Type]
      ,241 -- INV Line#3 [Type]
      ,247 -- INV Line#4 [Type]
      ,253 -- INV Line#5 [Type]
      ,259 -- INV Line#6 [Type]
      ,265 -- INV Line#7 [Type]
      ,271 -- INV Line#8 [Type]
      ,582 -- Procedures
    )
    and ce.value is not null
    and EXTRACT(EPOCH FROM ce.charttime - ie.intime)<0
  where ie.dbsource = 'carevue'
  group by ie.icustay_id
)
, mv_ce as
(
  select ie.icustay_id
    , 1 as RRT
  from icustays ie
  inner join chartevents ce
    on ie.icustay_id = ce.icustay_id
    and EXTRACT(EPOCH FROM ce.charttime - ie.intime)<0
    and itemid in
    (
      -- Checkboxes
        226118 -- | Dialysis Catheter placed in outside facility      | Access Lines - Invasive | chartevents        | Checkbox
      , 227357 -- | Dialysis Catheter Dressing Occlusive              | Access Lines - Invasive | chartevents        | Checkbox
      , 225725 -- | Dialysis Catheter Tip Cultured                    | Access Lines - Invasive | chartevents        | Checkbox
      -- Numeric values
      , 226499 -- | Hemodialysis Output                               | Dialysis                | chartevents        | Numeric
      , 224154 -- | Dialysate Rate                                    | Dialysis                | chartevents        | Numeric
      , 225810 -- | Dwell Time (Peritoneal Dialysis)                  | Dialysis                | chartevents        | Numeric
      , 227639 -- | Medication Added Amount  #2 (Peritoneal Dialysis) | Dialysis                | chartevents        | Numeric
      , 225183 -- | Current Goal                     | Dialysis | chartevents        | Numeric
      , 227438 -- | Volume not removed               | Dialysis | chartevents        | Numeric
      , 224191 -- | Hourly Patient Fluid Removal     | Dialysis | chartevents        | Numeric
      , 225806 -- | Volume In (PD)                   | Dialysis | chartevents        | Numeric
      , 225807 -- | Volume Out (PD)                  | Dialysis | chartevents        | Numeric
      , 228004 -- | Citrate (ACD-A)                  | Dialysis | chartevents        | Numeric
      , 228005 -- | PBP (Prefilter) Replacement Rate | Dialysis | chartevents        | Numeric
      , 228006 -- | Post Filter Replacement Rate     | Dialysis | chartevents        | Numeric
      , 224144 -- | Blood Flow (ml/min)              | Dialysis | chartevents        | Numeric
      , 224145 -- | Heparin Dose (per hour)          | Dialysis | chartevents        | Numeric
      , 224149 -- | Access Pressure                  | Dialysis | chartevents        | Numeric
      , 224150 -- | Filter Pressure                  | Dialysis | chartevents        | Numeric
      , 224151 -- | Effluent Pressure                | Dialysis | chartevents        | Numeric
      , 224152 -- | Return Pressure                  | Dialysis | chartevents        | Numeric
      , 224153 -- | Replacement Rate                 | Dialysis | chartevents        | Numeric
      , 224404 -- | ART Lumen Volume                 | Dialysis | chartevents        | Numeric
      , 224406 -- | VEN Lumen Volume                 | Dialysis | chartevents        | Numeric
      , 226457 -- | Ultrafiltrate Output             | Dialysis | chartevents        | Numeric
    )
    and valuenum > 0 -- also ensures it's not null
  group by ie.icustay_id
)
, mv_ie as
(
  select ie.icustay_id
    , 1 as RRT
  from icustays ie
  inner join inputevents_mv tt
    on ie.icustay_id = tt.icustay_id
    and EXTRACT(EPOCH FROM tt.starttime - ie.intime)<0
    and itemid in
    (
        227536 --	KCl (CRRT)	Medications	inputevents_mv	Solution
      , 227525 --	Calcium Gluconate (CRRT)	Medications	inputevents_mv	Solution
    )
    and amount > 0 -- also ensures it's not null
  group by ie.icustay_id
)
, mv_de as
(
  select ie.icustay_id
    , 1 as RRT
  from icustays ie
  inner join datetimeevents tt
    on ie.icustay_id = tt.icustay_id
    and EXTRACT(EPOCH FROM tt.charttime - ie.intime)<0
    and itemid in
    (
      -- TODO: unsure how to handle "Last dialysis"
      --  225128 -- | Last dialysis                                     | Adm History/FHPA        | datetimeevents     | Date time
        225318 -- | Dialysis Catheter Cap Change                      | Access Lines - Invasive | datetimeevents     | Date time
      , 225319 -- | Dialysis Catheter Change over Wire Date           | Access Lines - Invasive | datetimeevents     | Date time
      , 225321 -- | Dialysis Catheter Dressing Change                 | Access Lines - Invasive | datetimeevents     | Date time
      , 225322 -- | Dialysis Catheter Insertion Date                  | Access Lines - Invasive | datetimeevents     | Date time
      , 225324 -- | Dialysis CatheterTubing Change                    | Access Lines - Invasive | datetimeevents     | Date time
    )
  group by ie.icustay_id
)
, mv_pe as
(
    select ie.icustay_id
      , 1 as RRT
    from icustays ie
    inner join procedureevents_mv tt
      on ie.icustay_id = tt.icustay_id
      and EXTRACT(EPOCH FROM tt.starttime - ie.intime)<0
      and itemid in
      (
          225441 -- | Hemodialysis                                      | 4-Procedures            | procedureevents_mv | Process
        , 225802 -- | Dialysis - CRRT                                   | Dialysis                | procedureevents_mv | Process
        , 225803 -- | Dialysis - CVVHD                                  | Dialysis                | procedureevents_mv | Process
        , 225805 -- | Peritoneal Dialysis                               | Dialysis                | procedureevents_mv | Process
        , 224270 -- | Dialysis Catheter                                 | Access Lines - Invasive | procedureevents_mv | Process
        , 225809 -- | Dialysis - CVVHDF                                 | Dialysis                | procedureevents_mv | Process
        , 225955 -- | Dialysis - SCUF                                   | Dialysis                | procedureevents_mv | Process
        , 225436 -- | CRRT Filter Change               | Dialysis | procedureevents_mv | Process
      )
    group by ie.icustay_id
)
select ie.subject_id, ie.hadm_id, ie.icustay_id
  , case
      when cv.RRT = 1 then 1
      when mv_ce.RRT = 1 then 1
      when mv_ie.RRT = 1 then 1
      when mv_de.RRT = 1 then 1
      when mv_pe.RRT = 1 then 1
      else 0
    end as pre_RRT
from icustays ie
left join cv
  on ie.icustay_id = cv.icustay_id
left join mv_ce
  on ie.icustay_id = mv_ce.icustay_id
left join mv_ie
  on ie.icustay_id = mv_ie.icustay_id
left join mv_de
  on ie.icustay_id = mv_de.icustay_id
left join mv_pe
  on ie.icustay_id = mv_pe.icustay_id
order by ie.icustay_id;
"""
cur.execute(query_pre_rrt)
con.commit()

In [44]:
query_twcl = query_schema+"""
DROP MATERIALIZED VIEW IF EXISTS twcl CASCADE;
create materialized view twcl as
with t1 as 
(
        select i.icustay_id, l.charttime,l.valuenum,i.intime
        from icustays i
        join labo l
        on i.subject_id = l.subject_id
        where l.label = 'CHLORIDE' 
            and l.valuenum is not null 
            and l.charttime between i.intime and (i.intime + interval '2' day)    
        order by i.icustay_id,l.charttime
)

, t2 as
(
select *
,lag(charttime) over (order by icustay_id,charttime) as beforetime
,lag(valuenum) over (order by icustay_id,charttime) as beforevalue
,row_number() over (partition by icustay_id order by charttime) as rt
from t1
)

,t3 as
(
Select *
,EXTRACT(EPOCH FROM charttime - beforetime)/60.0 as diff_minu
from t2
)

,t4 as
(
select *
,((valuenum + beforevalue)/2)*diff_minu as valu_min
from t3 
where rt != 1
)

, t5 as
(
select icustay_id
, sum(valu_min) as valu_sum
from t4
group by icustay_id
)

, time as
(
select icustay_id
,EXTRACT(EPOCH FROM (max(charttime)-min(charttime)))/60 as TotalTimeDiff,
            count(*) as count
        from t1
        group by icustay_id
)

select
t5.icustay_id, time.count
,case when time.count!=1 then t5.valu_sum / time.totaltimediff
 else valu_sum end as twcl
from t5
left join time
on t5.icustay_id = time.icustay_id
where time.totaltimediff!=0
"""
cur.execute(query_twcl)
con.commit()

In [45]:
query_twna = query_schema+"""
DROP MATERIALIZED VIEW IF EXISTS twna CASCADE;
create materialized view twna as
with t1 as 
(
        select i.icustay_id, l.charttime,l.valuenum,i.intime
        from icustays i
        join labo l
        on i.subject_id = l.subject_id
        where l.label = 'SODIUM' 
            and l.valuenum is not null 
            and l.charttime between i.intime and (i.intime + interval '2' day)    
        order by i.icustay_id,l.charttime
)

, t2 as
(
select *
,lag(charttime) over (order by icustay_id,charttime) as beforetime
,lag(valuenum) over (order by icustay_id,charttime) as beforevalue
,row_number() over (partition by icustay_id order by charttime) as rt
from t1
)

,t3 as
(
Select *
,EXTRACT(EPOCH FROM charttime - beforetime)/60.0 as diff_minu
from t2
)

,t4 as
(
select *
,((valuenum + beforevalue)/2)*diff_minu as valu_min
from t3 
where rt != 1
)

, t5 as
(
select icustay_id
, sum(valu_min) as valu_sum
from t4
group by icustay_id
)

, time as
(
select icustay_id
,EXTRACT(EPOCH FROM (max(charttime)-min(charttime)))/60 as TotalTimeDiff,
            count(*) as count
        from t1
        group by icustay_id
)

select
t5.icustay_id, time.count
,case when time.count!=1 then t5.valu_sum / time.totaltimediff
 else valu_sum end as twna
from t5
left join time
on t5.icustay_id = time.icustay_id
where time.totaltimediff!=0
"""
cur.execute(query_twna)
con.commit()

In [46]:
query_twsid = query_schema+"""
DROP MATERIALIZED VIEW IF EXISTS twsid CASCADE;
create materialized view twsid as
with cl as
(
select
    ie.icustay_id
  , ie.intime, ie.outtime
  , la.valuenum as chloride
  , la.charttime
  from icustays ie
  left join labo la
    on ie.subject_id = la.subject_id
    and la.label = 'CHLORIDE'
    and la.VALUENUM is not null
    and la.CHARTTIME between ie.intime and (ie.intime + interval '2' day)
)

, na as 
(
select
    ie.icustay_id
  , ie.intime, ie.outtime
  , la.valuenum as sodium
  , la.charttime
  , la.subject_id
  from icustays ie
  left join labo la
    on ie.subject_id = la.subject_id
    and la.label = 'SODIUM'
    and la.VALUENUM is not null
    and la.CHARTTIME between ie.intime  and (ie.intime + interval '2' day)
)

,sid as
(
select
    na.icustay_id
  , na.subject_id
  , na.sodium
  , na.charttime
  , cl.chloride
  , na.sodium - cl.chloride as sid
  from na
  INNER JOIN cl
    ON na.icustay_id= cl.icustay_id
    AND na.charttime= cl.charttime
  where na.charttime <= na.intime + interval '48' hour
)


, t1 as 
(
        select i.icustay_id, l.charttime,l.sid,i.intime
        from icustays i
        join sid l
        on i.subject_id = l.subject_id
            and l.sid is not null 
            and l.charttime between i.intime and (i.intime + interval '2' day)    
        order by i.icustay_id,l.charttime
)

, t2 as
(
select *
,lag(charttime) over (order by icustay_id,charttime) as beforetime
,lag(sid) over (order by icustay_id,charttime) as beforevalue
,row_number() over (partition by icustay_id order by charttime) as rt
from t1
)

,t3 as
(
Select *
,EXTRACT(EPOCH FROM charttime - beforetime)/60.0 as diff_minu
from t2
)

,t4 as
(
select *
,((sid + beforevalue)/2)*diff_minu as valu_min
from t3 
where rt != 1
)

, t5 as
(
select icustay_id
, sum(valu_min) as valu_sum
from t4
group by icustay_id
)

, time as
(
select icustay_id
,EXTRACT(EPOCH FROM (max(charttime)-min(charttime)))/60 as TotalTimeDiff,
            count(*) as count
        from t1
        group by icustay_id
)

select
t5.icustay_id, time.count
,case when time.count!=1 then t5.valu_sum / time.totaltimediff
 else valu_sum end as twsid
from t5
left join time
on t5.icustay_id = time.icustay_id
where time.totaltimediff!=0
"""
cur.execute(query_twsid)
con.commit()

In [52]:
# Using creatinine before admission
df_1=pd.read_sql(query_schema+"""
select kd.subject_id,kd.hadm_id,kd.icustay_id,kd.gender,kd.dod,kd.dod_hosp,kd.expire_flag
,kd.los_hospital,kd.admission_age,kd.hospital_expire_flag,kd.hospstay_seq
,kd.first_hosp_stay,kd.los_icu,kd.icustay_seq,kd.first_icu_stay,kd.aki_48hr
,kd.aki_stage_48hr,kd.highcreat48hr,kd.aki_7day,kd.aki_stage_7day,kd.highcreat7day, kd.sum_icu_surge
,kd.BaseCre, kd.BaseCreTime, kd.rrt_2d, kd.rrt_7d
,su.admcl,su.lowcl48hr,su.highcl48hr,su.admna,su.lowna48hr,su.highna48hr
,su.admsid,su.lowsid48hr,su.highsid48hr,su.admbe,su.lowbe48hr,su.highbe48hr
,su.admbecl,su.lowbecl48hr,su.highbecl48hr,su.admcre
,co.congestive_heart_failure,co.hypertension,co.diabetes_uncomplicated
,co.diabetes_complicated,co.renal_failure,co.liver_disease,co.lymphoma
,co.metastatic_cancer,co.solid_tumor
,fl.total_crystalloid_bolus,fl.total_chloride_meq_sum, fl.total_sodium_meq_sum
,ap.apsiii, pr.pre_rrt, cl.twcl, na.twna, sid.twsid
from new_kdigo kd
left join summary_lab su
on kd.icustay_id = su.icustay_id
left join comorbidity co
on kd.hadm_id = co.hadm_id
left join new_fluid_intake_sum fl
on kd.icustay_id = fl.icustay_id
left join APSIII ap
on kd.icustay_id = ap.icustay_id
left join pre_rrt pr
on kd.icustay_id = pr.icustay_id
left join twcl cl
on kd.icustay_id = cl.icustay_id
left join twna na
on kd.icustay_id = na.icustay_id
left join twsid sid
on kd.icustay_id = sid.icustay_id
""", con)

In [53]:
df_1.to_csv("chloride.csv",index=False,sep='\t')

In [ ]:
pd.read_sql(query_schema+"""
select *
from labo
limit 10
""",con)